In [1]:


import torch
print(torch.__version__)
import pandas as pd
print(pd.__version__)
from torch.utils.data import DataLoader,ConcatDataset
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error



ModuleNotFoundError: No module named 'torch'

In [1]:


import torch
print(torch.__version__)
import pandas as pd
print(pd.__version__)
from torch.utils.data import DataLoader,ConcatDataset
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error



1.8.1
1.2.4


In [2]:

# ## Read pesticides and fertilizer products data


In [3]:
def load_data():
       pesticides_frame = pd.read_csv("Inputs_Pesticides_Use_E_All_Data_NOFLAG.csv", engine='python')
       pesticides_frame.head
  
       # Read fertilizer data in the form of products
       fertilizers_frame = pd.read_csv("Inputs_FertilizersProduct_E_All_Data_NOFLAG.csv", engine='python')
       fertilizers_frame.head
       
       # Read the crop yield data
       yield_frame = pd.read_csv("Production_Crops_E_All_Data_NOFLAG.csv",engine='python')
       yield_frame.head

       # %<br>
       # Get the fertilizer usage in terms of agriculture

       # In[4]:     
       print("Fertilizer products",fertilizers_frame.columns)
       print("\n")
       print("Input pesticides", pesticides_frame.columns)
       
       # Filter out the data for Australia
       is_Country_Australia_Products = fertilizers_frame['Area']=="Australia"
       Australia_Fertilizers_Products = fertilizers_frame[is_Country_Australia_Products]
       # In[5]:
       is_Country_Australia_Pesticides = pesticides_frame['Area']=="Australia"
       Australia_Pesticides = pesticides_frame[is_Country_Australia_Pesticides]

       # Get the fertilizer usage for agricultural use
       # In[6]:
       is_Agricultural = Australia_Fertilizers_Products['Element']=="Agricultural Use"
       Australia_Fertilizers_Products_Agricultural = Australia_Fertilizers_Products[is_Agricultural]
       print("Fertilizers \n")
       print(Australia_Fertilizers_Products_Agricultural.columns)
       print(Australia_Fertilizers_Products_Agricultural['Item'])
       print("\n")
       print("Pesticides \n")
       print(Australia_Pesticides.columns)
       print(Australia_Pesticides['Item'])
       
       # %<br>
       # 7 years for training - 2007 to 2013<br>
       # 3 years fro validation - 2014 to 2016<br>
       # 2017 for testing
       # In[7]:
       X_ten_years_fertlizers = Australia_Fertilizers_Products_Agricultural.drop(['Area Code', 'Area', 'Item Code', 'Element Code','Element',
              'Unit','Y2002', 'Y2003', 'Y2004', 'Y2005', 'Y2006', 'Y2017'], axis=1)
       print(X_ten_years_fertlizers)
       print(X_ten_years_fertlizers.columns)
       fertilizer_2017 = Australia_Fertilizers_Products_Agricultural['Y2017']
       print("2017 \n",fertilizer_2017)

       # In[8]:
       X_ten_years_pesticides = Australia_Pesticides.drop(['Area Code', 'Area', 'Item Code', 'Element Code', 'Element',
              'Unit', 'Y1990', 'Y1991', 'Y1992', 'Y1993', 'Y1994', 'Y1995', 'Y1996', 'Y1997', 'Y1998', 'Y1999', 'Y2000', 'Y2001', 'Y2002', 'Y2003', 'Y2004', 'Y2005', 'Y2006', 'Y2017'], axis=1)
       print(X_ten_years_pesticides)
       print(X_ten_years_pesticides.columns)
       pesticides_2017 = Australia_Pesticides['Y2017']
       print("2017 \n",pesticides_2017)

       # %<br>
       # Drop Ammonia anyhydrous and Ammonia nitrate from 2007 to 2013. It has not been used in these years.
       # In[9]:
       X_ten_years_fertlizers = X_ten_years_fertlizers.drop([602, 607], axis=0)
       print(X_ten_years_fertlizers)
       print(X_ten_years_fertlizers.columns)
       fertilizer_2017 = fertilizer_2017.drop([602, 607], axis=0)
       print(fertilizer_2017)
       
       # %<br>
       # 2009 to 2013
       # ## Handle missing values from 2009 to 2013
       # In[10]:
       interpolate_2009_to_2016 = X_ten_years_fertlizers.interpolate()
       interpolate_2009_to_2016
       interpolate_2017 = fertilizer_2017.interpolate()
       print("interpolated 2017 fertilizers \n",interpolate_2017)
       # TODO: Don't just interpolate. Use the means of the previous 2  years
       # interpolate_pesticides_2017 = pesticides_2017.interpolate()
       # print("interpolated 2017 pesticides \n",interpolate_pesticides_2017)

       # %<br>
       #  Interpolate missing fertlizers data<br>
       #  Replace na with mean values of 2004,2005,2009,2010
       # ##  Replace 2007 nan values with the mean of 2004 and 2005

       # In[11]:

       interpolate_2004_2005 = Australia_Fertilizers_Products_Agricultural[['Y2004','Y2005']].copy().interpolate()
       index_names = interpolate_2004_2005.index
       index_names
       interpolate_2004_2005=interpolate_2004_2005.drop([602,607], axis=0)
       interpolate_2004_2005['mean'] = interpolate_2004_2005.mean(axis=1)
       print("Mean of 2004 and 2005 ==== \n")
       print(interpolate_2004_2005)
       pesticides_2015_2016 = Australia_Pesticides[['Y2015','Y2016']]
       pesticides_2015_2016['mean'] = pesticides_2015_2016.mean(axis=1)
       print("Mean of 2015 and 2016 === \n")
       print(pesticides_2015_2016)

       # Use this for 2007

       # In[ ]:
       # ##  Replace 2008 nan values with the mean of 2009 and 2010

       # In[12]:
       interpolate_2009_2010 = Australia_Fertilizers_Products_Agricultural[['Y2009','Y2010']].copy().interpolate()
       index_names = interpolate_2009_2010.index
       index_names

       # In[13]:
       interpolate_2009_2010['mean'] = interpolate_2009_2010.mean(axis=1)
       print("Mean of 2009 and 2010 ==== \n")
       # Use this for 2008
       interpolate_2009_2010=interpolate_2009_2010.drop([602,607], axis=0)
       print(interpolate_2009_2010)


       # ## Populate 2007 nan values with computed mean
       # In[14]:
       # Fertilizers
       i=0
       for i in range(1,16):
              interpolate_2009_to_2016.iloc[0:i,1:2] = interpolate_2004_2005['mean'].iloc[0:i]

       #Pesticides
       j=0
       for j in range(1,7):
              pesticides_2017.iloc[0:j,] = pesticides_2015_2016['mean'].iloc[0:j]

       # ## Populate 2008 nan values with computed mean
       # In[15]:
       j=0
       for j in range(1,16):
              interpolate_2009_to_2016.iloc[0:j,2:3] = interpolate_2009_2010['mean'].iloc[0:j]
                
       interpolate_2009_to_2016.iloc[15, 1] = 1437658.50
       print(interpolate_2009_to_2016.iloc[15, 1])
       interpolate_2009_to_2016.iloc[15, 2] = 171351.00

       # In[16]:
       X_ten_years_fertilizers_interpolated = interpolate_2009_to_2016
       X_ten_years_fertilizers_interpolated
       fertilizer_2017_interpolated = interpolate_2017
       print(fertilizer_2017_interpolated)

       # #### TODO: Combine pesticdes and fertilizers data
       # In[17]:
       print(X_ten_years_pesticides)
       print(X_ten_years_fertilizers_interpolated)

       pesticides_without_Item = X_ten_years_pesticides.copy().drop(['Item'], axis=1)
       pesticides_2017_wo_item = pesticides_2017
       fertilizers_without_Item = X_ten_years_fertilizers_interpolated.copy().drop(['Item'], axis=1)
       fertilizer_2017_wo_item = fertilizer_2017_interpolated
       # torch.tensor(X_ten_years_pesticides.values.astype(np.float64))
       print(X_ten_years_pesticides.values)
       print(X_ten_years_fertilizers_interpolated.values)
       print(pesticides_2017_wo_item.values)
       print(fertilizer_2017_wo_item.values)

       print(pesticides_without_Item)
       pesticides_tensor = torch.from_numpy(pesticides_without_Item.values)
       print("\t Pesticides tensor", pesticides_tensor)
       pesticides_2017_tensor = torch.from_numpy(pesticides_2017_wo_item.values)

       print(fertilizers_without_Item)
       fertilizers_tensor = torch.from_numpy(fertilizers_without_Item.values)
       fertilizer_2017_tensor = torch.from_numpy(fertilizer_2017_wo_item.values)
       print("\t Fertilizers tensor", fertilizers_tensor)

       # In[18]:
       print("Train : \n",pesticides_tensor.shape)
       print("Train: \n",fertilizers_tensor.shape)
       print("Test: \n",pesticides_2017_tensor.shape)
       print("Test: \n",fertilizer_2017_tensor.shape)

       # TODO: Normalize inputs before concatenation
       pesticides_and_fertilizers = torch.cat((pesticides_tensor, fertilizers_tensor), dim=0)
       print(pesticides_and_fertilizers.shape)
       print("\n")
       print("Training X data for 10 years: ",pesticides_and_fertilizers)
       print("\n")
       pesticides_and_fertilizers_2017 = torch.cat((pesticides_2017_tensor, fertilizer_2017_tensor), dim=0)
       print(pesticides_and_fertilizers_2017.shape)
       print("\n")
       print("Testing X data for 1 year: ",pesticides_and_fertilizers_2017)
       print("\n")

       # Training + Validation data : Input
       X_train_ten = pesticides_and_fertilizers;
       X_test = pesticides_and_fertilizers_2017

       # #### TODO: Create output vector

       # In[19]:
       print(yield_frame.columns)
       #%%
       yield_frame.head
       #%%
       Australia = yield_frame['Area']=="Australia"
       Production = yield_frame['Element']=="Production"
       Australia_yield = yield_frame[Australia]
       Australia_yield_production = Australia_yield[Production]
       print(yield_frame[Australia])
       #%%
       # Get the yield data for the year 2017 only
       Australia_yield_2017 = Australia_yield_production.drop(['Area Code', 'Area', 'Item Code', 'Element Code','Element',
              'Unit','Y1961', 'Y1962', 'Y1963', 'Y1964', 'Y1965', 'Y1966', 'Y1967', 'Y1968', 'Y1969', 'Y1970', 'Y1971', 'Y1972', 'Y1973', 
              'Y1974', 'Y1975', 'Y1976', 'Y1977', 'Y1978', 'Y1979', 'Y1980', 'Y1981', 'Y1982', 'Y1983', 'Y1984', 'Y1985', 'Y1986', 'Y1987',
              'Y1988', 'Y1989', 'Y1990', 'Y1991', 'Y1992', 'Y1993', 'Y1994', 'Y1995', 'Y1996', 'Y1997', 'Y1998', 'Y1999', 'Y2000', 'Y2001', 
              'Y2002', 'Y2003', 'Y2004', 'Y2005', 'Y2006', 'Y2007', 'Y2008', 'Y2009', 'Y2010', 'Y2011', 'Y2012', 'Y2013', 'Y2014', 'Y2015', 
              'Y2016', 'Y2018'], axis=1)
       print("\n")
       print("Test Y data: ",Australia_yield_2017)
       print("\n")
       print(Australia_yield_2017.columns)
       print(Australia_yield_2017['Item'].unique())
       print("Number of unique crops in 2017: ",len(Australia_yield_2017['Item'].unique()))

       Australia_yield_2007_to_2016 = Australia_yield_production.drop(['Area Code', 'Area', 'Item Code', 'Element Code','Element',
              'Unit','Y1961', 'Y1962', 'Y1963', 'Y1964', 'Y1965', 'Y1966', 'Y1967', 'Y1968', 'Y1969', 'Y1970', 'Y1971', 'Y1972', 'Y1973', 
              'Y1974', 'Y1975', 'Y1976', 'Y1977', 'Y1978', 'Y1979', 'Y1980', 'Y1981', 'Y1982', 'Y1983', 'Y1984', 'Y1985', 'Y1986', 'Y1987',
              'Y1988', 'Y1989', 'Y1990', 'Y1991', 'Y1992', 'Y1993', 'Y1994', 'Y1995', 'Y1996', 'Y1997', 'Y1998', 'Y1999', 'Y2000', 'Y2001', 
              'Y2002', 'Y2003', 'Y2004', 'Y2005', 'Y2006', 'Y2017','Y2018'], axis=1)
       print(Australia_yield_2007_to_2016.columns)
       print(Australia_yield_2007_to_2016['Item'].unique())
       print("\n")
       print("Train Y data: ",Australia_yield_2007_to_2016)
       print("\n")
       print("Number of unique crops from 2007 to 2016",len(Australia_yield_2007_to_2016['Item'].unique()))

       # In[20]:
       yield_without_Item = Australia_yield_2007_to_2016.copy().drop(['Item'],axis=1)
       yield_without_Item_2017 = Australia_yield_2017.copy().drop(['Item'],axis=1)

       print(yield_without_Item)
       yield_tensor = torch.from_numpy(yield_without_Item.values)
       print("Shape: ", yield_tensor.shape)
       print("\t Yield tensor", yield_tensor)
       print(yield_without_Item_2017)
       yield_tensor_2017 = torch.from_numpy(yield_without_Item_2017.values)
       print("Shape: ", yield_tensor_2017.shape)
       print("\t Yield tensor", yield_tensor_2017)

       # Training + Validation: Output
       Y_train_ten = yield_tensor
       # Test Y
       Y_test = yield_tensor_2017

       print("Train X: \n",X_train_ten.shape)
       print("Train Y: \n",Y_train_ten.shape)
       print("Test X: \n",X_test.shape)
       print("Test Y: \n",Y_test.shape)
       
       return [X_train_ten, Y_train_ten, X_test, Y_test]

In [4]:

training_input = load_data()[0]
training_ouput = load_data()[1]
test_input = load_data()[2]
test_output = load_data()[3]

x = training_input
y = training_ouput

04, 5.0922e+04, 6.3416e+04],
        [7.2630e+03, 9.6109e+03, 8.5209e+03, 9.4395e+03, 1.0662e+04, 1.0899e+04,
         1.0113e+04, 1.1160e+04, 1.1399e+04, 1.4196e+04],
        [2.2404e+04, 2.9647e+04, 2.6284e+04, 2.9118e+04, 3.2890e+04, 3.3619e+04,
         3.1195e+04, 3.4426e+04, 3.5161e+04, 4.3789e+04],
        [2.3249e+03, 3.0765e+03, 2.7276e+03, 3.0216e+03, 3.4131e+03, 3.4887e+03,
         3.2371e+03, 3.5725e+03, 3.6487e+03, 4.5441e+03],
        [4.5447e+02, 6.0139e+02, 5.3318e+02, 5.9066e+02, 6.6719e+02, 6.8197e+02,
         6.3279e+02, 6.9835e+02, 7.1325e+02, 8.8827e+02],
        [4.5447e+02, 6.0139e+02, 5.3318e+02, 5.9066e+02, 6.6719e+02, 6.8197e+02,
         6.3279e+02, 6.9835e+02, 7.1325e+02, 8.8827e+02],
        [3.0365e+05, 1.6086e+05,        nan, 1.6086e+05, 4.7329e+04, 1.5680e+05,
         1.3463e+05, 1.3892e+05, 1.9200e+05, 1.7258e+05],
        [2.0392e+04, 2.9583e+05, 7.4000e+01, 5.9159e+05, 2.6952e+05, 6.4043e+05,
         1.4296e+05, 3.7954e+05, 4.9362e+05, 4.8665e+05]

In [5]:

# Transforms

# Training data
x_tensor = torch.Tensor(x.float())
x_data = (x - x.mean())/(x.max() - x.min())
test = torch.Tensor([[0,4,5]])
x_tensor_normalized = x.normal_()
y_data = (y.normal_())

transformed_x = torch.reshape(x_tensor_normalized,(1,220))
transformed_y = y_data
y = torch.reshape(y, (1,1010))


In [6]:
# Test data
transformed_test_x = torch.Tensor(test_input.float())
transformed_test_x = transformed_test_x.normal_()
transformed_test_y = torch.Tensor(test_output.float())
transformed_test_y = transformed_test_y.normal_()

In [7]:
# Define model
class NeuralNet(nn.Module):
       def __init__(self, D_in, H1, H2, H3, H4, H5, H6, D_out):
              super(NeuralNet, self).__init__()
              self.linear1 = nn.Linear(D_in, H1)
              self.linear2 = nn.Linear(H1, H2)
              self.linear3 = nn.Linear(H2, H3)
              self.linear4 = nn.Linear(H3, H4)
              self.linear5 = nn.Linear(H4, H5)
              self.linear6 = nn.Linear(H5, H6)
              self.linear7 = nn.Linear(H6, D_out)
              self.relu = nn.ReLU()
       def forward(self, x):
              y_pred = self.linear1(x)
              y_pred = torch.tanh(self.linear2(y_pred))
              y_pred = self.linear3(y_pred)
              y_pred = torch.tanh(self.linear4(y_pred))
              y_pred = self.linear5(y_pred)
              y_pred = torch.sigmoid(self.linear6(y_pred))
              y_pred = self.linear7(y_pred)
              return y_pred


In [8]:


from functools import partial
import numpy as np
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import random_split
import torchvision
import torchvision.transforms as transforms
from random import randint
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler


In [9]:


def get_accuracy(model, predicted, actual, threshold_percentage):
    num_items = len(actual)
    X = predicted.view(num_items)
    Y = actual.view(num_items)
    num_correct = torch.sum(torch.abs(X-Y)<torch.abs(threshold_percentage*Y))
    accuracy = (num_correct.item()*100.0/num_items)
    return (num_correct, accuracy)
    

def random_split_training(trainset_x, trainset_y):
       # TODO:
       # Select 2 random numbers between 1 and 10
       # Use them to index into x and y.
       # That will give us the training and validation set
       # Rest of the indices are thus our training set
       index1 = randint(0,9)
       index2 = randint(0,9)
       indices=[]
       i=0
       for i in range(0,10):
              if(i not in [index1,index2]):
                 indices.append(i)    
       
       indices = torch.tensor(indices)
       val_indices = torch.tensor([index1, index2])
       t_subset_x = torch.index_select(trainset_x, 1, indices)
       t_subset_y = torch.index_select(trainset_y, 1, indices)
       v_subset_x = torch.index_select(trainset_x, 1,val_indices)
       v_subset_y = torch.index_select(trainset_y, 1, val_indices)
       return (t_subset_x, t_subset_y, v_subset_x, v_subset_y)
# 


In [10]:
# Function to train and validate data. Save the best model based on validation loss
# 
# params:
# config: hyperparameter search space
# 
def train_crop_yield(config):
       net = NeuralNet(176, config['H1'], config['H2'], config['H3'], config["H4"], config["H5"], config["H6"],8)
       criterion = torch.nn.MSELoss()
       optimizer = torch.optim.SGD(net.parameters(), lr=config["lr"], momentum=0.9)
       # train_set = torch.cat((transformed_x, transformed_y), dim=0)
       # test_set = torch.cat((test_input, test_output), dim=0)
       
       x_by_years = transformed_x.reshape(22,10)
       y_by_years = transformed_y.reshape(101,10)
       
       train_subset_x, train_subset_y, val_subset_x, val_subset_y = random_split_training(x_by_years, y_by_years)
       
       for epoch in range(500):
              running_loss = 0.0
              epoch_steps = 0
              # Zero the accumulated gradients
              optimizer.zero_grad()
              # forward + backward + optimize
              output = net(train_subset_x.float().reshape(1,176))
              loss = criterion(output, train_subset_y.float())
              loss.backward()
              optimizer.step()
              # print statistics
              running_loss += loss.item()
              epoch_steps += 1
              if epoch%5==4:
                     print("[%d] loss: %.3f"%(epoch+1, running_loss/epoch_steps))
                     running_loss=0.0
       
       # Validation loss
       net = NeuralNet(44, config['H1'], config['H2'], config['H3'], config["H4"], config["H5"], config["H6"],2)
       val_loss = 0.0
       val_steps = 0
       total = 0
       correct = 0
       with torch.no_grad():
              val_output = net(val_subset_x.float().reshape(1,44))
              total = val_subset_y.size(0)
              correct = torch.sum(torch.abs(val_output-val_subset_y)<torch.abs(0.10*val_subset_y))
              loss = criterion(val_output, val_subset_y.float())
              val_loss += loss.cpu().numpy()
              torch.save((net.state_dict(), optimizer.state_dict()), './trained_net.dat')
              tune.report(loss=(val_loss), accuracy=(correct/total))
       print("\n Finished Training \n")
              

In [11]:
# Evaluation:
def test_accuracy(net, device='cpu'):
       net.eval()
       with torch.no_grad():
              predicted = net(transformed_test_x.float().reshape(1,22))
              return get_accuracy(net, predicted.reshape(101,1), transformed_test_y.reshape(101,1), 0.50)
       
# Define hyperparameters
config = {
       "H1":tune.sample_from(lambda _: 2**np.random.randint(7,9)),
       "H2":tune.sample_from(lambda _: 2**np.random.randint(7,9)),
       "H3":tune.sample_from(lambda _: 2**np.random.randint(7,12)),
       "H4":tune.sample_from(lambda _: 2**np.random.randint(7,12)),
       "H5":tune.sample_from(lambda _: 2**np.random.randint(7,12)),
       "H6":tune.sample_from(lambda _: 2**np.random.randint(7,9)),
       "lr":tune.loguniform(1e-2, 1e-1)
}

# Scheduler. Randomly try out combination of hyperparameters
scheduler = ASHAScheduler(
       metric="loss",
       mode="min",
       max_t=10,
       grace_period=1,
       reduction_factor=2
)

reporter=CLIReporter(metric_columns=["loss","accuracy", "training_iteration"])

result = tune.run(
       partial(train_crop_yield),
       config=config,
       num_samples=10,
       scheduler=scheduler
)

best_trial = result.get_best_trial("loss", "min", "last")

print("Best trial config: {}".format(best_trial.config))
print("Best trial final validation loss: {}".format(best_trial.last_result["loss"]))
print("Best trial fnal validation accuracy: {}".format(best_trial.last_result["accuracy"]))

best_trained_model = NeuralNet(22, best_trial.config["H1"], best_trial.config["H2"], best_trial.config["H3"],best_trial.config["H4"],best_trial.config["H5"],best_trial.config["H6"],101)

device="cpu"
best_trained_model.to(device)

# best_checkpoint_dir = best_trial.checkpoint.value
# print(" \n",type(best_trial.evaluated_params))
# model_state, optimizer_state = torch.load(os.path.join(best_checkpoint_dir, "checkpoint"))
# best_trained_model.load_state_dict(model_state)

num_correct, test_acc = test_accuracy(best_trained_model, device)
print("Best trial test set accuracy: {}".format(test_acc))


2021-05-16 17:39:46,270	INFO services.py:1269 -- View the Ray dashboard at http://127.0.0.1:8265
2021-05-16 17:39:50,331	WARNING experiment.py:294 -- No name detected on trainable. Using DEFAULT.
2021-05-16 17:39:50,334	INFO registry.py:65 -- Detected unknown callable for trainable. Converting to class.
2021-05-16 17:39:50,336	WARNING function_runner.py:545 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.


Trial name,status,loc,H1,H2,H3,H4,H5,H6,lr
DEFAULT_16490_00000,RUNNING,,128,128,256,512,2048,256,0.0872086
DEFAULT_16490_00001,PENDING,,128,256,2048,128,2048,256,0.0109812
DEFAULT_16490_00002,PENDING,,256,128,2048,2048,256,256,0.0146765
DEFAULT_16490_00003,PENDING,,256,256,512,2048,1024,256,0.0464081
DEFAULT_16490_00004,PENDING,,256,128,256,1024,256,128,0.0194283
DEFAULT_16490_00005,PENDING,,128,256,128,512,1024,256,0.0597779
DEFAULT_16490_00006,PENDING,,256,128,256,128,128,128,0.0135337
DEFAULT_16490_00007,PENDING,,256,256,256,1024,128,256,0.035487
DEFAULT_16490_00008,PENDING,,128,256,256,512,128,256,0.0402187
DEFAULT_16490_00009,PENDING,,128,256,256,256,1024,256,0.010686


(pid=79726) /opt/anaconda3/lib/python3.7/site-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([101, 8])) that is different to the input size (torch.Size([1, 8])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
(pid=79726)   return F.mse_loss(input, target, reduction=self.reduction)
(pid=79724) /opt/anaconda3/lib/python3.7/site-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([101, 8])) that is different to the input size (torch.Size([1, 8])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
(pid=79724)   return F.mse_loss(input, target, reduction=self.reduction)
(pid=79725) /opt/anaconda3/lib/python3.7/site-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([101, 8])) that is different to the input size (torch.Size([1, 8])). This will likely lead to incorrect results due to b

Trial name,status,loc,H1,H2,H3,H4,H5,H6,lr
DEFAULT_16490_00000,RUNNING,,128,128,256,512,2048,256,0.0872086
DEFAULT_16490_00001,RUNNING,,128,256,2048,128,2048,256,0.0109812
DEFAULT_16490_00002,RUNNING,,256,128,2048,2048,256,256,0.0146765
DEFAULT_16490_00003,RUNNING,,256,256,512,2048,1024,256,0.0464081
DEFAULT_16490_00004,PENDING,,256,128,256,1024,256,128,0.0194283
DEFAULT_16490_00005,PENDING,,128,256,128,512,1024,256,0.0597779
DEFAULT_16490_00006,PENDING,,256,128,256,128,128,128,0.0135337
DEFAULT_16490_00007,PENDING,,256,256,256,1024,128,256,0.035487
DEFAULT_16490_00008,PENDING,,128,256,256,512,128,256,0.0402187
DEFAULT_16490_00009,PENDING,,128,256,256,256,1024,256,0.010686


(pid=79724) [5] loss: 1.092
(pid=79726) [10] loss: 1.108
(pid=79725) [15] loss: 1.050
(pid=79727) [5] loss: 1.068
(pid=79726) [15] loss: 1.076
(pid=79725) [20] loss: 1.033
(pid=79724) [10] loss: 1.094
(pid=79726) [20] loss: 1.056
(pid=79725) [25] loss: 1.032
(pid=79725) [30] loss: 1.035
(pid=79726) [25] loss: 1.044
(pid=79727) [10] loss: 1.031
(pid=79724) [15] loss: 1.067
(pid=79725) [35] loss: 1.032
(pid=79726) [30] loss: 1.037
(pid=79725) [40] loss: 1.031
(pid=79726) [35] loss: 1.034
(pid=79724) [20] loss: 1.042
(pid=79727) [15] loss: 1.045
(pid=79725) [45] loss: 1.032
(pid=79726) [40] loss: 1.033
(pid=79725) [50] loss: 1.031
(pid=79726) [45] loss: 1.032
(pid=79724) [25] loss: 1.050
(pid=79725) [55] loss: 1.031
(pid=79727) [20] loss: 1.053
(pid=79726) [50] loss: 1.031
(pid=79725) [60] loss: 1.031
(pid=79726) [55] loss: 1.031
(pid=79724) [30] loss: 1.032
(pid=79725) [65] loss: 1.031
(pid=79726) [60] loss: 1.031
(pid=79727) [25] loss: 1.044
(pid=79725) [70] loss: 1.031
(pid=79724) [35]

...

(pid=79771) [340] loss: 1.031
(pid=79771) [345] loss: 1.031
(pid=79765) [250] loss: 1.031
(pid=79769) [195] loss: 1.031
(pid=79771) [350] loss: 1.031
(pid=79765) [255] loss: 1.031
(pid=79769) [200] loss: 1.031
(pid=79771) [355] loss: 1.031
(pid=79765) [260] loss: 1.031
(pid=79769) [205] loss: 1.031
(pid=79771) [360] loss: 1.031
(pid=79765) [265] loss: 1.031
(pid=79771) [365] loss: 1.031
(pid=79765) [270] loss: 1.031
(pid=79765) [275] loss: 1.031
(pid=79771) [370] loss: 1.031
(pid=79769) [210] loss: 1.031
(pid=79771) [375] loss: 1.031
(pid=79771) [380] loss: 1.031
(pid=79765) [280] loss: 1.031
(pid=79765) [285] loss: 1.031
(pid=79769) [215] loss: 1.031
(pid=79771) [385] loss: 1.031
(pid=79769) [220] loss: 1.031
(pid=79771) [390] loss: 1.031
(pid=79771) [395] loss: 1.031
(pid=79765) [290] loss: 1.031
(pid=79765) [295] loss: 1.031
(pid=79769) [225] loss: 1.031
(pid=79771) [400] loss: 1.031
(pid=79765) [300] loss: 1.031
(pid=79769) [230] loss: 1.031
(pid=79771) [405] loss: 1.031
(pid=79771

Trial name,status,loc,H1,H2,H3,H4,H5,H6,lr,iter,total time (s),loss
DEFAULT_16490_00009,RUNNING,10.201.135.226:79792,128,256,256,256,1024,256,0.010686,1,3.40566,1.10588
DEFAULT_16490_00000,TERMINATED,,128,128,256,512,2048,256,0.0872086,1,23.3879,1.16501
DEFAULT_16490_00001,TERMINATED,,128,256,2048,128,2048,256,0.0109812,1,21.427,1.13854
DEFAULT_16490_00002,TERMINATED,,256,128,2048,2048,256,256,0.0146765,1,56.288,1.55619
DEFAULT_16490_00003,TERMINATED,,256,256,512,2048,1024,256,0.0464081,1,41.7518,1.20924
DEFAULT_16490_00004,TERMINATED,,256,128,256,1024,256,128,0.0194283,1,10.0341,1.36364
DEFAULT_16490_00005,TERMINATED,,128,256,128,512,1024,256,0.0597779,1,13.633,1.09921
DEFAULT_16490_00006,TERMINATED,,256,128,256,128,128,128,0.0135337,1,4.40388,1.10768
DEFAULT_16490_00007,TERMINATED,,256,256,256,1024,128,256,0.035487,1,7.74633,1.07334
DEFAULT_16490_00008,TERMINATED,,128,256,256,512,128,256,0.0402187,1,5.90203,1.11994


Result for DEFAULT_16490_00009:
  accuracy: tensor(0.0297)
  date: 2021-05-16_17-41-02
  done: true
  experiment_id: e8bfa6dd690c438ab65b8380fb9424cb
  experiment_tag: 9_H1=128,H2=256,H3=256,H4=256,H5=1024,H6=256,lr=0.010686
  hostname: tprovpk.local
  iterations_since_restore: 1
  loss: 1.1058752536773682
  node_ip: 10.201.135.226
  pid: 79792
  time_since_restore: 3.4056596755981445
  time_this_iter_s: 3.4056596755981445
  time_total_s: 3.4056596755981445
  timestamp: 1621152662
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: '16490_00009'
  


Trial name,status,loc,H1,H2,H3,H4,H5,H6,lr,iter,total time (s),loss
DEFAULT_16490_00000,TERMINATED,,128,128,256,512,2048,256,0.0872086,1,23.3879,1.16501
DEFAULT_16490_00001,TERMINATED,,128,256,2048,128,2048,256,0.0109812,1,21.427,1.13854
DEFAULT_16490_00002,TERMINATED,,256,128,2048,2048,256,256,0.0146765,1,56.288,1.55619
DEFAULT_16490_00003,TERMINATED,,256,256,512,2048,1024,256,0.0464081,1,41.7518,1.20924
DEFAULT_16490_00004,TERMINATED,,256,128,256,1024,256,128,0.0194283,1,10.0341,1.36364
DEFAULT_16490_00005,TERMINATED,,128,256,128,512,1024,256,0.0597779,1,13.633,1.09921
DEFAULT_16490_00006,TERMINATED,,256,128,256,128,128,128,0.0135337,1,4.40388,1.10768
DEFAULT_16490_00007,TERMINATED,,256,256,256,1024,128,256,0.035487,1,7.74633,1.07334
DEFAULT_16490_00008,TERMINATED,,128,256,256,512,128,256,0.0402187,1,5.90203,1.11994
DEFAULT_16490_00009,TERMINATED,,128,256,256,256,1024,256,0.010686,1,3.40566,1.10588


2021-05-16 17:41:02,797	INFO tune.py:549 -- Total run time: 72.47 seconds (71.25 seconds for the tuning loop).
Best trial config: {'H1': 256, 'H2': 256, 'H3': 256, 'H4': 1024, 'H5': 128, 'H6': 256, 'lr': 0.03548703659971478}
Best trial final validation loss: 1.073341727256775
Best trial fnal validation accuracy: 0.009900989942252636
Best trial test set accuracy: 12.871287128712872


In [12]:
# Define model
class NeuralNet(nn.Module):
       def __init__(self, D_in, H1, H2, H3, H4, H5, H6, D_out):
              super(NeuralNet, self).__init__()
              self.linear1 = nn.Linear(D_in, H1)
              self.linear2 = nn.Linear(H1, H2)
              self.linear3 = nn.Linear(H2, H3)
              self.linear4 = nn.Linear(H3, H4)
              self.linear5 = nn.Linear(H4, H5)
              self.linear6 = nn.Linear(H5, H6)
              self.linear7 = nn.Linear(H6, D_out)
              self.relu = nn.ReLU()
       def forward(self, x):
              y_pred = self.linear1(x)
              y_pred = self.linear2(y_pred)
              y_pred = self.linear3(y_pred)
              y_pred = torch.tanh(self.linear4(y_pred))
              y_pred = self.linear5(y_pred)
              y_pred = torch.sigmoid(self.linear6(y_pred))
              y_pred = self.linear7(y_pred)
              return y_pred


In [13]:


from functools import partial
import numpy as np
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import random_split
import torchvision
import torchvision.transforms as transforms
from random import randint
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler


In [14]:


def get_accuracy(model, predicted, actual, threshold_percentage):
    num_items = len(actual)
    X = predicted.view(num_items)
    Y = actual.view(num_items)
    num_correct = torch.sum(torch.abs(X-Y)<torch.abs(threshold_percentage*Y))
    accuracy = (num_correct.item()*100.0/num_items)
    return (num_correct, accuracy)
    

def random_split_training(trainset_x, trainset_y):
       # TODO:
       # Select 2 random numbers between 1 and 10
       # Use them to index into x and y.
       # That will give us the training and validation set
       # Rest of the indices are thus our training set
       index1 = randint(0,9)
       index2 = randint(0,9)
       indices=[]
       i=0
       for i in range(0,10):
              if(i not in [index1,index2]):
                 indices.append(i)    
       
       indices = torch.tensor(indices)
       val_indices = torch.tensor([index1, index2])
       t_subset_x = torch.index_select(trainset_x, 1, indices)
       t_subset_y = torch.index_select(trainset_y, 1, indices)
       v_subset_x = torch.index_select(trainset_x, 1,val_indices)
       v_subset_y = torch.index_select(trainset_y, 1, val_indices)
       return (t_subset_x, t_subset_y, v_subset_x, v_subset_y)
# 


In [15]:
# Function to train and validate data. Save the best model based on validation loss
# 
# params:
# config: hyperparameter search space
# 
def train_crop_yield(config):
       net = NeuralNet(176, config['H1'], config['H2'], config['H3'], config["H4"], config["H5"], config["H6"],8)
       criterion = torch.nn.MSELoss()
       optimizer = torch.optim.SGD(net.parameters(), lr=config["lr"], momentum=0.9)
       # train_set = torch.cat((transformed_x, transformed_y), dim=0)
       # test_set = torch.cat((test_input, test_output), dim=0)
       
       x_by_years = transformed_x.reshape(22,10)
       y_by_years = transformed_y.reshape(101,10)
       
       train_subset_x, train_subset_y, val_subset_x, val_subset_y = random_split_training(x_by_years, y_by_years)
       
       for epoch in range(500):
              running_loss = 0.0
              epoch_steps = 0
              # Zero the accumulated gradients
              optimizer.zero_grad()
              # forward + backward + optimize
              output = net(train_subset_x.float().reshape(1,176))
              loss = criterion(output, train_subset_y.float())
              loss.backward()
              optimizer.step()
              # print statistics
              running_loss += loss.item()
              epoch_steps += 1
              if epoch%5==4:
                     print("[%d] loss: %.3f"%(epoch+1, running_loss/epoch_steps))
                     running_loss=0.0
       
       # Validation loss
       net = NeuralNet(44, config['H1'], config['H2'], config['H3'], config["H4"], config["H5"], config["H6"],2)
       val_loss = 0.0
       val_steps = 0
       total = 0
       correct = 0
       with torch.no_grad():
              val_output = net(val_subset_x.float().reshape(1,44))
              total = val_subset_y.size(0)
              correct = torch.sum(torch.abs(val_output-val_subset_y)<torch.abs(0.10*val_subset_y))
              loss = criterion(val_output, val_subset_y.float())
              val_loss += loss.cpu().numpy()
              torch.save((net.state_dict(), optimizer.state_dict()), './trained_net.dat')
              tune.report(loss=(val_loss), accuracy=(correct/total))
       print("\n Finished Training \n")
              

In [16]:
# Evaluation:
def test_accuracy(net, device='cpu'):
       net.eval()
       with torch.no_grad():
              predicted = net(transformed_test_x.float().reshape(1,22))
              return get_accuracy(net, predicted.reshape(101,1), transformed_test_y.reshape(101,1), 0.50)
       
# Define hyperparameters
config = {
       "H1":tune.sample_from(lambda _: 2**np.random.randint(7,9)),
       "H2":tune.sample_from(lambda _: 2**np.random.randint(7,9)),
       "H3":tune.sample_from(lambda _: 2**np.random.randint(7,12)),
       "H4":tune.sample_from(lambda _: 2**np.random.randint(7,12)),
       "H5":tune.sample_from(lambda _: 2**np.random.randint(7,12)),
       "H6":tune.sample_from(lambda _: 2**np.random.randint(7,9)),
       "lr":tune.loguniform(1e-2, 1e-1)
}

# Scheduler. Randomly try out combination of hyperparameters
scheduler = ASHAScheduler(
       metric="loss",
       mode="min",
       max_t=10,
       grace_period=1,
       reduction_factor=2
)

reporter=CLIReporter(metric_columns=["loss","accuracy", "training_iteration"])

result = tune.run(
       partial(train_crop_yield),
       config=config,
       num_samples=10,
       scheduler=scheduler
)

best_trial = result.get_best_trial("loss", "min", "last")

print("Best trial config: {}".format(best_trial.config))
print("Best trial final validation loss: {}".format(best_trial.last_result["loss"]))
print("Best trial fnal validation accuracy: {}".format(best_trial.last_result["accuracy"]))

best_trained_model = NeuralNet(22, best_trial.config["H1"], best_trial.config["H2"], best_trial.config["H3"],best_trial.config["H4"],best_trial.config["H5"],best_trial.config["H6"],101)

device="cpu"
best_trained_model.to(device)

# best_checkpoint_dir = best_trial.checkpoint.value
# print(" \n",type(best_trial.evaluated_params))
# model_state, optimizer_state = torch.load(os.path.join(best_checkpoint_dir, "checkpoint"))
# best_trained_model.load_state_dict(model_state)

num_correct, test_acc = test_accuracy(best_trained_model, device)
print("Best trial test set accuracy: {}".format(test_acc))


2021-05-16 17:41:44,760	WARNING experiment.py:294 -- No name detected on trainable. Using DEFAULT.
2021-05-16 17:41:44,760	INFO registry.py:65 -- Detected unknown callable for trainable. Converting to class.


Trial name,status,loc,H1,H2,H3,H4,H5,H6,lr
DEFAULT_5a742_00000,RUNNING,,256,128,256,256,512,256,0.0514636
DEFAULT_5a742_00001,PENDING,,128,128,2048,128,1024,256,0.0450736
DEFAULT_5a742_00002,PENDING,,128,128,128,512,1024,256,0.0533672
DEFAULT_5a742_00003,PENDING,,128,128,512,1024,2048,128,0.0180032
DEFAULT_5a742_00004,PENDING,,128,256,2048,1024,1024,256,0.0195965
DEFAULT_5a742_00005,PENDING,,256,256,2048,128,1024,256,0.0189042
DEFAULT_5a742_00006,PENDING,,128,128,128,256,128,256,0.0500462
DEFAULT_5a742_00007,PENDING,,128,128,128,256,2048,128,0.0206985
DEFAULT_5a742_00008,PENDING,,256,128,1024,2048,256,128,0.0175938
DEFAULT_5a742_00009,PENDING,,256,128,1024,256,512,256,0.0418781


(pid=79796) /opt/anaconda3/lib/python3.7/site-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([101, 8])) that is different to the input size (torch.Size([1, 8])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
(pid=79796)   return F.mse_loss(input, target, reduction=self.reduction)
(pid=79795) /opt/anaconda3/lib/python3.7/site-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([101, 8])) that is different to the input size (torch.Size([1, 8])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
(pid=79795)   return F.mse_loss(input, target, reduction=self.reduction)
(pid=79803) /opt/anaconda3/lib/python3.7/site-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([101, 8])) that is different to the input size (torch.Size([1, 8])). This will likely lead to incorrect results due to b

Trial name,status,loc,H1,H2,H3,H4,H5,H6,lr
DEFAULT_5a742_00000,RUNNING,,256,128,256,256,512,256,0.0514636
DEFAULT_5a742_00001,RUNNING,,128,128,2048,128,1024,256,0.0450736
DEFAULT_5a742_00002,RUNNING,,128,128,128,512,1024,256,0.0533672
DEFAULT_5a742_00003,RUNNING,,128,128,512,1024,2048,128,0.0180032
DEFAULT_5a742_00004,PENDING,,128,256,2048,1024,1024,256,0.0195965
DEFAULT_5a742_00005,PENDING,,256,256,2048,128,1024,256,0.0189042
DEFAULT_5a742_00006,PENDING,,128,128,128,256,128,256,0.0500462
DEFAULT_5a742_00007,PENDING,,128,128,128,256,2048,128,0.0206985
DEFAULT_5a742_00008,PENDING,,256,128,1024,2048,256,128,0.0175938
DEFAULT_5a742_00009,PENDING,,256,128,1024,256,512,256,0.0418781


(pid=79796) 
(pid=79803) [40] loss: 1.030
(pid=79803) [45] loss: 1.029
(pid=79796) [30] loss: 1.029
(pid=79795) [30] loss: 1.029
(pid=79803) [50] loss: 1.029
(pid=79796) [35] loss: 1.030
(pid=79795) [35] loss: 1.031
(pid=79803) [55] loss: 1.029
(pid=79803) [60] loss: 1.029
(pid=79796) [40] loss: 1.029
(pid=79795) [40] loss: 1.029
(pid=79803) [65] loss: 1.029
(pid=79796) [45] loss: 1.029
(pid=79803) [70] loss: 1.029
(pid=79795) [45] loss: 1.029
(pid=79796) [50] loss: 1.029
(pid=79795) [50] loss: 1.029
(pid=79803) [75] loss: 1.029
(pid=79803) [80] loss: 1.029
(pid=79796) [55] loss: 1.029
(pid=79803) [85] loss: 1.029
(pid=79795) [55] loss: 1.029
(pid=79803) [90] loss: 1.029
(pid=79796) [60] loss: 1.029
(pid=79795) [60] loss: 1.029
(pid=79803) [95] loss: 1.029
(pid=79796) [65] loss: 1.029
(pid=79803) [100] loss: 1.029
(pid=79817) /opt/anaconda3/lib/python3.7/site-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([101, 8])) that is different to the input si

...

(pid=79832) [465] loss: 1.029
(pid=79863) [155] loss: 1.029
(pid=79863) [160] loss: 1.029
(pid=79832) [470] loss: 1.029
(pid=79863) [165] loss: 1.029
(pid=79832) [475] loss: 1.029
(pid=79863) [170] loss: 1.029
(pid=79832) [480] loss: 1.029
(pid=79863) [175] loss: 1.029
(pid=79832) [485] loss: 1.029
(pid=79863) [180] loss: 1.029
(pid=79863) [185] loss: 1.029
(pid=79832) [490] loss: 1.029
(pid=79863) [190] loss: 1.029
(pid=79832) [495] loss: 1.029
(pid=79863) [195] loss: 1.029
(pid=79832) [500] loss: 1.029
(pid=79863) [200] loss: 1.029
(pid=79832) /opt/anaconda3/lib/python3.7/site-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([101, 2])) that is different to the input size (torch.Size([1, 2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
(pid=79832)   return F.mse_loss(input, target, reduction=self.reduction)
(pid=79863) [205] loss: 1.029
Result for DEFAULT_5a742_00004:
  accuracy: tensor(0.02

Trial name,status,loc,H1,H2,H3,H4,H5,H6,lr,iter,total time (s),loss
DEFAULT_5a742_00008,RUNNING,10.201.135.226:79863,256,128,1024,2048,256,128,0.0175938,1,17.6937,1.0957
DEFAULT_5a742_00000,TERMINATED,,256,128,256,256,512,256,0.0514636,1,8.73112,1.27775
DEFAULT_5a742_00001,TERMINATED,,128,128,2048,128,1024,256,0.0450736,1,13.7717,1.08197
DEFAULT_5a742_00002,TERMINATED,,128,128,128,512,1024,256,0.0533672,1,13.5344,1.11708
DEFAULT_5a742_00003,TERMINATED,,128,128,512,1024,2048,128,0.0180032,1,34.6047,1.09273
DEFAULT_5a742_00004,TERMINATED,,128,256,2048,1024,1024,256,0.0195965,1,41.4295,1.18449
DEFAULT_5a742_00005,TERMINATED,,256,256,2048,128,1024,256,0.0189042,1,16.0013,1.30933
DEFAULT_5a742_00006,TERMINATED,,128,128,128,256,128,256,0.0500462,1,4.62155,1.25305
DEFAULT_5a742_00007,TERMINATED,,128,128,128,256,2048,128,0.0206985,1,13.0954,1.22721
DEFAULT_5a742_00009,TERMINATED,,256,128,1024,256,512,256,0.0418781,1,9.87205,1.28778


Result for DEFAULT_5a742_00008:
  accuracy: tensor(0.0297)
  date: 2021-05-16_17-42-55
  done: true
  experiment_id: aab80c53f6bd4cfea142c0dec5ec8705
  experiment_tag: 8_H1=256,H2=128,H3=1024,H4=2048,H5=256,H6=128,lr=0.017594
  hostname: tprovpk.local
  iterations_since_restore: 1
  loss: 1.095700740814209
  node_ip: 10.201.135.226
  pid: 79863
  time_since_restore: 17.69373321533203
  time_this_iter_s: 17.69373321533203
  time_total_s: 17.69373321533203
  timestamp: 1621152775
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 5a742_00008
  


Trial name,status,loc,H1,H2,H3,H4,H5,H6,lr,iter,total time (s),loss
DEFAULT_5a742_00000,TERMINATED,,256,128,256,256,512,256,0.0514636,1,8.73112,1.27775
DEFAULT_5a742_00001,TERMINATED,,128,128,2048,128,1024,256,0.0450736,1,13.7717,1.08197
DEFAULT_5a742_00002,TERMINATED,,128,128,128,512,1024,256,0.0533672,1,13.5344,1.11708
DEFAULT_5a742_00003,TERMINATED,,128,128,512,1024,2048,128,0.0180032,1,34.6047,1.09273
DEFAULT_5a742_00004,TERMINATED,,128,256,2048,1024,1024,256,0.0195965,1,41.4295,1.18449
DEFAULT_5a742_00005,TERMINATED,,256,256,2048,128,1024,256,0.0189042,1,16.0013,1.30933
DEFAULT_5a742_00006,TERMINATED,,128,128,128,256,128,256,0.0500462,1,4.62155,1.25305
DEFAULT_5a742_00007,TERMINATED,,128,128,128,256,2048,128,0.0206985,1,13.0954,1.22721
DEFAULT_5a742_00008,TERMINATED,,256,128,1024,2048,256,128,0.0175938,1,17.6937,1.0957
DEFAULT_5a742_00009,TERMINATED,,256,128,1024,256,512,256,0.0418781,1,9.87205,1.28778


(pid=79863) 
(pid=79863)  Finished Training 
(pid=79863) 
2021-05-16 17:42:55,791	INFO tune.py:549 -- Total run time: 71.03 seconds (70.85 seconds for the tuning loop).
Best trial config: {'H1': 128, 'H2': 128, 'H3': 2048, 'H4': 128, 'H5': 1024, 'H6': 256, 'lr': 0.045073629794893315}
Best trial final validation loss: 1.0819658041000366
Best trial fnal validation accuracy: 0.019801979884505272
Best trial test set accuracy: 8.910891089108912


In [17]:
# Define model
class NeuralNet(nn.Module):
       def __init__(self, D_in, H1, H2, H3, H4, H5, H6, D_out):
              super(NeuralNet, self).__init__()
              self.linear1 = nn.Linear(D_in, H1)
              self.linear2 = nn.Linear(H1, H2)
              self.linear3 = nn.Linear(H2, H3)
              self.linear4 = nn.Linear(H3, H4)
              self.linear5 = nn.Linear(H4, H5)
              self.linear6 = nn.Linear(H5, H6)
              self.linear7 = nn.Linear(H6, D_out)
              self.relu = nn.ReLU()
       def forward(self, x):
              y_pred = torch.tanh(self.linear1(x))
              y_pred = torch.tanh(self.linear2(y_pred))
              y_pred = torch.tanh(self.linear3(y_pred))
              y_pred = torch.tanh(self.linear4(y_pred))
              y_pred = torch.tanh(self.linear5(y_pred))
              y_pred = torch.sigmoid(self.linear6(y_pred))
              y_pred = self.linear7(y_pred)
              return y_pred


In [18]:


from functools import partial
import numpy as np
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import random_split
import torchvision
import torchvision.transforms as transforms
from random import randint
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler


In [19]:


def get_accuracy(model, predicted, actual, threshold_percentage):
    num_items = len(actual)
    X = predicted.view(num_items)
    Y = actual.view(num_items)
    num_correct = torch.sum(torch.abs(X-Y)<torch.abs(threshold_percentage*Y))
    accuracy = (num_correct.item()*100.0/num_items)
    return (num_correct, accuracy)
    

def random_split_training(trainset_x, trainset_y):
       # TODO:
       # Select 2 random numbers between 1 and 10
       # Use them to index into x and y.
       # That will give us the training and validation set
       # Rest of the indices are thus our training set
       index1 = randint(0,9)
       index2 = randint(0,9)
       indices=[]
       i=0
       for i in range(0,10):
              if(i not in [index1,index2]):
                 indices.append(i)    
       
       indices = torch.tensor(indices)
       val_indices = torch.tensor([index1, index2])
       t_subset_x = torch.index_select(trainset_x, 1, indices)
       t_subset_y = torch.index_select(trainset_y, 1, indices)
       v_subset_x = torch.index_select(trainset_x, 1,val_indices)
       v_subset_y = torch.index_select(trainset_y, 1, val_indices)
       return (t_subset_x, t_subset_y, v_subset_x, v_subset_y)
# 


In [20]:
# Function to train and validate data. Save the best model based on validation loss
# 
# params:
# config: hyperparameter search space
# 
def train_crop_yield(config):
       net = NeuralNet(176, config['H1'], config['H2'], config['H3'], config["H4"], config["H5"], config["H6"],8)
       criterion = torch.nn.MSELoss()
       optimizer = torch.optim.SGD(net.parameters(), lr=config["lr"], momentum=0.9)
       # train_set = torch.cat((transformed_x, transformed_y), dim=0)
       # test_set = torch.cat((test_input, test_output), dim=0)
       
       x_by_years = transformed_x.reshape(22,10)
       y_by_years = transformed_y.reshape(101,10)
       
       train_subset_x, train_subset_y, val_subset_x, val_subset_y = random_split_training(x_by_years, y_by_years)
       
       for epoch in range(500):
              running_loss = 0.0
              epoch_steps = 0
              # Zero the accumulated gradients
              optimizer.zero_grad()
              # forward + backward + optimize
              output = net(train_subset_x.float().reshape(1,176))
              loss = criterion(output, train_subset_y.float())
              loss.backward()
              optimizer.step()
              # print statistics
              running_loss += loss.item()
              epoch_steps += 1
              if epoch%5==4:
                     print("[%d] loss: %.3f"%(epoch+1, running_loss/epoch_steps))
                     running_loss=0.0
       
       # Validation loss
       net = NeuralNet(44, config['H1'], config['H2'], config['H3'], config["H4"], config["H5"], config["H6"],2)
       val_loss = 0.0
       val_steps = 0
       total = 0
       correct = 0
       with torch.no_grad():
              val_output = net(val_subset_x.float().reshape(1,44))
              total = val_subset_y.size(0)
              correct = torch.sum(torch.abs(val_output-val_subset_y)<torch.abs(0.10*val_subset_y))
              loss = criterion(val_output, val_subset_y.float())
              val_loss += loss.cpu().numpy()
              torch.save((net.state_dict(), optimizer.state_dict()), './trained_net.dat')
              tune.report(loss=(val_loss), accuracy=(correct/total))
       print("\n Finished Training \n")
              

In [21]:
# Evaluation:
def test_accuracy(net, device='cpu'):
       net.eval()
       with torch.no_grad():
              predicted = net(transformed_test_x.float().reshape(1,22))
              return get_accuracy(net, predicted.reshape(101,1), transformed_test_y.reshape(101,1), 0.50)
       
# Define hyperparameters
config = {
       "H1":tune.sample_from(lambda _: 2**np.random.randint(7,9)),
       "H2":tune.sample_from(lambda _: 2**np.random.randint(7,9)),
       "H3":tune.sample_from(lambda _: 2**np.random.randint(7,12)),
       "H4":tune.sample_from(lambda _: 2**np.random.randint(7,12)),
       "H5":tune.sample_from(lambda _: 2**np.random.randint(7,12)),
       "H6":tune.sample_from(lambda _: 2**np.random.randint(7,9)),
       "lr":tune.loguniform(1e-2, 1e-1)
}

# Scheduler. Randomly try out combination of hyperparameters
scheduler = ASHAScheduler(
       metric="loss",
       mode="min",
       max_t=10,
       grace_period=1,
       reduction_factor=2
)

reporter=CLIReporter(metric_columns=["loss","accuracy", "training_iteration"])

result = tune.run(
       partial(train_crop_yield),
       config=config,
       num_samples=10,
       scheduler=scheduler
)

best_trial = result.get_best_trial("loss", "min", "last")

print("Best trial config: {}".format(best_trial.config))
print("Best trial final validation loss: {}".format(best_trial.last_result["loss"]))
print("Best trial fnal validation accuracy: {}".format(best_trial.last_result["accuracy"]))

best_trained_model = NeuralNet(22, best_trial.config["H1"], best_trial.config["H2"], best_trial.config["H3"],best_trial.config["H4"],best_trial.config["H5"],best_trial.config["H6"],101)

device="cpu"
best_trained_model.to(device)

# best_checkpoint_dir = best_trial.checkpoint.value
# print(" \n",type(best_trial.evaluated_params))
# model_state, optimizer_state = torch.load(os.path.join(best_checkpoint_dir, "checkpoint"))
# best_trained_model.load_state_dict(model_state)

num_correct, test_acc = test_accuracy(best_trained_model, device)
print("Best trial test set accuracy: {}".format(test_acc))


2021-05-16 17:45:21,527	WARNING experiment.py:294 -- No name detected on trainable. Using DEFAULT.
2021-05-16 17:45:21,529	INFO registry.py:65 -- Detected unknown callable for trainable. Converting to class.


Trial name,status,loc,H1,H2,H3,H4,H5,H6,lr
DEFAULT_dbaa4_00000,PENDING,,256,128,128,256,2048,256,0.0256193
DEFAULT_dbaa4_00001,PENDING,,256,128,1024,1024,256,128,0.0136277
DEFAULT_dbaa4_00002,PENDING,,128,256,128,256,512,256,0.0126557
DEFAULT_dbaa4_00003,PENDING,,256,256,256,256,128,256,0.0131832
DEFAULT_dbaa4_00004,PENDING,,128,256,1024,256,2048,128,0.0125532
DEFAULT_dbaa4_00005,PENDING,,256,128,256,2048,2048,256,0.0709722
DEFAULT_dbaa4_00006,PENDING,,256,256,2048,512,128,128,0.0725692
DEFAULT_dbaa4_00007,PENDING,,256,256,2048,1024,512,256,0.0513366
DEFAULT_dbaa4_00008,PENDING,,256,128,512,2048,128,256,0.0139405
DEFAULT_dbaa4_00009,PENDING,,256,256,2048,128,2048,128,0.0728766


Trial name,status,loc,H1,H2,H3,H4,H5,H6,lr
DEFAULT_dbaa4_00000,RUNNING,,256,128,128,256,2048,256,0.0256193
DEFAULT_dbaa4_00001,RUNNING,,256,128,1024,1024,256,128,0.0136277
DEFAULT_dbaa4_00002,RUNNING,,128,256,128,256,512,256,0.0126557
DEFAULT_dbaa4_00003,RUNNING,,256,256,256,256,128,256,0.0131832
DEFAULT_dbaa4_00004,PENDING,,128,256,1024,256,2048,128,0.0125532
DEFAULT_dbaa4_00005,PENDING,,256,128,256,2048,2048,256,0.0709722
DEFAULT_dbaa4_00006,PENDING,,256,256,2048,512,128,128,0.0725692
DEFAULT_dbaa4_00007,PENDING,,256,256,2048,1024,512,256,0.0513366
DEFAULT_dbaa4_00008,PENDING,,256,128,512,2048,128,256,0.0139405
DEFAULT_dbaa4_00009,PENDING,,256,256,2048,128,2048,128,0.0728766


(pid=79887) /opt/anaconda3/lib/python3.7/site-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([101, 8])) that is different to the input size (torch.Size([1, 8])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
(pid=79887)   return F.mse_loss(input, target, reduction=self.reduction)
(pid=79888) /opt/anaconda3/lib/python3.7/site-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([101, 8])) that is different to the input size (torch.Size([1, 8])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
(pid=79888)   return F.mse_loss(input, target, reduction=self.reduction)
(pid=79889) /opt/anaconda3/lib/python3.7/site-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([101, 8])) that is different to the input size (torch.Size([1, 8])). This will likely lead to incorrect results due to b

Trial name,status,loc,H1,H2,H3,H4,H5,H6,lr,iter,total time (s),loss
DEFAULT_dbaa4_00000,RUNNING,,256,128,128,256,2048,256,0.0256193,,,
DEFAULT_dbaa4_00001,RUNNING,,256,128,1024,1024,256,128,0.0136277,,,
DEFAULT_dbaa4_00002,RUNNING,,128,256,128,256,512,256,0.0126557,,,
DEFAULT_dbaa4_00003,RUNNING,10.201.135.226:79888,256,256,256,256,128,256,0.0131832,1,5.9224,0.995622
DEFAULT_dbaa4_00004,PENDING,,128,256,1024,256,2048,128,0.0125532,,,
DEFAULT_dbaa4_00005,PENDING,,256,128,256,2048,2048,256,0.0709722,,,
DEFAULT_dbaa4_00006,PENDING,,256,256,2048,512,128,128,0.0725692,,,
DEFAULT_dbaa4_00007,PENDING,,256,256,2048,1024,512,256,0.0513366,,,
DEFAULT_dbaa4_00008,PENDING,,256,128,512,2048,128,256,0.0139405,,,
DEFAULT_dbaa4_00009,PENDING,,256,256,2048,128,2048,128,0.0728766,,,


...

Result for DEFAULT_dbaa4_00007:
  accuracy: tensor(0.)
  date: 2021-05-16_17-46-34
  done: true
  experiment_id: d244c656f93840b7917d901ad89f876a
  experiment_tag: 7_H1=256,H2=256,H3=2048,H4=1024,H5=512,H6=256,lr=0.051337
  hostname: tprovpk.local
  iterations_since_restore: 1
  loss: 1.0177665948867798
  node_ip: 10.201.135.226
  pid: 79935
  time_since_restore: 39.83930587768555
  time_this_iter_s: 39.83930587768555
  time_total_s: 39.83930587768555
  timestamp: 1621152994
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: dbaa4_00007
  
(pid=79920) [395] loss: 1.050
(pid=79949) [265] loss: 1.050
(pid=79945) [440] loss: 1.050
(pid=79949) [270] loss: 1.050
(pid=79945) [445] loss: 1.050
(pid=79949) [275] loss: 1.050
(pid=79945) [450] loss: 1.050
(pid=79920) [400] loss: 1.050
(pid=79949) [280] loss: 1.050
(pid=79945) [455] loss: 1.050
(pid=79949) [285] loss: 1.050
(pid=79945) [460] loss: 1.050
(pid=79949) [290] loss: 1.050
(pid=79945) [465] loss: 1.050
(pid=79920) [405] lo

Trial name,status,loc,H1,H2,H3,H4,H5,H6,lr,iter,total time (s),loss
DEFAULT_dbaa4_00005,RUNNING,,256,128,256,2048,2048,256,0.0709722,,,
DEFAULT_dbaa4_00000,TERMINATED,,256,128,128,256,2048,256,0.0256193,1,14.501,1.10906
DEFAULT_dbaa4_00001,TERMINATED,,256,128,1024,1024,256,128,0.0136277,1,17.6087,1.10905
DEFAULT_dbaa4_00002,TERMINATED,,128,256,128,256,512,256,0.0126557,1,6.72713,1.28712
DEFAULT_dbaa4_00003,TERMINATED,,256,256,256,256,128,256,0.0131832,1,5.9224,0.995622
DEFAULT_dbaa4_00004,TERMINATED,,128,256,1024,256,2048,128,0.0125532,1,18.5985,0.987668
DEFAULT_dbaa4_00006,TERMINATED,,256,256,2048,512,128,128,0.0725692,1,23.2868,1.33153
DEFAULT_dbaa4_00007,TERMINATED,,256,256,2048,1024,512,256,0.0513366,1,39.8393,1.01777
DEFAULT_dbaa4_00008,TERMINATED,,256,128,512,2048,128,256,0.0139405,1,18.7097,0.9842
DEFAULT_dbaa4_00009,TERMINATED,,256,256,2048,128,2048,128,0.0728766,1,15.5842,1.06631


(pid=79920) [475] loss: 1.050
(pid=79920) [480] loss: 1.050
(pid=79920) [485] loss: 1.050
(pid=79920) [490] loss: 1.050
(pid=79920) [495] loss: 1.050
(pid=79920) [500] loss: 1.050
(pid=79920) /opt/anaconda3/lib/python3.7/site-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([101, 2])) that is different to the input size (torch.Size([1, 2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
(pid=79920)   return F.mse_loss(input, target, reduction=self.reduction)
Result for DEFAULT_dbaa4_00005:
  accuracy: tensor(0.0198)
  date: 2021-05-16_17-46-41
  done: true
  experiment_id: acc87d3e1ce34223ae93d14d3245f74c
  hostname: tprovpk.local
  iterations_since_restore: 1
  loss: 1.112239956855774
  node_ip: 10.201.135.226
  pid: 79920
  time_since_restore: 54.510109186172485
  time_this_iter_s: 54.510109186172485
  time_total_s: 54.510109186172485
  timestamp: 1621153001
  timesteps_since_restore: 0
  trai

Trial name,status,loc,H1,H2,H3,H4,H5,H6,lr,iter,total time (s),loss
DEFAULT_dbaa4_00000,TERMINATED,,256,128,128,256,2048,256,0.0256193,1,14.501,1.10906
DEFAULT_dbaa4_00001,TERMINATED,,256,128,1024,1024,256,128,0.0136277,1,17.6087,1.10905
DEFAULT_dbaa4_00002,TERMINATED,,128,256,128,256,512,256,0.0126557,1,6.72713,1.28712
DEFAULT_dbaa4_00003,TERMINATED,,256,256,256,256,128,256,0.0131832,1,5.9224,0.995622
DEFAULT_dbaa4_00004,TERMINATED,,128,256,1024,256,2048,128,0.0125532,1,18.5985,0.987668
DEFAULT_dbaa4_00005,TERMINATED,,256,128,256,2048,2048,256,0.0709722,1,54.5101,1.11224
DEFAULT_dbaa4_00006,TERMINATED,,256,256,2048,512,128,128,0.0725692,1,23.2868,1.33153
DEFAULT_dbaa4_00007,TERMINATED,,256,256,2048,1024,512,256,0.0513366,1,39.8393,1.01777
DEFAULT_dbaa4_00008,TERMINATED,,256,128,512,2048,128,256,0.0139405,1,18.7097,0.9842
DEFAULT_dbaa4_00009,TERMINATED,,256,256,2048,128,2048,128,0.0728766,1,15.5842,1.06631


2021-05-16 17:46:42,165	INFO tune.py:549 -- Total run time: 80.64 seconds (80.42 seconds for the tuning loop).
Best trial config: {'H1': 256, 'H2': 128, 'H3': 512, 'H4': 2048, 'H5': 128, 'H6': 256, 'lr': 0.013940532761118583}
Best trial final validation loss: 0.9842002987861633
Best trial fnal validation accuracy: 0.0
Best trial test set accuracy: 6.930693069306931


In [22]:


import torch
print(torch.__version__)
import pandas as pd
print(pd.__version__)
from torch.utils.data import DataLoader,ConcatDataset
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error



1.8.1
1.2.4


In [23]:

# ## Read pesticides and fertilizer products data


In [24]:
def load_data():
       pesticides_frame = pd.read_csv("Inputs_Pesticides_Use_E_All_Data_NOFLAG.csv", engine='python')
       pesticides_frame.head
  
       # Read fertilizer data in the form of products
       fertilizers_frame = pd.read_csv("Inputs_FertilizersProduct_E_All_Data_NOFLAG.csv", engine='python')
       fertilizers_frame.head
       
       # Read the crop yield data
       yield_frame = pd.read_csv("Production_Crops_E_All_Data_NOFLAG.csv",engine='python')
       yield_frame.head

       # %<br>
       # Get the fertilizer usage in terms of agriculture

       # In[4]:     
       print("Fertilizer products",fertilizers_frame.columns)
       print("\n")
       print("Input pesticides", pesticides_frame.columns)
       
       # Filter out the data for Australia
       is_Country_Australia_Products = fertilizers_frame['Area']=="Australia"
       Australia_Fertilizers_Products = fertilizers_frame[is_Country_Australia_Products]
       # In[5]:
       is_Country_Australia_Pesticides = pesticides_frame['Area']=="Australia"
       Australia_Pesticides = pesticides_frame[is_Country_Australia_Pesticides]

       # Get the fertilizer usage for agricultural use
       # In[6]:
       is_Agricultural = Australia_Fertilizers_Products['Element']=="Agricultural Use"
       Australia_Fertilizers_Products_Agricultural = Australia_Fertilizers_Products[is_Agricultural]
       print("Fertilizers \n")
       print(Australia_Fertilizers_Products_Agricultural.columns)
       print(Australia_Fertilizers_Products_Agricultural['Item'])
       print("\n")
       print("Pesticides \n")
       print(Australia_Pesticides.columns)
       print(Australia_Pesticides['Item'])
       
       # %<br>
       # 7 years for training - 2007 to 2013<br>
       # 3 years fro validation - 2014 to 2016<br>
       # 2017 for testing
       # In[7]:
       X_ten_years_fertlizers = Australia_Fertilizers_Products_Agricultural.drop(['Area Code', 'Area', 'Item Code', 'Element Code','Element',
              'Unit','Y2002', 'Y2003', 'Y2004', 'Y2005', 'Y2006', 'Y2017'], axis=1)
       print(X_ten_years_fertlizers)
       print(X_ten_years_fertlizers.columns)
       fertilizer_2017 = Australia_Fertilizers_Products_Agricultural['Y2017']
       print("2017 \n",fertilizer_2017)

       # In[8]:
       X_ten_years_pesticides = Australia_Pesticides.drop(['Area Code', 'Area', 'Item Code', 'Element Code', 'Element',
              'Unit', 'Y1990', 'Y1991', 'Y1992', 'Y1993', 'Y1994', 'Y1995', 'Y1996', 'Y1997', 'Y1998', 'Y1999', 'Y2000', 'Y2001', 'Y2002', 'Y2003', 'Y2004', 'Y2005', 'Y2006', 'Y2017'], axis=1)
       print(X_ten_years_pesticides)
       print(X_ten_years_pesticides.columns)
       pesticides_2017 = Australia_Pesticides['Y2017']
       print("2017 \n",pesticides_2017)

       # %<br>
       # Drop Ammonia anyhydrous and Ammonia nitrate from 2007 to 2013. It has not been used in these years.
       # In[9]:
       X_ten_years_fertlizers = X_ten_years_fertlizers.drop([602, 607], axis=0)
       print(X_ten_years_fertlizers)
       print(X_ten_years_fertlizers.columns)
       fertilizer_2017 = fertilizer_2017.drop([602, 607], axis=0)
       print(fertilizer_2017)
       
       # %<br>
       # 2009 to 2013
       # ## Handle missing values from 2009 to 2013
       # In[10]:
       interpolate_2009_to_2016 = X_ten_years_fertlizers.interpolate()
       interpolate_2009_to_2016
       interpolate_2017 = fertilizer_2017.interpolate()
       print("interpolated 2017 fertilizers \n",interpolate_2017)
       # TODO: Don't just interpolate. Use the means of the previous 2  years
       # interpolate_pesticides_2017 = pesticides_2017.interpolate()
       # print("interpolated 2017 pesticides \n",interpolate_pesticides_2017)

       # %<br>
       #  Interpolate missing fertlizers data<br>
       #  Replace na with mean values of 2004,2005,2009,2010
       # ##  Replace 2007 nan values with the mean of 2004 and 2005

       # In[11]:

       interpolate_2004_2005 = Australia_Fertilizers_Products_Agricultural[['Y2004','Y2005']].copy().interpolate()
       index_names = interpolate_2004_2005.index
       index_names
       interpolate_2004_2005=interpolate_2004_2005.drop([602,607], axis=0)
       interpolate_2004_2005['mean'] = interpolate_2004_2005.mean(axis=1)
       print("Mean of 2004 and 2005 ==== \n")
       print(interpolate_2004_2005)
       pesticides_2015_2016 = Australia_Pesticides[['Y2015','Y2016']]
       pesticides_2015_2016['mean'] = pesticides_2015_2016.mean(axis=1)
       print("Mean of 2015 and 2016 === \n")
       print(pesticides_2015_2016)

       # Use this for 2007

       # In[ ]:
       # ##  Replace 2008 nan values with the mean of 2009 and 2010

       # In[12]:
       interpolate_2009_2010 = Australia_Fertilizers_Products_Agricultural[['Y2009','Y2010']].copy().interpolate()
       index_names = interpolate_2009_2010.index
       index_names

       # In[13]:
       interpolate_2009_2010['mean'] = interpolate_2009_2010.mean(axis=1)
       print("Mean of 2009 and 2010 ==== \n")
       # Use this for 2008
       interpolate_2009_2010=interpolate_2009_2010.drop([602,607], axis=0)
       print(interpolate_2009_2010)


       # ## Populate 2007 nan values with computed mean
       # In[14]:
       # Fertilizers
       i=0
       for i in range(1,16):
              interpolate_2009_to_2016.iloc[0:i,1:2] = interpolate_2004_2005['mean'].iloc[0:i]

       #Pesticides
       j=0
       for j in range(1,7):
              pesticides_2017.iloc[0:j,] = pesticides_2015_2016['mean'].iloc[0:j]

       # ## Populate 2008 nan values with computed mean
       # In[15]:
       j=0
       for j in range(1,16):
              interpolate_2009_to_2016.iloc[0:j,2:3] = interpolate_2009_2010['mean'].iloc[0:j]
                
       interpolate_2009_to_2016.iloc[15, 1] = 1437658.50
       print(interpolate_2009_to_2016.iloc[15, 1])
       interpolate_2009_to_2016.iloc[15, 2] = 171351.00

       # In[16]:
       X_ten_years_fertilizers_interpolated = interpolate_2009_to_2016
       X_ten_years_fertilizers_interpolated
       fertilizer_2017_interpolated = interpolate_2017
       print(fertilizer_2017_interpolated)

       # #### TODO: Combine pesticdes and fertilizers data
       # In[17]:
       print(X_ten_years_pesticides)
       print(X_ten_years_fertilizers_interpolated)

       pesticides_without_Item = X_ten_years_pesticides.copy().drop(['Item'], axis=1)
       pesticides_2017_wo_item = pesticides_2017
       fertilizers_without_Item = X_ten_years_fertilizers_interpolated.copy().drop(['Item'], axis=1)
       fertilizer_2017_wo_item = fertilizer_2017_interpolated
       # torch.tensor(X_ten_years_pesticides.values.astype(np.float64))
       print(X_ten_years_pesticides.values)
       print(X_ten_years_fertilizers_interpolated.values)
       print(pesticides_2017_wo_item.values)
       print(fertilizer_2017_wo_item.values)

       print(pesticides_without_Item)
       pesticides_tensor = torch.from_numpy(pesticides_without_Item.values)
       print("\t Pesticides tensor", pesticides_tensor)
       pesticides_2017_tensor = torch.from_numpy(pesticides_2017_wo_item.values)

       print(fertilizers_without_Item)
       fertilizers_tensor = torch.from_numpy(fertilizers_without_Item.values)
       fertilizer_2017_tensor = torch.from_numpy(fertilizer_2017_wo_item.values)
       print("\t Fertilizers tensor", fertilizers_tensor)

       # In[18]:
       print("Train : \n",pesticides_tensor.shape)
       print("Train: \n",fertilizers_tensor.shape)
       print("Test: \n",pesticides_2017_tensor.shape)
       print("Test: \n",fertilizer_2017_tensor.shape)

       # TODO: Normalize inputs before concatenation
       pesticides_and_fertilizers = torch.cat((pesticides_tensor, fertilizers_tensor), dim=0)
       print(pesticides_and_fertilizers.shape)
       print("\n")
       print("Training X data for 10 years: ",pesticides_and_fertilizers)
       print("\n")
       pesticides_and_fertilizers_2017 = torch.cat((pesticides_2017_tensor, fertilizer_2017_tensor), dim=0)
       print(pesticides_and_fertilizers_2017.shape)
       print("\n")
       print("Testing X data for 1 year: ",pesticides_and_fertilizers_2017)
       print("\n")

       # Training + Validation data : Input
       X_train_ten = pesticides_and_fertilizers;
       X_test = pesticides_and_fertilizers_2017

       # #### TODO: Create output vector

       # In[19]:
       print(yield_frame.columns)
       #%%
       yield_frame.head
       #%%
       Australia = yield_frame['Area']=="Australia"
       Production = yield_frame['Element']=="Production"
       Australia_yield = yield_frame[Australia]
       Australia_yield_production = Australia_yield[Production]
       print(yield_frame[Australia])
       #%%
       # Get the yield data for the year 2017 only
       Australia_yield_2017 = Australia_yield_production.drop(['Area Code', 'Area', 'Item Code', 'Element Code','Element',
              'Unit','Y1961', 'Y1962', 'Y1963', 'Y1964', 'Y1965', 'Y1966', 'Y1967', 'Y1968', 'Y1969', 'Y1970', 'Y1971', 'Y1972', 'Y1973', 
              'Y1974', 'Y1975', 'Y1976', 'Y1977', 'Y1978', 'Y1979', 'Y1980', 'Y1981', 'Y1982', 'Y1983', 'Y1984', 'Y1985', 'Y1986', 'Y1987',
              'Y1988', 'Y1989', 'Y1990', 'Y1991', 'Y1992', 'Y1993', 'Y1994', 'Y1995', 'Y1996', 'Y1997', 'Y1998', 'Y1999', 'Y2000', 'Y2001', 
              'Y2002', 'Y2003', 'Y2004', 'Y2005', 'Y2006', 'Y2007', 'Y2008', 'Y2009', 'Y2010', 'Y2011', 'Y2012', 'Y2013', 'Y2014', 'Y2015', 
              'Y2016', 'Y2018'], axis=1)
       print("\n")
       print("Test Y data: ",Australia_yield_2017)
       print("\n")
       print(Australia_yield_2017.columns)
       print(Australia_yield_2017['Item'].unique())
       print("Number of unique crops in 2017: ",len(Australia_yield_2017['Item'].unique()))

       Australia_yield_2007_to_2016 = Australia_yield_production.drop(['Area Code', 'Area', 'Item Code', 'Element Code','Element',
              'Unit','Y1961', 'Y1962', 'Y1963', 'Y1964', 'Y1965', 'Y1966', 'Y1967', 'Y1968', 'Y1969', 'Y1970', 'Y1971', 'Y1972', 'Y1973', 
              'Y1974', 'Y1975', 'Y1976', 'Y1977', 'Y1978', 'Y1979', 'Y1980', 'Y1981', 'Y1982', 'Y1983', 'Y1984', 'Y1985', 'Y1986', 'Y1987',
              'Y1988', 'Y1989', 'Y1990', 'Y1991', 'Y1992', 'Y1993', 'Y1994', 'Y1995', 'Y1996', 'Y1997', 'Y1998', 'Y1999', 'Y2000', 'Y2001', 
              'Y2002', 'Y2003', 'Y2004', 'Y2005', 'Y2006', 'Y2017','Y2018'], axis=1)
       print(Australia_yield_2007_to_2016.columns)
       print(Australia_yield_2007_to_2016['Item'].unique())
       print("\n")
       print("Train Y data: ",Australia_yield_2007_to_2016)
       print("\n")
       print("Number of unique crops from 2007 to 2016",len(Australia_yield_2007_to_2016['Item'].unique()))

       # In[20]:
       yield_without_Item = Australia_yield_2007_to_2016.copy().drop(['Item'],axis=1)
       yield_without_Item_2017 = Australia_yield_2017.copy().drop(['Item'],axis=1)

       print(yield_without_Item)
       yield_tensor = torch.from_numpy(yield_without_Item.values)
       print("Shape: ", yield_tensor.shape)
       print("\t Yield tensor", yield_tensor)
       print(yield_without_Item_2017)
       yield_tensor_2017 = torch.from_numpy(yield_without_Item_2017.values)
       print("Shape: ", yield_tensor_2017.shape)
       print("\t Yield tensor", yield_tensor_2017)

       # Training + Validation: Output
       Y_train_ten = yield_tensor
       # Test Y
       Y_test = yield_tensor_2017

       print("Train X: \n",X_train_ten.shape)
       print("Train Y: \n",Y_train_ten.shape)
       print("Test X: \n",X_test.shape)
       print("Test Y: \n",Y_test.shape)
       
       return [X_train_ten, Y_train_ten, X_test, Y_test]

In [25]:

training_input = load_data()[0]
training_ouput = load_data()[1]
test_input = load_data()[2]
test_output = load_data()[3]

x = training_input
y = training_ouput

04, 5.0922e+04, 6.3416e+04],
        [7.2630e+03, 9.6109e+03, 8.5209e+03, 9.4395e+03, 1.0662e+04, 1.0899e+04,
         1.0113e+04, 1.1160e+04, 1.1399e+04, 1.4196e+04],
        [2.2404e+04, 2.9647e+04, 2.6284e+04, 2.9118e+04, 3.2890e+04, 3.3619e+04,
         3.1195e+04, 3.4426e+04, 3.5161e+04, 4.3789e+04],
        [2.3249e+03, 3.0765e+03, 2.7276e+03, 3.0216e+03, 3.4131e+03, 3.4887e+03,
         3.2371e+03, 3.5725e+03, 3.6487e+03, 4.5441e+03],
        [4.5447e+02, 6.0139e+02, 5.3318e+02, 5.9066e+02, 6.6719e+02, 6.8197e+02,
         6.3279e+02, 6.9835e+02, 7.1325e+02, 8.8827e+02],
        [4.5447e+02, 6.0139e+02, 5.3318e+02, 5.9066e+02, 6.6719e+02, 6.8197e+02,
         6.3279e+02, 6.9835e+02, 7.1325e+02, 8.8827e+02],
        [3.0365e+05, 1.6086e+05,        nan, 1.6086e+05, 4.7329e+04, 1.5680e+05,
         1.3463e+05, 1.3892e+05, 1.9200e+05, 1.7258e+05],
        [2.0392e+04, 2.9583e+05, 7.4000e+01, 5.9159e+05, 2.6952e+05, 6.4043e+05,
         1.4296e+05, 3.7954e+05, 4.9362e+05, 4.8665e+05]

In [26]:

# Transforms

# Training data
x_tensor = torch.Tensor(x.float())
x_data = (x - x.mean())/(x.max() - x.min())
test = torch.Tensor([[0,4,5]])
x_tensor_normalized = x.normal_()
y_data = (y.normal_())

transformed_x = torch.reshape(x_tensor_normalized,(1,220))
transformed_y = y_data
y = torch.reshape(y, (1,1010))


In [27]:
# Test data
transformed_test_x = torch.Tensor(test_input.float())
transformed_test_x = transformed_test_x.normal_()
transformed_test_y = torch.Tensor(test_output.float())
transformed_test_y = transformed_test_y.normal_()

In [28]:
# Define model
class NeuralNet(nn.Module):
       def __init__(self, D_in, H1, H2, H3, H4, H5, H6, D_out):
              super(NeuralNet, self).__init__()
              self.linear1 = nn.Linear(D_in, H1)
              self.linear2 = nn.Linear(H1, H2)
              self.linear3 = nn.Linear(H2, H3)
              self.linear4 = nn.Linear(H3, H4)
              self.linear5 = nn.Linear(H4, H5)
              self.linear6 = nn.Linear(H5, H6)
              self.linear7 = nn.Linear(H6, D_out)
              self.relu = nn.ReLU()
       def forward(self, x):
              y_pred = torch.tanh(self.linear1(x))
              y_pred = self.linear2(y_pred)
              y_pred = torch.tanh(self.linear3(y_pred))
              y_pred = self.linear4(y_pred)
              y_pred = torch.tanh(self.linear5(y_pred))
              y_pred = self.linear6(y_pred)
              y_pred = torch.sigmoid(self.linear7(y_pred))
              return y_pred


In [29]:


from functools import partial
import numpy as np
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import random_split
import torchvision
import torchvision.transforms as transforms
from random import randint
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler


In [30]:


def get_accuracy(model, predicted, actual, threshold_percentage):
    num_items = len(actual)
    X = predicted.view(num_items)
    Y = actual.view(num_items)
    num_correct = torch.sum(torch.abs(X-Y)<torch.abs(threshold_percentage*Y))
    accuracy = (num_correct.item()*100.0/num_items)
    return (num_correct, accuracy)
    

def random_split_training(trainset_x, trainset_y):
       # TODO:
       # Select 2 random numbers between 1 and 10
       # Use them to index into x and y.
       # That will give us the training and validation set
       # Rest of the indices are thus our training set
       index1 = randint(0,9)
       index2 = randint(0,9)
       indices=[]
       i=0
       for i in range(0,10):
              if(i not in [index1,index2]):
                 indices.append(i)    
       
       indices = torch.tensor(indices)
       val_indices = torch.tensor([index1, index2])
       t_subset_x = torch.index_select(trainset_x, 1, indices)
       t_subset_y = torch.index_select(trainset_y, 1, indices)
       v_subset_x = torch.index_select(trainset_x, 1,val_indices)
       v_subset_y = torch.index_select(trainset_y, 1, val_indices)
       return (t_subset_x, t_subset_y, v_subset_x, v_subset_y)
# 


In [31]:
# Function to train and validate data. Save the best model based on validation loss
# 
# params:
# config: hyperparameter search space
# 
def train_crop_yield(config):
       net = NeuralNet(176, config['H1'], config['H2'], config['H3'], config["H4"], config["H5"], config["H6"],8)
       criterion = torch.nn.MSELoss()
       optimizer = torch.optim.SGD(net.parameters(), lr=config["lr"], momentum=0.9)
       # train_set = torch.cat((transformed_x, transformed_y), dim=0)
       # test_set = torch.cat((test_input, test_output), dim=0)
       
       x_by_years = transformed_x.reshape(22,10)
       y_by_years = transformed_y.reshape(101,10)
       
       train_subset_x, train_subset_y, val_subset_x, val_subset_y = random_split_training(x_by_years, y_by_years)
       
       for epoch in range(500):
              running_loss = 0.0
              epoch_steps = 0
              # Zero the accumulated gradients
              optimizer.zero_grad()
              # forward + backward + optimize
              output = net(train_subset_x.float().reshape(1,176))
              loss = criterion(output, train_subset_y.float())
              loss.backward()
              optimizer.step()
              # print statistics
              running_loss += loss.item()
              epoch_steps += 1
              if epoch%5==4:
                     print("[%d] loss: %.3f"%(epoch+1, running_loss/epoch_steps))
                     running_loss=0.0
       
       # Validation loss
       net = NeuralNet(44, config['H1'], config['H2'], config['H3'], config["H4"], config["H5"], config["H6"],2)
       val_loss = 0.0
       val_steps = 0
       total = 0
       correct = 0
       with torch.no_grad():
              val_output = net(val_subset_x.float().reshape(1,44))
              total = val_subset_y.size(0)
              correct = torch.sum(torch.abs(val_output-val_subset_y)<torch.abs(0.10*val_subset_y))
              loss = criterion(val_output, val_subset_y.float())
              val_loss += loss.cpu().numpy()
              torch.save((net.state_dict(), optimizer.state_dict()), './trained_net.dat')
              tune.report(loss=(val_loss), accuracy=(correct/total))
       print("\n Finished Training \n")
              

In [32]:
# Evaluation:
def test_accuracy(net, device='cpu'):
       net.eval()
       with torch.no_grad():
              predicted = net(transformed_test_x.float().reshape(1,22))
              return get_accuracy(net, predicted.reshape(101,1), transformed_test_y.reshape(101,1), 0.50)
       
# Define hyperparameters
config = {
       "H1":tune.sample_from(lambda _: 2**np.random.randint(7,9)),
       "H2":tune.sample_from(lambda _: 2**np.random.randint(5,7)),
       "H3":tune.sample_from(lambda _: 2**np.random.randint(7,12)),
       "H4":tune.sample_from(lambda _: 2**np.random.randint(7,12)),
       "H5":tune.sample_from(lambda _: 2**np.random.randint(5,7)),
       "H6":tune.sample_from(lambda _: 2**np.random.randint(5,9)),
       "lr":tune.loguniform(1e-2, 1e-1)
}

# Scheduler. Randomly try out combination of hyperparameters
scheduler = ASHAScheduler(
       metric="loss",
       mode="min",
       max_t=10,
       grace_period=1,
       reduction_factor=2
)

reporter=CLIReporter(metric_columns=["loss","accuracy", "training_iteration"])

result = tune.run(
       partial(train_crop_yield),
       config=config,
       num_samples=10,
       scheduler=scheduler
)

best_trial = result.get_best_trial("loss", "min", "last")

print("Best trial config: {}".format(best_trial.config))
print("Best trial final validation loss: {}".format(best_trial.last_result["loss"]))
print("Best trial fnal validation accuracy: {}".format(best_trial.last_result["accuracy"]))

best_trained_model = NeuralNet(22, best_trial.config["H1"], best_trial.config["H2"], best_trial.config["H3"],best_trial.config["H4"],best_trial.config["H5"],best_trial.config["H6"],101)

device="cpu"
best_trained_model.to(device)

# best_checkpoint_dir = best_trial.checkpoint.value
# print(" \n",type(best_trial.evaluated_params))
# model_state, optimizer_state = torch.load(os.path.join(best_checkpoint_dir, "checkpoint"))
# best_trained_model.load_state_dict(model_state)

num_correct, test_acc = test_accuracy(best_trained_model, device)
print("Best trial test set accuracy: {}".format(test_acc))


2021-05-16 17:49:58,497	WARNING experiment.py:294 -- No name detected on trainable. Using DEFAULT.
2021-05-16 17:49:58,501	INFO registry.py:65 -- Detected unknown callable for trainable. Converting to class.


Trial name,status,loc,H1,H2,H3,H4,H5,H6,lr
DEFAULT_80c0d_00000,PENDING,,256,32,512,256,32,32,0.0317173
DEFAULT_80c0d_00001,PENDING,,256,32,1024,256,32,256,0.0209972
DEFAULT_80c0d_00002,PENDING,,256,64,2048,512,64,32,0.052015
DEFAULT_80c0d_00003,PENDING,,128,64,128,2048,64,32,0.0978931
DEFAULT_80c0d_00004,PENDING,,256,32,512,128,64,32,0.0898699
DEFAULT_80c0d_00005,PENDING,,128,32,256,512,32,64,0.0107664
DEFAULT_80c0d_00006,PENDING,,256,64,128,1024,64,128,0.0111338
DEFAULT_80c0d_00007,PENDING,,128,32,128,1024,32,64,0.0274834
DEFAULT_80c0d_00008,PENDING,,256,32,256,128,32,256,0.022764
DEFAULT_80c0d_00009,PENDING,,128,64,1024,512,32,64,0.0117928


Trial name,status,loc,H1,H2,H3,H4,H5,H6,lr
DEFAULT_80c0d_00000,RUNNING,,256,32,512,256,32,32,0.0317173
DEFAULT_80c0d_00001,RUNNING,,256,32,1024,256,32,256,0.0209972
DEFAULT_80c0d_00002,RUNNING,,256,64,2048,512,64,32,0.052015
DEFAULT_80c0d_00003,RUNNING,,128,64,128,2048,64,32,0.0978931
DEFAULT_80c0d_00004,PENDING,,256,32,512,128,64,32,0.0898699
DEFAULT_80c0d_00005,PENDING,,128,32,256,512,32,64,0.0107664
DEFAULT_80c0d_00006,PENDING,,256,64,128,1024,64,128,0.0111338
DEFAULT_80c0d_00007,PENDING,,128,32,128,1024,32,64,0.0274834
DEFAULT_80c0d_00008,PENDING,,256,32,256,128,32,256,0.022764
DEFAULT_80c0d_00009,PENDING,,128,64,1024,512,32,64,0.0117928


(pid=80005) 2021-05-16 17:50:09,004	ERROR function_runner.py:254 -- Runner Thread raised error.
(pid=80005) Traceback (most recent call last):
(pid=80005)   File "/opt/anaconda3/lib/python3.7/site-packages/ray/tune/function_runner.py", line 248, in run
(pid=80005)     self._entrypoint()
(pid=80005)   File "/opt/anaconda3/lib/python3.7/site-packages/ray/tune/function_runner.py", line 316, in entrypoint
(pid=80005)     self._status_reporter.get_checkpoint())
(pid=80005)   File "/opt/anaconda3/lib/python3.7/site-packages/ray/tune/function_runner.py", line 580, in _trainable_func
(pid=80005)     output = fn()
(pid=80005)   File "<ipython-input-31-d9d213d1f510>", line 24, in train_crop_yield
(pid=80005) RuntimeError: shape '[1, 176]' is invalid for input of size 198
(pid=80005) Exception in thread Thread-2:
(pid=80005) Traceback (most recent call last):
(pid=80005)   File "/opt/anaconda3/lib/python3.7/threading.py", line 926, in _bootstrap_inner
(pid=80005)     self.run()
(pid=80005)   File

Trial name,status,loc,H1,H2,H3,H4,H5,H6,lr
DEFAULT_80c0d_00000,RUNNING,,256,32,512,256,32,32,0.0317173
DEFAULT_80c0d_00001,RUNNING,,256,32,1024,256,32,256,0.0209972
DEFAULT_80c0d_00002,RUNNING,,256,64,2048,512,64,32,0.052015
DEFAULT_80c0d_00004,PENDING,,256,32,512,128,64,32,0.0898699
DEFAULT_80c0d_00005,PENDING,,128,32,256,512,32,64,0.0107664
DEFAULT_80c0d_00006,PENDING,,256,64,128,1024,64,128,0.0111338
DEFAULT_80c0d_00007,PENDING,,128,32,128,1024,32,64,0.0274834
DEFAULT_80c0d_00008,PENDING,,256,32,256,128,32,256,0.022764
DEFAULT_80c0d_00009,PENDING,,128,64,1024,512,32,64,0.0117928
DEFAULT_80c0d_00003,ERROR,,128,64,128,2048,64,32,0.0978931


2021-05-16 17:50:09,312	ERROR trial_runner.py:732 -- Trial DEFAULT_80c0d_00001: Error processing event.
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/ray/tune/trial_runner.py", line 702, in _process_trial
    results = self.trial_executor.fetch_result(trial)
  File "/opt/anaconda3/lib/python3.7/site-packages/ray/tune/ray_trial_executor.py", line 686, in fetch_result
    result = ray.get(trial_future[0], timeout=DEFAULT_GET_TIMEOUT)
  File "/opt/anaconda3/lib/python3.7/site-packages/ray/_private/client_mode_hook.py", line 47, in wrapper
    return func(*args, **kwargs)
  File "/opt/anaconda3/lib/python3.7/site-packages/ray/worker.py", line 1481, in get
    raise value.as_instanceof_cause()
ray.exceptions.RayTaskError(TuneError): ray::ImplicitFunc.train_buffered() (pid=80005, ip=10.201.135.226)
  File "python/ray/_raylet.pyx", line 505, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 449, in ray._raylet.execute_task.function_execu

Trial name,status,loc,H1,H2,H3,H4,H5,H6,lr
DEFAULT_80c0d_00004,RUNNING,,256,32,512,128,64,32,0.0898699
DEFAULT_80c0d_00005,RUNNING,,128,32,256,512,32,64,0.0107664
DEFAULT_80c0d_00006,RUNNING,,256,64,128,1024,64,128,0.0111338
DEFAULT_80c0d_00008,PENDING,,256,32,256,128,32,256,0.022764
DEFAULT_80c0d_00009,PENDING,,128,64,1024,512,32,64,0.0117928
DEFAULT_80c0d_00000,ERROR,,256,32,512,256,32,32,0.0317173
DEFAULT_80c0d_00001,ERROR,,256,32,1024,256,32,256,0.0209972
DEFAULT_80c0d_00002,ERROR,,256,64,2048,512,64,32,0.052015
DEFAULT_80c0d_00003,ERROR,,128,64,128,2048,64,32,0.0978931
DEFAULT_80c0d_00007,ERROR,,128,32,128,1024,32,64,0.0274834


2021-05-16 17:50:16,545	ERROR trial_runner.py:732 -- Trial DEFAULT_80c0d_00004: Error processing event.
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/ray/tune/trial_runner.py", line 702, in _process_trial
    results = self.trial_executor.fetch_result(trial)
  File "/opt/anaconda3/lib/python3.7/site-packages/ray/tune/ray_trial_executor.py", line 686, in fetch_result
    result = ray.get(trial_future[0], timeout=DEFAULT_GET_TIMEOUT)
  File "/opt/anaconda3/lib/python3.7/site-packages/ray/_private/client_mode_hook.py", line 47, in wrapper
    return func(*args, **kwargs)
  File "/opt/anaconda3/lib/python3.7/site-packages/ray/worker.py", line 1481, in get
    raise value.as_instanceof_cause()
ray.exceptions.RayTaskError(TuneError): ray::ImplicitFunc.train_buffered() (pid=80026, ip=10.201.135.226)
  File "python/ray/_raylet.pyx", line 505, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 449, in ray._raylet.execute_task.function_execu

Trial name,status,loc,H1,H2,H3,H4,H5,H6,lr
DEFAULT_80c0d_00000,ERROR,,256,32,512,256,32,32,0.0317173
DEFAULT_80c0d_00001,ERROR,,256,32,1024,256,32,256,0.0209972
DEFAULT_80c0d_00002,ERROR,,256,64,2048,512,64,32,0.052015
DEFAULT_80c0d_00003,ERROR,,128,64,128,2048,64,32,0.0978931
DEFAULT_80c0d_00004,ERROR,,256,32,512,128,64,32,0.0898699
DEFAULT_80c0d_00005,ERROR,,128,32,256,512,32,64,0.0107664
DEFAULT_80c0d_00006,ERROR,,256,64,128,1024,64,128,0.0111338
DEFAULT_80c0d_00007,ERROR,,128,32,128,1024,32,64,0.0274834
DEFAULT_80c0d_00008,ERROR,,256,32,256,128,32,256,0.022764
DEFAULT_80c0d_00009,ERROR,,128,64,1024,512,32,64,0.0117928


TuneError: ('Trials did not complete', [DEFAULT_80c0d_00000, DEFAULT_80c0d_00001, DEFAULT_80c0d_00002, DEFAULT_80c0d_00003, DEFAULT_80c0d_00004, DEFAULT_80c0d_00005, DEFAULT_80c0d_00006, DEFAULT_80c0d_00007, DEFAULT_80c0d_00008, DEFAULT_80c0d_00009])

In [33]:


import torch
print(torch.__version__)
import pandas as pd
print(pd.__version__)
from torch.utils.data import DataLoader,ConcatDataset
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error



1.8.1
1.2.4


In [34]:

# ## Read pesticides and fertilizer products data


In [35]:
def load_data():
       pesticides_frame = pd.read_csv("Inputs_Pesticides_Use_E_All_Data_NOFLAG.csv", engine='python')
       pesticides_frame.head
  
       # Read fertilizer data in the form of products
       fertilizers_frame = pd.read_csv("Inputs_FertilizersProduct_E_All_Data_NOFLAG.csv", engine='python')
       fertilizers_frame.head
       
       # Read the crop yield data
       yield_frame = pd.read_csv("Production_Crops_E_All_Data_NOFLAG.csv",engine='python')
       yield_frame.head

       # %<br>
       # Get the fertilizer usage in terms of agriculture

       # In[4]:     
       print("Fertilizer products",fertilizers_frame.columns)
       print("\n")
       print("Input pesticides", pesticides_frame.columns)
       
       # Filter out the data for Australia
       is_Country_Australia_Products = fertilizers_frame['Area']=="Australia"
       Australia_Fertilizers_Products = fertilizers_frame[is_Country_Australia_Products]
       # In[5]:
       is_Country_Australia_Pesticides = pesticides_frame['Area']=="Australia"
       Australia_Pesticides = pesticides_frame[is_Country_Australia_Pesticides]

       # Get the fertilizer usage for agricultural use
       # In[6]:
       is_Agricultural = Australia_Fertilizers_Products['Element']=="Agricultural Use"
       Australia_Fertilizers_Products_Agricultural = Australia_Fertilizers_Products[is_Agricultural]
       print("Fertilizers \n")
       print(Australia_Fertilizers_Products_Agricultural.columns)
       print(Australia_Fertilizers_Products_Agricultural['Item'])
       print("\n")
       print("Pesticides \n")
       print(Australia_Pesticides.columns)
       print(Australia_Pesticides['Item'])
       
       # %<br>
       # 7 years for training - 2007 to 2013<br>
       # 3 years fro validation - 2014 to 2016<br>
       # 2017 for testing
       # In[7]:
       X_ten_years_fertlizers = Australia_Fertilizers_Products_Agricultural.drop(['Area Code', 'Area', 'Item Code', 'Element Code','Element',
              'Unit','Y2002', 'Y2003', 'Y2004', 'Y2005', 'Y2006', 'Y2017'], axis=1)
       print(X_ten_years_fertlizers)
       print(X_ten_years_fertlizers.columns)
       fertilizer_2017 = Australia_Fertilizers_Products_Agricultural['Y2017']
       print("2017 \n",fertilizer_2017)

       # In[8]:
       X_ten_years_pesticides = Australia_Pesticides.drop(['Area Code', 'Area', 'Item Code', 'Element Code', 'Element',
              'Unit', 'Y1990', 'Y1991', 'Y1992', 'Y1993', 'Y1994', 'Y1995', 'Y1996', 'Y1997', 'Y1998', 'Y1999', 'Y2000', 'Y2001', 'Y2002', 'Y2003', 'Y2004', 'Y2005', 'Y2006', 'Y2017'], axis=1)
       print(X_ten_years_pesticides)
       print(X_ten_years_pesticides.columns)
       pesticides_2017 = Australia_Pesticides['Y2017']
       print("2017 \n",pesticides_2017)

       # %<br>
       # Drop Ammonia anyhydrous and Ammonia nitrate from 2007 to 2013. It has not been used in these years.
       # In[9]:
       X_ten_years_fertlizers = X_ten_years_fertlizers.drop([602, 607], axis=0)
       print(X_ten_years_fertlizers)
       print(X_ten_years_fertlizers.columns)
       fertilizer_2017 = fertilizer_2017.drop([602, 607], axis=0)
       print(fertilizer_2017)
       
       # %<br>
       # 2009 to 2013
       # ## Handle missing values from 2009 to 2013
       # In[10]:
       interpolate_2009_to_2016 = X_ten_years_fertlizers.interpolate()
       interpolate_2009_to_2016
       interpolate_2017 = fertilizer_2017.interpolate()
       print("interpolated 2017 fertilizers \n",interpolate_2017)
       # TODO: Don't just interpolate. Use the means of the previous 2  years
       # interpolate_pesticides_2017 = pesticides_2017.interpolate()
       # print("interpolated 2017 pesticides \n",interpolate_pesticides_2017)

       # %<br>
       #  Interpolate missing fertlizers data<br>
       #  Replace na with mean values of 2004,2005,2009,2010
       # ##  Replace 2007 nan values with the mean of 2004 and 2005

       # In[11]:

       interpolate_2004_2005 = Australia_Fertilizers_Products_Agricultural[['Y2004','Y2005']].copy().interpolate()
       index_names = interpolate_2004_2005.index
       index_names
       interpolate_2004_2005=interpolate_2004_2005.drop([602,607], axis=0)
       interpolate_2004_2005['mean'] = interpolate_2004_2005.mean(axis=1)
       print("Mean of 2004 and 2005 ==== \n")
       print(interpolate_2004_2005)
       pesticides_2015_2016 = Australia_Pesticides[['Y2015','Y2016']]
       pesticides_2015_2016['mean'] = pesticides_2015_2016.mean(axis=1)
       print("Mean of 2015 and 2016 === \n")
       print(pesticides_2015_2016)

       # Use this for 2007

       # In[ ]:
       # ##  Replace 2008 nan values with the mean of 2009 and 2010

       # In[12]:
       interpolate_2009_2010 = Australia_Fertilizers_Products_Agricultural[['Y2009','Y2010']].copy().interpolate()
       index_names = interpolate_2009_2010.index
       index_names

       # In[13]:
       interpolate_2009_2010['mean'] = interpolate_2009_2010.mean(axis=1)
       print("Mean of 2009 and 2010 ==== \n")
       # Use this for 2008
       interpolate_2009_2010=interpolate_2009_2010.drop([602,607], axis=0)
       print(interpolate_2009_2010)


       # ## Populate 2007 nan values with computed mean
       # In[14]:
       # Fertilizers
       i=0
       for i in range(1,16):
              interpolate_2009_to_2016.iloc[0:i,1:2] = interpolate_2004_2005['mean'].iloc[0:i]

       #Pesticides
       j=0
       for j in range(1,7):
              pesticides_2017.iloc[0:j,] = pesticides_2015_2016['mean'].iloc[0:j]

       # ## Populate 2008 nan values with computed mean
       # In[15]:
       j=0
       for j in range(1,16):
              interpolate_2009_to_2016.iloc[0:j,2:3] = interpolate_2009_2010['mean'].iloc[0:j]
                
       interpolate_2009_to_2016.iloc[15, 1] = 1437658.50
       print(interpolate_2009_to_2016.iloc[15, 1])
       interpolate_2009_to_2016.iloc[15, 2] = 171351.00

       # In[16]:
       X_ten_years_fertilizers_interpolated = interpolate_2009_to_2016
       X_ten_years_fertilizers_interpolated
       fertilizer_2017_interpolated = interpolate_2017
       print(fertilizer_2017_interpolated)

       # #### TODO: Combine pesticdes and fertilizers data
       # In[17]:
       print(X_ten_years_pesticides)
       print(X_ten_years_fertilizers_interpolated)

       pesticides_without_Item = X_ten_years_pesticides.copy().drop(['Item'], axis=1)
       pesticides_2017_wo_item = pesticides_2017
       fertilizers_without_Item = X_ten_years_fertilizers_interpolated.copy().drop(['Item'], axis=1)
       fertilizer_2017_wo_item = fertilizer_2017_interpolated
       # torch.tensor(X_ten_years_pesticides.values.astype(np.float64))
       print(X_ten_years_pesticides.values)
       print(X_ten_years_fertilizers_interpolated.values)
       print(pesticides_2017_wo_item.values)
       print(fertilizer_2017_wo_item.values)

       print(pesticides_without_Item)
       pesticides_tensor = torch.from_numpy(pesticides_without_Item.values)
       print("\t Pesticides tensor", pesticides_tensor)
       pesticides_2017_tensor = torch.from_numpy(pesticides_2017_wo_item.values)

       print(fertilizers_without_Item)
       fertilizers_tensor = torch.from_numpy(fertilizers_without_Item.values)
       fertilizer_2017_tensor = torch.from_numpy(fertilizer_2017_wo_item.values)
       print("\t Fertilizers tensor", fertilizers_tensor)

       # In[18]:
       print("Train : \n",pesticides_tensor.shape)
       print("Train: \n",fertilizers_tensor.shape)
       print("Test: \n",pesticides_2017_tensor.shape)
       print("Test: \n",fertilizer_2017_tensor.shape)

       # TODO: Normalize inputs before concatenation
       pesticides_and_fertilizers = torch.cat((pesticides_tensor, fertilizers_tensor), dim=0)
       print(pesticides_and_fertilizers.shape)
       print("\n")
       print("Training X data for 10 years: ",pesticides_and_fertilizers)
       print("\n")
       pesticides_and_fertilizers_2017 = torch.cat((pesticides_2017_tensor, fertilizer_2017_tensor), dim=0)
       print(pesticides_and_fertilizers_2017.shape)
       print("\n")
       print("Testing X data for 1 year: ",pesticides_and_fertilizers_2017)
       print("\n")

       # Training + Validation data : Input
       X_train_ten = pesticides_and_fertilizers;
       X_test = pesticides_and_fertilizers_2017

       # #### TODO: Create output vector

       # In[19]:
       print(yield_frame.columns)
       #%%
       yield_frame.head
       #%%
       Australia = yield_frame['Area']=="Australia"
       Production = yield_frame['Element']=="Production"
       Australia_yield = yield_frame[Australia]
       Australia_yield_production = Australia_yield[Production]
       print(yield_frame[Australia])
       #%%
       # Get the yield data for the year 2017 only
       Australia_yield_2017 = Australia_yield_production.drop(['Area Code', 'Area', 'Item Code', 'Element Code','Element',
              'Unit','Y1961', 'Y1962', 'Y1963', 'Y1964', 'Y1965', 'Y1966', 'Y1967', 'Y1968', 'Y1969', 'Y1970', 'Y1971', 'Y1972', 'Y1973', 
              'Y1974', 'Y1975', 'Y1976', 'Y1977', 'Y1978', 'Y1979', 'Y1980', 'Y1981', 'Y1982', 'Y1983', 'Y1984', 'Y1985', 'Y1986', 'Y1987',
              'Y1988', 'Y1989', 'Y1990', 'Y1991', 'Y1992', 'Y1993', 'Y1994', 'Y1995', 'Y1996', 'Y1997', 'Y1998', 'Y1999', 'Y2000', 'Y2001', 
              'Y2002', 'Y2003', 'Y2004', 'Y2005', 'Y2006', 'Y2007', 'Y2008', 'Y2009', 'Y2010', 'Y2011', 'Y2012', 'Y2013', 'Y2014', 'Y2015', 
              'Y2016', 'Y2018'], axis=1)
       print("\n")
       print("Test Y data: ",Australia_yield_2017)
       print("\n")
       print(Australia_yield_2017.columns)
       print(Australia_yield_2017['Item'].unique())
       print("Number of unique crops in 2017: ",len(Australia_yield_2017['Item'].unique()))

       Australia_yield_2007_to_2016 = Australia_yield_production.drop(['Area Code', 'Area', 'Item Code', 'Element Code','Element',
              'Unit','Y1961', 'Y1962', 'Y1963', 'Y1964', 'Y1965', 'Y1966', 'Y1967', 'Y1968', 'Y1969', 'Y1970', 'Y1971', 'Y1972', 'Y1973', 
              'Y1974', 'Y1975', 'Y1976', 'Y1977', 'Y1978', 'Y1979', 'Y1980', 'Y1981', 'Y1982', 'Y1983', 'Y1984', 'Y1985', 'Y1986', 'Y1987',
              'Y1988', 'Y1989', 'Y1990', 'Y1991', 'Y1992', 'Y1993', 'Y1994', 'Y1995', 'Y1996', 'Y1997', 'Y1998', 'Y1999', 'Y2000', 'Y2001', 
              'Y2002', 'Y2003', 'Y2004', 'Y2005', 'Y2006', 'Y2017','Y2018'], axis=1)
       print(Australia_yield_2007_to_2016.columns)
       print(Australia_yield_2007_to_2016['Item'].unique())
       print("\n")
       print("Train Y data: ",Australia_yield_2007_to_2016)
       print("\n")
       print("Number of unique crops from 2007 to 2016",len(Australia_yield_2007_to_2016['Item'].unique()))

       # In[20]:
       yield_without_Item = Australia_yield_2007_to_2016.copy().drop(['Item'],axis=1)
       yield_without_Item_2017 = Australia_yield_2017.copy().drop(['Item'],axis=1)

       print(yield_without_Item)
       yield_tensor = torch.from_numpy(yield_without_Item.values)
       print("Shape: ", yield_tensor.shape)
       print("\t Yield tensor", yield_tensor)
       print(yield_without_Item_2017)
       yield_tensor_2017 = torch.from_numpy(yield_without_Item_2017.values)
       print("Shape: ", yield_tensor_2017.shape)
       print("\t Yield tensor", yield_tensor_2017)

       # Training + Validation: Output
       Y_train_ten = yield_tensor
       # Test Y
       Y_test = yield_tensor_2017

       print("Train X: \n",X_train_ten.shape)
       print("Train Y: \n",Y_train_ten.shape)
       print("Test X: \n",X_test.shape)
       print("Test Y: \n",Y_test.shape)
       
       return [X_train_ten, Y_train_ten, X_test, Y_test]

In [36]:

training_input = load_data()[0]
training_ouput = load_data()[1]
test_input = load_data()[2]
test_output = load_data()[3]

x = training_input
y = training_ouput

04, 5.0922e+04, 6.3416e+04],
        [7.2630e+03, 9.6109e+03, 8.5209e+03, 9.4395e+03, 1.0662e+04, 1.0899e+04,
         1.0113e+04, 1.1160e+04, 1.1399e+04, 1.4196e+04],
        [2.2404e+04, 2.9647e+04, 2.6284e+04, 2.9118e+04, 3.2890e+04, 3.3619e+04,
         3.1195e+04, 3.4426e+04, 3.5161e+04, 4.3789e+04],
        [2.3249e+03, 3.0765e+03, 2.7276e+03, 3.0216e+03, 3.4131e+03, 3.4887e+03,
         3.2371e+03, 3.5725e+03, 3.6487e+03, 4.5441e+03],
        [4.5447e+02, 6.0139e+02, 5.3318e+02, 5.9066e+02, 6.6719e+02, 6.8197e+02,
         6.3279e+02, 6.9835e+02, 7.1325e+02, 8.8827e+02],
        [4.5447e+02, 6.0139e+02, 5.3318e+02, 5.9066e+02, 6.6719e+02, 6.8197e+02,
         6.3279e+02, 6.9835e+02, 7.1325e+02, 8.8827e+02],
        [3.0365e+05, 1.6086e+05,        nan, 1.6086e+05, 4.7329e+04, 1.5680e+05,
         1.3463e+05, 1.3892e+05, 1.9200e+05, 1.7258e+05],
        [2.0392e+04, 2.9583e+05, 7.4000e+01, 5.9159e+05, 2.6952e+05, 6.4043e+05,
         1.4296e+05, 3.7954e+05, 4.9362e+05, 4.8665e+05]

In [37]:

# Transforms

# Training data
x_tensor = torch.Tensor(x.float())
x_data = (x - x.mean())/(x.max() - x.min())
test = torch.Tensor([[0,4,5]])
x_tensor_normalized = x.normal_()
y_data = (y.normal_())

transformed_x = torch.reshape(x_tensor_normalized,(1,220))
transformed_y = y_data
y = torch.reshape(y, (1,1010))


In [38]:
# Test data
transformed_test_x = torch.Tensor(test_input.float())
transformed_test_x = transformed_test_x.normal_()
transformed_test_y = torch.Tensor(test_output.float())
transformed_test_y = transformed_test_y.normal_()

In [39]:
# Define model
class NeuralNet(nn.Module):
       def __init__(self, D_in, H1, H2, H3, H4, H5, H6, D_out):
              super(NeuralNet, self).__init__()
              self.linear1 = nn.Linear(D_in, H1)
              self.linear2 = nn.Linear(H1, H2)
              self.linear3 = nn.Linear(H2, H3)
              self.linear4 = nn.Linear(H3, H4)
              self.linear5 = nn.Linear(H4, H5)
              self.linear6 = nn.Linear(H5, H6)
              self.linear7 = nn.Linear(H6, D_out)
              self.relu = nn.ReLU()
       def forward(self, x):
              y_pred = torch.tanh(self.linear1(x))
              y_pred = self.linear2(y_pred)
              y_pred = torch.tanh(self.linear3(y_pred))
              y_pred = self.linear4(y_pred)
              y_pred = torch.tanh(self.linear5(y_pred))
              y_pred = self.linear6(y_pred)
              y_pred = torch.sigmoid(self.linear7(y_pred))
              return y_pred


In [40]:


from functools import partial
import numpy as np
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import random_split
import torchvision
import torchvision.transforms as transforms
from random import randint
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler


In [41]:


def get_accuracy(model, predicted, actual, threshold_percentage):
    num_items = len(actual)
    X = predicted.view(num_items)
    Y = actual.view(num_items)
    num_correct = torch.sum(torch.abs(X-Y)<torch.abs(threshold_percentage*Y))
    accuracy = (num_correct.item()*100.0/num_items)
    return (num_correct, accuracy)
    

def random_split_training(trainset_x, trainset_y):
       # TODO:
       # Select 2 random numbers between 1 and 10
       # Use them to index into x and y.
       # That will give us the training and validation set
       # Rest of the indices are thus our training set
       index1 = randint(0,9)
       index2 = randint(0,9)
       indices=[]
       i=0
       for i in range(0,10):
              if(i not in [index1,index2]):
                 indices.append(i)    
       
       indices = torch.tensor(indices)
       val_indices = torch.tensor([index1, index2])
       t_subset_x = torch.index_select(trainset_x, 1, indices)
       t_subset_y = torch.index_select(trainset_y, 1, indices)
       v_subset_x = torch.index_select(trainset_x, 1,val_indices)
       v_subset_y = torch.index_select(trainset_y, 1, val_indices)
       return (t_subset_x, t_subset_y, v_subset_x, v_subset_y)
# 


In [42]:
# Function to train and validate data. Save the best model based on validation loss
# 
# params:
# config: hyperparameter search space
# 
def train_crop_yield(config):
       net = NeuralNet(176, config['H1'], config['H2'], config['H3'], config["H4"], config["H5"], config["H6"],8)
       criterion = torch.nn.MSELoss()
       optimizer = torch.optim.SGD(net.parameters(), lr=config["lr"], momentum=0.9)
       # train_set = torch.cat((transformed_x, transformed_y), dim=0)
       # test_set = torch.cat((test_input, test_output), dim=0)
       
       x_by_years = transformed_x.reshape(22,10)
       y_by_years = transformed_y.reshape(101,10)
       
       train_subset_x, train_subset_y, val_subset_x, val_subset_y = random_split_training(x_by_years, y_by_years)
       
       for epoch in range(500):
              running_loss = 0.0
              epoch_steps = 0
              # Zero the accumulated gradients
              optimizer.zero_grad()
              # forward + backward + optimize
              output = net(train_subset_x.float().reshape(1,176))
              loss = criterion(output, train_subset_y.float())
              loss.backward()
              optimizer.step()
              # print statistics
              running_loss += loss.item()
              epoch_steps += 1
              if epoch%5==4:
                     print("[%d] loss: %.3f"%(epoch+1, running_loss/epoch_steps))
                     running_loss=0.0
       
       # Validation loss
       net = NeuralNet(44, config['H1'], config['H2'], config['H3'], config["H4"], config["H5"], config["H6"],2)
       val_loss = 0.0
       val_steps = 0
       total = 0
       correct = 0
       with torch.no_grad():
              val_output = net(val_subset_x.float().reshape(1,44))
              total = val_subset_y.size(0)
              correct = torch.sum(torch.abs(val_output-val_subset_y)<torch.abs(0.10*val_subset_y))
              loss = criterion(val_output, val_subset_y.float())
              val_loss += loss.cpu().numpy()
              torch.save((net.state_dict(), optimizer.state_dict()), './trained_net.dat')
              tune.report(loss=(val_loss), accuracy=(correct/total))
       print("\n Finished Training \n")
              

In [43]:
# Evaluation:
def test_accuracy(net, device='cpu'):
       net.eval()
       with torch.no_grad():
              predicted = net(transformed_test_x.float().reshape(1,22))
              return get_accuracy(net, predicted.reshape(101,1), transformed_test_y.reshape(101,1), 0.50)
       
# Define hyperparameters
config = {
       "H1":tune.sample_from(lambda _: 2**np.random.randint(7,9)),
       "H2":tune.sample_from(lambda _: 2**np.random.randint(5,7)),
       "H3":tune.sample_from(lambda _: 2**np.random.randint(7,12)),
       "H4":tune.sample_from(lambda _: 2**np.random.randint(7,12)),
       "H5":tune.sample_from(lambda _: 2**np.random.randint(5,7)),
       "H6":tune.sample_from(lambda _: 2**np.random.randint(5,9)),
       "lr":tune.loguniform(1e-2, 1e-1)
}

# Scheduler. Randomly try out combination of hyperparameters
scheduler = ASHAScheduler(
       metric="loss",
       mode="min",
       max_t=10,
       grace_period=1,
       reduction_factor=2
)

reporter=CLIReporter(metric_columns=["loss","accuracy", "training_iteration"])

result = tune.run(
       partial(train_crop_yield),
       config=config,
       num_samples=10,
       scheduler=scheduler
)

best_trial = result.get_best_trial("loss", "min", "last")

print("Best trial config: {}".format(best_trial.config))
print("Best trial final validation loss: {}".format(best_trial.last_result["loss"]))
print("Best trial fnal validation accuracy: {}".format(best_trial.last_result["accuracy"]))

best_trained_model = NeuralNet(22, best_trial.config["H1"], best_trial.config["H2"], best_trial.config["H3"],best_trial.config["H4"],best_trial.config["H5"],best_trial.config["H6"],101)

device="cpu"
best_trained_model.to(device)

# best_checkpoint_dir = best_trial.checkpoint.value
# print(" \n",type(best_trial.evaluated_params))
# model_state, optimizer_state = torch.load(os.path.join(best_checkpoint_dir, "checkpoint"))
# best_trained_model.load_state_dict(model_state)

num_correct, test_acc = test_accuracy(best_trained_model, device)
print("Best trial test set accuracy: {}".format(test_acc))


2021-05-16 17:50:47,092	WARNING experiment.py:294 -- No name detected on trainable. Using DEFAULT.
2021-05-16 17:50:47,097	INFO registry.py:65 -- Detected unknown callable for trainable. Converting to class.


Trial name,status,loc,H1,H2,H3,H4,H5,H6,lr
DEFAULT_9db58_00000,RUNNING,,256,32,256,2048,32,128,0.0139584
DEFAULT_9db58_00001,PENDING,,256,32,1024,128,64,256,0.0863757
DEFAULT_9db58_00002,PENDING,,256,64,128,512,64,256,0.0285607
DEFAULT_9db58_00003,PENDING,,128,32,1024,1024,32,32,0.0213535
DEFAULT_9db58_00004,PENDING,,256,32,128,2048,64,32,0.0221542
DEFAULT_9db58_00005,PENDING,,128,32,256,256,64,64,0.0478909
DEFAULT_9db58_00006,PENDING,,256,32,128,1024,64,128,0.0777031
DEFAULT_9db58_00007,PENDING,,256,32,2048,128,64,256,0.0190126
DEFAULT_9db58_00008,PENDING,,128,32,2048,2048,32,256,0.0223487
DEFAULT_9db58_00009,PENDING,,256,32,1024,128,32,32,0.0579518


(pid=80050) 2021-05-16 17:50:51,218	ERROR function_runner.py:254 -- Runner Thread raised error.
(pid=80050) Traceback (most recent call last):
(pid=80050)   File "/opt/anaconda3/lib/python3.7/site-packages/ray/tune/function_runner.py", line 248, in run
(pid=80050)     self._entrypoint()
(pid=80050)   File "/opt/anaconda3/lib/python3.7/site-packages/ray/tune/function_runner.py", line 316, in entrypoint
(pid=80050)     self._status_reporter.get_checkpoint())
(pid=80050)   File "/opt/anaconda3/lib/python3.7/site-packages/ray/tune/function_runner.py", line 580, in _trainable_func
(pid=80050)     output = fn()
(pid=80050)   File "<ipython-input-42-d9d213d1f510>", line 24, in train_crop_yield
(pid=80050) RuntimeError: shape '[1, 176]' is invalid for input of size 198
(pid=80050) Exception in thread Thread-2:
(pid=80050) Traceback (most recent call last):
(pid=80050)   File "/opt/anaconda3/lib/python3.7/threading.py", line 926, in _bootstrap_inner
(pid=80050)     self.run()
(pid=80050)   File

Trial name,status,loc,H1,H2,H3,H4,H5,H6,lr
DEFAULT_9db58_00001,RUNNING,,256,32,1024,128,64,256,0.0863757
DEFAULT_9db58_00002,RUNNING,,256,64,128,512,64,256,0.0285607
DEFAULT_9db58_00003,RUNNING,,128,32,1024,1024,32,32,0.0213535
DEFAULT_9db58_00004,RUNNING,,256,32,128,2048,64,32,0.0221542
DEFAULT_9db58_00005,PENDING,,128,32,256,256,64,64,0.0478909
DEFAULT_9db58_00006,PENDING,,256,32,128,1024,64,128,0.0777031
DEFAULT_9db58_00007,PENDING,,256,32,2048,128,64,256,0.0190126
DEFAULT_9db58_00008,PENDING,,128,32,2048,2048,32,256,0.0223487
DEFAULT_9db58_00009,PENDING,,256,32,1024,128,32,32,0.0579518
DEFAULT_9db58_00000,ERROR,,256,32,256,2048,32,128,0.0139584


(pid=80059) 2021-05-16 17:50:55,025	ERROR function_runner.py:254 -- Runner Thread raised error.
(pid=80059) Traceback (most recent call last):
(pid=80059)   File "/opt/anaconda3/lib/python3.7/site-packages/ray/tune/function_runner.py", line 248, in run
(pid=80059)     self._entrypoint()
(pid=80059)   File "/opt/anaconda3/lib/python3.7/site-packages/ray/tune/function_runner.py", line 316, in entrypoint
(pid=80059)     self._status_reporter.get_checkpoint())
(pid=80059)   File "/opt/anaconda3/lib/python3.7/site-packages/ray/tune/function_runner.py", line 580, in _trainable_func
(pid=80059)     output = fn()
(pid=80059)   File "<ipython-input-42-d9d213d1f510>", line 24, in train_crop_yield
(pid=80059) RuntimeError: shape '[1, 176]' is invalid for input of size 198
(pid=80059) Exception in thread Thread-2:
(pid=80059) Traceback (most recent call last):
(pid=80059)   File "/opt/anaconda3/lib/python3.7/threading.py", line 926, in _bootstrap_inner
(pid=80059)     self.run()
(pid=80059)   File

...

(pid=80074) 2021-05-16 17:50:58,619	ERROR function_runner.py:254 -- Runner Thread raised error.
(pid=80074) Traceback (most recent call last):
(pid=80074)   File "/opt/anaconda3/lib/python3.7/site-packages/ray/tune/function_runner.py", line 248, in run
(pid=80074)     self._entrypoint()
(pid=80074)   File "/opt/anaconda3/lib/python3.7/site-packages/ray/tune/function_runner.py", line 316, in entrypoint
(pid=80074)     self._status_reporter.get_checkpoint())
(pid=80074)   File "/opt/anaconda3/lib/python3.7/site-packages/ray/tune/function_runner.py", line 580, in _trainable_func
(pid=80074)     output = fn()
(pid=80074)   File "<ipython-input-42-d9d213d1f510>", line 24, in train_crop_yield
(pid=80074) RuntimeError: shape '[1, 176]' is invalid for input of size 198
(pid=80074) Exception in thread Thread-2:
(pid=80074) Traceback (most recent call last):
(pid=80074)   File "/opt/anaconda3/lib/python3.7/threading.py", line 926, in _bootstrap_inner
(pid=80074)     self.run()
(pid=80074)   File

Trial name,status,loc,H1,H2,H3,H4,H5,H6,lr
DEFAULT_9db58_00006,RUNNING,,256,32,128,1024,64,128,0.0777031
DEFAULT_9db58_00007,RUNNING,,256,32,2048,128,64,256,0.0190126
DEFAULT_9db58_00008,RUNNING,,128,32,2048,2048,32,256,0.0223487
DEFAULT_9db58_00000,ERROR,,256,32,256,2048,32,128,0.0139584
DEFAULT_9db58_00001,ERROR,,256,32,1024,128,64,256,0.0863757
DEFAULT_9db58_00002,ERROR,,256,64,128,512,64,256,0.0285607
DEFAULT_9db58_00003,ERROR,,128,32,1024,1024,32,32,0.0213535
DEFAULT_9db58_00004,ERROR,,256,32,128,2048,64,32,0.0221542
DEFAULT_9db58_00005,ERROR,,128,32,256,256,64,64,0.0478909
DEFAULT_9db58_00009,ERROR,,256,32,1024,128,32,32,0.0579518


2021-05-16 17:51:03,798	ERROR trial_runner.py:732 -- Trial DEFAULT_9db58_00006: Error processing event.
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/ray/tune/trial_runner.py", line 702, in _process_trial
    results = self.trial_executor.fetch_result(trial)
  File "/opt/anaconda3/lib/python3.7/site-packages/ray/tune/ray_trial_executor.py", line 686, in fetch_result
    result = ray.get(trial_future[0], timeout=DEFAULT_GET_TIMEOUT)
  File "/opt/anaconda3/lib/python3.7/site-packages/ray/_private/client_mode_hook.py", line 47, in wrapper
    return func(*args, **kwargs)
  File "/opt/anaconda3/lib/python3.7/site-packages/ray/worker.py", line 1481, in get
    raise value.as_instanceof_cause()
ray.exceptions.RayTaskError(TuneError): ray::ImplicitFunc.train_buffered() (pid=80079, ip=10.201.135.226)
  File "python/ray/_raylet.pyx", line 505, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 449, in ray._raylet.execute_task.function_execu

Trial name,status,loc,H1,H2,H3,H4,H5,H6,lr
DEFAULT_9db58_00000,ERROR,,256,32,256,2048,32,128,0.0139584
DEFAULT_9db58_00001,ERROR,,256,32,1024,128,64,256,0.0863757
DEFAULT_9db58_00002,ERROR,,256,64,128,512,64,256,0.0285607
DEFAULT_9db58_00003,ERROR,,128,32,1024,1024,32,32,0.0213535
DEFAULT_9db58_00004,ERROR,,256,32,128,2048,64,32,0.0221542
DEFAULT_9db58_00005,ERROR,,128,32,256,256,64,64,0.0478909
DEFAULT_9db58_00006,ERROR,,256,32,128,1024,64,128,0.0777031
DEFAULT_9db58_00007,ERROR,,256,32,2048,128,64,256,0.0190126
DEFAULT_9db58_00008,ERROR,,128,32,2048,2048,32,256,0.0223487
DEFAULT_9db58_00009,ERROR,,256,32,1024,128,32,32,0.0579518


TuneError: ('Trials did not complete', [DEFAULT_9db58_00000, DEFAULT_9db58_00001, DEFAULT_9db58_00002, DEFAULT_9db58_00003, DEFAULT_9db58_00004, DEFAULT_9db58_00005, DEFAULT_9db58_00006, DEFAULT_9db58_00007, DEFAULT_9db58_00008, DEFAULT_9db58_00009])

In [44]:


import torch
print(torch.__version__)
import pandas as pd
print(pd.__version__)
from torch.utils.data import DataLoader,ConcatDataset
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error



1.8.1
1.2.4


In [45]:

# ## Read pesticides and fertilizer products data


In [46]:
def load_data():
       pesticides_frame = pd.read_csv("Inputs_Pesticides_Use_E_All_Data_NOFLAG.csv", engine='python')
       pesticides_frame.head
  
       # Read fertilizer data in the form of products
       fertilizers_frame = pd.read_csv("Inputs_FertilizersProduct_E_All_Data_NOFLAG.csv", engine='python')
       fertilizers_frame.head
       
       # Read the crop yield data
       yield_frame = pd.read_csv("Production_Crops_E_All_Data_NOFLAG.csv",engine='python')
       yield_frame.head

       # %<br>
       # Get the fertilizer usage in terms of agriculture

       # In[4]:     
       print("Fertilizer products",fertilizers_frame.columns)
       print("\n")
       print("Input pesticides", pesticides_frame.columns)
       
       # Filter out the data for Australia
       is_Country_Australia_Products = fertilizers_frame['Area']=="Australia"
       Australia_Fertilizers_Products = fertilizers_frame[is_Country_Australia_Products]
       # In[5]:
       is_Country_Australia_Pesticides = pesticides_frame['Area']=="Australia"
       Australia_Pesticides = pesticides_frame[is_Country_Australia_Pesticides]

       # Get the fertilizer usage for agricultural use
       # In[6]:
       is_Agricultural = Australia_Fertilizers_Products['Element']=="Agricultural Use"
       Australia_Fertilizers_Products_Agricultural = Australia_Fertilizers_Products[is_Agricultural]
       print("Fertilizers \n")
       print(Australia_Fertilizers_Products_Agricultural.columns)
       print(Australia_Fertilizers_Products_Agricultural['Item'])
       print("\n")
       print("Pesticides \n")
       print(Australia_Pesticides.columns)
       print(Australia_Pesticides['Item'])
       
       # %<br>
       # 7 years for training - 2007 to 2013<br>
       # 3 years fro validation - 2014 to 2016<br>
       # 2017 for testing
       # In[7]:
       X_ten_years_fertlizers = Australia_Fertilizers_Products_Agricultural.drop(['Area Code', 'Area', 'Item Code', 'Element Code','Element',
              'Unit','Y2002', 'Y2003', 'Y2004', 'Y2005', 'Y2006', 'Y2017'], axis=1)
       print(X_ten_years_fertlizers)
       print(X_ten_years_fertlizers.columns)
       fertilizer_2017 = Australia_Fertilizers_Products_Agricultural['Y2017']
       print("2017 \n",fertilizer_2017)

       # In[8]:
       X_ten_years_pesticides = Australia_Pesticides.drop(['Area Code', 'Area', 'Item Code', 'Element Code', 'Element',
              'Unit', 'Y1990', 'Y1991', 'Y1992', 'Y1993', 'Y1994', 'Y1995', 'Y1996', 'Y1997', 'Y1998', 'Y1999', 'Y2000', 'Y2001', 'Y2002', 'Y2003', 'Y2004', 'Y2005', 'Y2006', 'Y2017'], axis=1)
       print(X_ten_years_pesticides)
       print(X_ten_years_pesticides.columns)
       pesticides_2017 = Australia_Pesticides['Y2017']
       print("2017 \n",pesticides_2017)

       # %<br>
       # Drop Ammonia anyhydrous and Ammonia nitrate from 2007 to 2013. It has not been used in these years.
       # In[9]:
       X_ten_years_fertlizers = X_ten_years_fertlizers.drop([602, 607], axis=0)
       print(X_ten_years_fertlizers)
       print(X_ten_years_fertlizers.columns)
       fertilizer_2017 = fertilizer_2017.drop([602, 607], axis=0)
       print(fertilizer_2017)
       
       # %<br>
       # 2009 to 2013
       # ## Handle missing values from 2009 to 2013
       # In[10]:
       interpolate_2009_to_2016 = X_ten_years_fertlizers.interpolate()
       interpolate_2009_to_2016
       interpolate_2017 = fertilizer_2017.interpolate()
       print("interpolated 2017 fertilizers \n",interpolate_2017)
       # TODO: Don't just interpolate. Use the means of the previous 2  years
       # interpolate_pesticides_2017 = pesticides_2017.interpolate()
       # print("interpolated 2017 pesticides \n",interpolate_pesticides_2017)

       # %<br>
       #  Interpolate missing fertlizers data<br>
       #  Replace na with mean values of 2004,2005,2009,2010
       # ##  Replace 2007 nan values with the mean of 2004 and 2005

       # In[11]:

       interpolate_2004_2005 = Australia_Fertilizers_Products_Agricultural[['Y2004','Y2005']].copy().interpolate()
       index_names = interpolate_2004_2005.index
       index_names
       interpolate_2004_2005=interpolate_2004_2005.drop([602,607], axis=0)
       interpolate_2004_2005['mean'] = interpolate_2004_2005.mean(axis=1)
       print("Mean of 2004 and 2005 ==== \n")
       print(interpolate_2004_2005)
       pesticides_2015_2016 = Australia_Pesticides[['Y2015','Y2016']]
       pesticides_2015_2016['mean'] = pesticides_2015_2016.mean(axis=1)
       print("Mean of 2015 and 2016 === \n")
       print(pesticides_2015_2016)

       # Use this for 2007

       # In[ ]:
       # ##  Replace 2008 nan values with the mean of 2009 and 2010

       # In[12]:
       interpolate_2009_2010 = Australia_Fertilizers_Products_Agricultural[['Y2009','Y2010']].copy().interpolate()
       index_names = interpolate_2009_2010.index
       index_names

       # In[13]:
       interpolate_2009_2010['mean'] = interpolate_2009_2010.mean(axis=1)
       print("Mean of 2009 and 2010 ==== \n")
       # Use this for 2008
       interpolate_2009_2010=interpolate_2009_2010.drop([602,607], axis=0)
       print(interpolate_2009_2010)


       # ## Populate 2007 nan values with computed mean
       # In[14]:
       # Fertilizers
       i=0
       for i in range(1,16):
              interpolate_2009_to_2016.iloc[0:i,1:2] = interpolate_2004_2005['mean'].iloc[0:i]

       #Pesticides
       j=0
       for j in range(1,7):
              pesticides_2017.iloc[0:j,] = pesticides_2015_2016['mean'].iloc[0:j]

       # ## Populate 2008 nan values with computed mean
       # In[15]:
       j=0
       for j in range(1,16):
              interpolate_2009_to_2016.iloc[0:j,2:3] = interpolate_2009_2010['mean'].iloc[0:j]
                
       interpolate_2009_to_2016.iloc[15, 1] = 1437658.50
       print(interpolate_2009_to_2016.iloc[15, 1])
       interpolate_2009_to_2016.iloc[15, 2] = 171351.00

       # In[16]:
       X_ten_years_fertilizers_interpolated = interpolate_2009_to_2016
       X_ten_years_fertilizers_interpolated
       fertilizer_2017_interpolated = interpolate_2017
       print(fertilizer_2017_interpolated)

       # #### TODO: Combine pesticdes and fertilizers data
       # In[17]:
       print(X_ten_years_pesticides)
       print(X_ten_years_fertilizers_interpolated)

       pesticides_without_Item = X_ten_years_pesticides.copy().drop(['Item'], axis=1)
       pesticides_2017_wo_item = pesticides_2017
       fertilizers_without_Item = X_ten_years_fertilizers_interpolated.copy().drop(['Item'], axis=1)
       fertilizer_2017_wo_item = fertilizer_2017_interpolated
       # torch.tensor(X_ten_years_pesticides.values.astype(np.float64))
       print(X_ten_years_pesticides.values)
       print(X_ten_years_fertilizers_interpolated.values)
       print(pesticides_2017_wo_item.values)
       print(fertilizer_2017_wo_item.values)

       print(pesticides_without_Item)
       pesticides_tensor = torch.from_numpy(pesticides_without_Item.values)
       print("\t Pesticides tensor", pesticides_tensor)
       pesticides_2017_tensor = torch.from_numpy(pesticides_2017_wo_item.values)

       print(fertilizers_without_Item)
       fertilizers_tensor = torch.from_numpy(fertilizers_without_Item.values)
       fertilizer_2017_tensor = torch.from_numpy(fertilizer_2017_wo_item.values)
       print("\t Fertilizers tensor", fertilizers_tensor)

       # In[18]:
       print("Train : \n",pesticides_tensor.shape)
       print("Train: \n",fertilizers_tensor.shape)
       print("Test: \n",pesticides_2017_tensor.shape)
       print("Test: \n",fertilizer_2017_tensor.shape)

       # TODO: Normalize inputs before concatenation
       pesticides_and_fertilizers = torch.cat((pesticides_tensor, fertilizers_tensor), dim=0)
       print(pesticides_and_fertilizers.shape)
       print("\n")
       print("Training X data for 10 years: ",pesticides_and_fertilizers)
       print("\n")
       pesticides_and_fertilizers_2017 = torch.cat((pesticides_2017_tensor, fertilizer_2017_tensor), dim=0)
       print(pesticides_and_fertilizers_2017.shape)
       print("\n")
       print("Testing X data for 1 year: ",pesticides_and_fertilizers_2017)
       print("\n")

       # Training + Validation data : Input
       X_train_ten = pesticides_and_fertilizers;
       X_test = pesticides_and_fertilizers_2017

       # #### TODO: Create output vector

       # In[19]:
       print(yield_frame.columns)
       #%%
       yield_frame.head
       #%%
       Australia = yield_frame['Area']=="Australia"
       Production = yield_frame['Element']=="Production"
       Australia_yield = yield_frame[Australia]
       Australia_yield_production = Australia_yield[Production]
       print(yield_frame[Australia])
       #%%
       # Get the yield data for the year 2017 only
       Australia_yield_2017 = Australia_yield_production.drop(['Area Code', 'Area', 'Item Code', 'Element Code','Element',
              'Unit','Y1961', 'Y1962', 'Y1963', 'Y1964', 'Y1965', 'Y1966', 'Y1967', 'Y1968', 'Y1969', 'Y1970', 'Y1971', 'Y1972', 'Y1973', 
              'Y1974', 'Y1975', 'Y1976', 'Y1977', 'Y1978', 'Y1979', 'Y1980', 'Y1981', 'Y1982', 'Y1983', 'Y1984', 'Y1985', 'Y1986', 'Y1987',
              'Y1988', 'Y1989', 'Y1990', 'Y1991', 'Y1992', 'Y1993', 'Y1994', 'Y1995', 'Y1996', 'Y1997', 'Y1998', 'Y1999', 'Y2000', 'Y2001', 
              'Y2002', 'Y2003', 'Y2004', 'Y2005', 'Y2006', 'Y2007', 'Y2008', 'Y2009', 'Y2010', 'Y2011', 'Y2012', 'Y2013', 'Y2014', 'Y2015', 
              'Y2016', 'Y2018'], axis=1)
       print("\n")
       print("Test Y data: ",Australia_yield_2017)
       print("\n")
       print(Australia_yield_2017.columns)
       print(Australia_yield_2017['Item'].unique())
       print("Number of unique crops in 2017: ",len(Australia_yield_2017['Item'].unique()))

       Australia_yield_2007_to_2016 = Australia_yield_production.drop(['Area Code', 'Area', 'Item Code', 'Element Code','Element',
              'Unit','Y1961', 'Y1962', 'Y1963', 'Y1964', 'Y1965', 'Y1966', 'Y1967', 'Y1968', 'Y1969', 'Y1970', 'Y1971', 'Y1972', 'Y1973', 
              'Y1974', 'Y1975', 'Y1976', 'Y1977', 'Y1978', 'Y1979', 'Y1980', 'Y1981', 'Y1982', 'Y1983', 'Y1984', 'Y1985', 'Y1986', 'Y1987',
              'Y1988', 'Y1989', 'Y1990', 'Y1991', 'Y1992', 'Y1993', 'Y1994', 'Y1995', 'Y1996', 'Y1997', 'Y1998', 'Y1999', 'Y2000', 'Y2001', 
              'Y2002', 'Y2003', 'Y2004', 'Y2005', 'Y2006', 'Y2017','Y2018'], axis=1)
       print(Australia_yield_2007_to_2016.columns)
       print(Australia_yield_2007_to_2016['Item'].unique())
       print("\n")
       print("Train Y data: ",Australia_yield_2007_to_2016)
       print("\n")
       print("Number of unique crops from 2007 to 2016",len(Australia_yield_2007_to_2016['Item'].unique()))

       # In[20]:
       yield_without_Item = Australia_yield_2007_to_2016.copy().drop(['Item'],axis=1)
       yield_without_Item_2017 = Australia_yield_2017.copy().drop(['Item'],axis=1)

       print(yield_without_Item)
       yield_tensor = torch.from_numpy(yield_without_Item.values)
       print("Shape: ", yield_tensor.shape)
       print("\t Yield tensor", yield_tensor)
       print(yield_without_Item_2017)
       yield_tensor_2017 = torch.from_numpy(yield_without_Item_2017.values)
       print("Shape: ", yield_tensor_2017.shape)
       print("\t Yield tensor", yield_tensor_2017)

       # Training + Validation: Output
       Y_train_ten = yield_tensor
       # Test Y
       Y_test = yield_tensor_2017

       print("Train X: \n",X_train_ten.shape)
       print("Train Y: \n",Y_train_ten.shape)
       print("Test X: \n",X_test.shape)
       print("Test Y: \n",Y_test.shape)
       
       return [X_train_ten, Y_train_ten, X_test, Y_test]

In [47]:

training_input = load_data()[0]
training_ouput = load_data()[1]
test_input = load_data()[2]
test_output = load_data()[3]

x = training_input
y = training_ouput

04, 5.0922e+04, 6.3416e+04],
        [7.2630e+03, 9.6109e+03, 8.5209e+03, 9.4395e+03, 1.0662e+04, 1.0899e+04,
         1.0113e+04, 1.1160e+04, 1.1399e+04, 1.4196e+04],
        [2.2404e+04, 2.9647e+04, 2.6284e+04, 2.9118e+04, 3.2890e+04, 3.3619e+04,
         3.1195e+04, 3.4426e+04, 3.5161e+04, 4.3789e+04],
        [2.3249e+03, 3.0765e+03, 2.7276e+03, 3.0216e+03, 3.4131e+03, 3.4887e+03,
         3.2371e+03, 3.5725e+03, 3.6487e+03, 4.5441e+03],
        [4.5447e+02, 6.0139e+02, 5.3318e+02, 5.9066e+02, 6.6719e+02, 6.8197e+02,
         6.3279e+02, 6.9835e+02, 7.1325e+02, 8.8827e+02],
        [4.5447e+02, 6.0139e+02, 5.3318e+02, 5.9066e+02, 6.6719e+02, 6.8197e+02,
         6.3279e+02, 6.9835e+02, 7.1325e+02, 8.8827e+02],
        [3.0365e+05, 1.6086e+05,        nan, 1.6086e+05, 4.7329e+04, 1.5680e+05,
         1.3463e+05, 1.3892e+05, 1.9200e+05, 1.7258e+05],
        [2.0392e+04, 2.9583e+05, 7.4000e+01, 5.9159e+05, 2.6952e+05, 6.4043e+05,
         1.4296e+05, 3.7954e+05, 4.9362e+05, 4.8665e+05]

In [48]:

# Transforms

# Training data
x_tensor = torch.Tensor(x.float())
x_data = (x - x.mean())/(x.max() - x.min())
test = torch.Tensor([[0,4,5]])
x_tensor_normalized = x.normal_()
y_data = (y.normal_())

transformed_x = torch.reshape(x_tensor_normalized,(1,220))
transformed_y = y_data
y = torch.reshape(y, (1,1010))


In [49]:
# Test data
transformed_test_x = torch.Tensor(test_input.float())
transformed_test_x = transformed_test_x.normal_()
transformed_test_y = torch.Tensor(test_output.float())
transformed_test_y = transformed_test_y.normal_()

In [50]:
# Define model
class NeuralNet(nn.Module):
       def __init__(self, D_in, H1, H2, H3, H4, H5, H6, D_out):
              super(NeuralNet, self).__init__()
              self.linear1 = nn.Linear(D_in, H1)
              self.linear2 = nn.Linear(H1, H2)
              self.linear3 = nn.Linear(H2, H3)
              self.linear4 = nn.Linear(H3, H4)
              self.linear5 = nn.Linear(H4, H5)
              self.linear6 = nn.Linear(H5, H6)
              self.linear7 = nn.Linear(H6, D_out)
              self.relu = nn.ReLU()
       def forward(self, x):
              y_pred = torch.tanh(self.linear1(x))
              y_pred = self.linear2(y_pred)
              y_pred = torch.tanh(self.linear3(y_pred))
              y_pred = self.linear4(y_pred)
              y_pred = torch.tanh(self.linear5(y_pred))
              y_pred = self.linear6(y_pred)
              y_pred = torch.sigmoid(self.linear7(y_pred))
              return y_pred


In [51]:


from functools import partial
import numpy as np
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import random_split
import torchvision
import torchvision.transforms as transforms
from random import randint
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler


In [52]:


def get_accuracy(model, predicted, actual, threshold_percentage):
    num_items = len(actual)
    X = predicted.view(num_items)
    Y = actual.view(num_items)
    num_correct = torch.sum(torch.abs(X-Y)<torch.abs(threshold_percentage*Y))
    accuracy = (num_correct.item()*100.0/num_items)
    return (num_correct, accuracy)
    

def random_split_training(trainset_x, trainset_y):
       # TODO:
       # Select 2 random numbers between 1 and 10
       # Use them to index into x and y.
       # That will give us the training and validation set
       # Rest of the indices are thus our training set
       index1 = randint(0,9)
       index2 = randint(0,9)
       indices=[]
       i=0
       for i in range(0,10):
              if(i not in [index1,index2]):
                 indices.append(i)    
       
       indices = torch.tensor(indices)
       val_indices = torch.tensor([index1, index2])
       t_subset_x = torch.index_select(trainset_x, 1, indices)
       t_subset_y = torch.index_select(trainset_y, 1, indices)
       v_subset_x = torch.index_select(trainset_x, 1,val_indices)
       v_subset_y = torch.index_select(trainset_y, 1, val_indices)
       return (t_subset_x, t_subset_y, v_subset_x, v_subset_y)
# 


In [53]:
# Function to train and validate data. Save the best model based on validation loss
# 
# params:
# config: hyperparameter search space
# 
def train_crop_yield(config):
       net = NeuralNet(176, config['H1'], config['H2'], config['H3'], config["H4"], config["H5"], config["H6"],8)
       criterion = torch.nn.MSELoss()
       optimizer = torch.optim.SGD(net.parameters(), lr=config["lr"], momentum=0.9)
       # train_set = torch.cat((transformed_x, transformed_y), dim=0)
       # test_set = torch.cat((test_input, test_output), dim=0)
       
       x_by_years = transformed_x.reshape(22,10)
       y_by_years = transformed_y.reshape(101,10)
       
       train_subset_x, train_subset_y, val_subset_x, val_subset_y = random_split_training(x_by_years, y_by_years)
       
       for epoch in range(500):
              running_loss = 0.0
              epoch_steps = 0
              # Zero the accumulated gradients
              optimizer.zero_grad()
              # forward + backward + optimize
              output = net(train_subset_x.float().reshape(1,176))
              loss = criterion(output, train_subset_y.float())
              loss.backward()
              optimizer.step()
              # print statistics
              running_loss += loss.item()
              epoch_steps += 1
              if epoch%5==4:
                     print("[%d] loss: %.3f"%(epoch+1, running_loss/epoch_steps))
                     running_loss=0.0
       
       # Validation loss
       net = NeuralNet(44, config['H1'], config['H2'], config['H3'], config["H4"], config["H5"], config["H6"],2)
       val_loss = 0.0
       val_steps = 0
       total = 0
       correct = 0
       with torch.no_grad():
              val_output = net(val_subset_x.float().reshape(1,44))
              total = val_subset_y.size(0)
              correct = torch.sum(torch.abs(val_output-val_subset_y)<torch.abs(0.10*val_subset_y))
              loss = criterion(val_output, val_subset_y.float())
              val_loss += loss.cpu().numpy()
              torch.save((net.state_dict(), optimizer.state_dict()), './trained_net.dat')
              tune.report(loss=(val_loss), accuracy=(correct/total))
       print("\n Finished Training \n")
              

In [54]:
# Evaluation:
def test_accuracy(net, device='cpu'):
       net.eval()
       with torch.no_grad():
              predicted = net(transformed_test_x.float().reshape(1,22))
              return get_accuracy(net, predicted.reshape(101,1), transformed_test_y.reshape(101,1), 0.50)
       
# Define hyperparameters
config = {
       "H1":tune.sample_from(lambda _: 2**np.random.randint(7,9)),
       "H2":tune.sample_from(lambda _: 2**np.random.randint(5,7)),
       "H3":tune.sample_from(lambda _: 2**np.random.randint(7,12)),
       "H4":tune.sample_from(lambda _: 2**np.random.randint(7,12)),
       "H5":tune.sample_from(lambda _: 2**np.random.randint(5,7)),
       "H6":tune.sample_from(lambda _: 2**np.random.randint(5,9)),
       "lr":tune.loguniform(1e-2, 1e-1)
}

# Scheduler. Randomly try out combination of hyperparameters
scheduler = ASHAScheduler(
       metric="loss",
       mode="min",
       max_t=10,
       grace_period=1,
       reduction_factor=2
)

reporter=CLIReporter(metric_columns=["loss","accuracy", "training_iteration"])

result = tune.run(
       partial(train_crop_yield),
       config=config,
       num_samples=10,
       scheduler=scheduler
)

best_trial = result.get_best_trial("loss", "min", "last")

print("Best trial config: {}".format(best_trial.config))
print("Best trial final validation loss: {}".format(best_trial.last_result["loss"]))
print("Best trial fnal validation accuracy: {}".format(best_trial.last_result["accuracy"]))

best_trained_model = NeuralNet(22, best_trial.config["H1"], best_trial.config["H2"], best_trial.config["H3"],best_trial.config["H4"],best_trial.config["H5"],best_trial.config["H6"],101)

device="cpu"
best_trained_model.to(device)

# best_checkpoint_dir = best_trial.checkpoint.value
# print(" \n",type(best_trial.evaluated_params))
# model_state, optimizer_state = torch.load(os.path.join(best_checkpoint_dir, "checkpoint"))
# best_trained_model.load_state_dict(model_state)

num_correct, test_acc = test_accuracy(best_trained_model, device)
print("Best trial test set accuracy: {}".format(test_acc))


2021-05-16 17:52:02,275	WARNING experiment.py:294 -- No name detected on trainable. Using DEFAULT.
2021-05-16 17:52:02,291	INFO registry.py:65 -- Detected unknown callable for trainable. Converting to class.


Trial name,status,loc,H1,H2,H3,H4,H5,H6,lr
DEFAULT_ca897_00000,RUNNING,,256,64,128,512,64,256,0.0323816
DEFAULT_ca897_00001,PENDING,,128,32,2048,2048,32,256,0.0140777
DEFAULT_ca897_00002,PENDING,,128,32,512,2048,64,256,0.0812958
DEFAULT_ca897_00003,PENDING,,128,64,256,512,32,128,0.0614726
DEFAULT_ca897_00004,PENDING,,256,64,512,256,64,256,0.0456498
DEFAULT_ca897_00005,PENDING,,256,32,256,1024,32,256,0.0869788
DEFAULT_ca897_00006,PENDING,,128,64,128,512,32,128,0.0214249
DEFAULT_ca897_00007,PENDING,,256,32,1024,256,32,64,0.0435258
DEFAULT_ca897_00008,PENDING,,128,32,256,1024,32,256,0.0206187
DEFAULT_ca897_00009,PENDING,,256,32,1024,512,64,256,0.0287099


(pid=80091) 2021-05-16 17:52:04,952	ERROR function_runner.py:254 -- Runner Thread raised error.
(pid=80091) Traceback (most recent call last):
(pid=80091)   File "/opt/anaconda3/lib/python3.7/site-packages/ray/tune/function_runner.py", line 248, in run
(pid=80091)     self._entrypoint()
(pid=80091)   File "/opt/anaconda3/lib/python3.7/site-packages/ray/tune/function_runner.py", line 316, in entrypoint
(pid=80091)     self._status_reporter.get_checkpoint())
(pid=80091)   File "/opt/anaconda3/lib/python3.7/site-packages/ray/tune/function_runner.py", line 580, in _trainable_func
(pid=80091)     output = fn()
(pid=80091)   File "<ipython-input-53-d9d213d1f510>", line 24, in train_crop_yield
(pid=80091) RuntimeError: shape '[1, 176]' is invalid for input of size 198
(pid=80091) Exception in thread Thread-2:
(pid=80091) Traceback (most recent call last):
(pid=80091)   File "/opt/anaconda3/lib/python3.7/threading.py", line 926, in _bootstrap_inner
(pid=80091)     self.run()
(pid=80091)   File

Trial name,status,loc,H1,H2,H3,H4,H5,H6,lr
DEFAULT_ca897_00003,RUNNING,,128,64,256,512,32,128,0.0614726
DEFAULT_ca897_00004,RUNNING,,256,64,512,256,64,256,0.0456498
DEFAULT_ca897_00005,RUNNING,,256,32,256,1024,32,256,0.0869788
DEFAULT_ca897_00006,RUNNING,,128,64,128,512,32,128,0.0214249
DEFAULT_ca897_00007,PENDING,,256,32,1024,256,32,64,0.0435258
DEFAULT_ca897_00008,PENDING,,128,32,256,1024,32,256,0.0206187
DEFAULT_ca897_00009,PENDING,,256,32,1024,512,64,256,0.0287099
DEFAULT_ca897_00000,ERROR,,256,64,128,512,64,256,0.0323816
DEFAULT_ca897_00001,ERROR,,128,32,2048,2048,32,256,0.0140777
DEFAULT_ca897_00002,ERROR,,128,32,512,2048,64,256,0.0812958


exceptions.RayTaskError(TuneError): ray::ImplicitFunc.train_buffered() (pid=80111, ip=10.201.135.226)
  File "python/ray/_raylet.pyx", line 505, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 449, in ray._raylet.execute_task.function_executor
  File "/opt/anaconda3/lib/python3.7/site-packages/ray/_private/function_manager.py", line 556, in actor_method_executor
    return method(__ray_actor, *args, **kwargs)
  File "/opt/anaconda3/lib/python3.7/site-packages/ray/tune/trainable.py", line 173, in train_buffered
    result = self.train()
  File "/opt/anaconda3/lib/python3.7/site-packages/ray/tune/trainable.py", line 232, in train
    result = self.step()
  File "/opt/anaconda3/lib/python3.7/site-packages/ray/tune/function_runner.py", line 366, in step
    self._report_thread_runner_error(block=True)
  File "/opt/anaconda3/lib/python3.7/site-packages/ray/tune/function_runner.py", line 513, in _report_thread_runner_error
    ("Trial raised an exception. Traceback:\n{}".fo

Trial name,status,loc,H1,H2,H3,H4,H5,H6,lr
DEFAULT_ca897_00008,RUNNING,,128,32,256,1024,32,256,0.0206187
DEFAULT_ca897_00009,RUNNING,,256,32,1024,512,64,256,0.0287099
DEFAULT_ca897_00000,ERROR,,256,64,128,512,64,256,0.0323816
DEFAULT_ca897_00001,ERROR,,128,32,2048,2048,32,256,0.0140777
DEFAULT_ca897_00002,ERROR,,128,32,512,2048,64,256,0.0812958
DEFAULT_ca897_00003,ERROR,,128,64,256,512,32,128,0.0614726
DEFAULT_ca897_00004,ERROR,,256,64,512,256,64,256,0.0456498
DEFAULT_ca897_00005,ERROR,,256,32,256,1024,32,256,0.0869788
DEFAULT_ca897_00006,ERROR,,128,64,128,512,32,128,0.0214249
DEFAULT_ca897_00007,ERROR,,256,32,1024,256,32,64,0.0435258


(pid=80126) 2021-05-16 17:52:14,916	ERROR function_runner.py:254 -- Runner Thread raised error.
(pid=80126) Traceback (most recent call last):
(pid=80126)   File "/opt/anaconda3/lib/python3.7/site-packages/ray/tune/function_runner.py", line 248, in run
(pid=80126)     self._entrypoint()
(pid=80126)   File "/opt/anaconda3/lib/python3.7/site-packages/ray/tune/function_runner.py", line 316, in entrypoint
(pid=80126)     self._status_reporter.get_checkpoint())
(pid=80126)   File "/opt/anaconda3/lib/python3.7/site-packages/ray/tune/function_runner.py", line 580, in _trainable_func
(pid=80126)     output = fn()
(pid=80126)   File "<ipython-input-53-d9d213d1f510>", line 24, in train_crop_yield
(pid=80126) RuntimeError: shape '[1, 176]' is invalid for input of size 198
(pid=80126) Exception in thread Thread-2:
(pid=80126) Traceback (most recent call last):
(pid=80126)   File "/opt/anaconda3/lib/python3.7/threading.py", line 926, in _bootstrap_inner
(pid=80126)     self.run()
(pid=80126)   File

Trial name,status,loc,H1,H2,H3,H4,H5,H6,lr
DEFAULT_ca897_00000,ERROR,,256,64,128,512,64,256,0.0323816
DEFAULT_ca897_00001,ERROR,,128,32,2048,2048,32,256,0.0140777
DEFAULT_ca897_00002,ERROR,,128,32,512,2048,64,256,0.0812958
DEFAULT_ca897_00003,ERROR,,128,64,256,512,32,128,0.0614726
DEFAULT_ca897_00004,ERROR,,256,64,512,256,64,256,0.0456498
DEFAULT_ca897_00005,ERROR,,256,32,256,1024,32,256,0.0869788
DEFAULT_ca897_00006,ERROR,,128,64,128,512,32,128,0.0214249
DEFAULT_ca897_00007,ERROR,,256,32,1024,256,32,64,0.0435258
DEFAULT_ca897_00008,ERROR,,128,32,256,1024,32,256,0.0206187
DEFAULT_ca897_00009,ERROR,,256,32,1024,512,64,256,0.0287099


TuneError: ('Trials did not complete', [DEFAULT_ca897_00000, DEFAULT_ca897_00001, DEFAULT_ca897_00002, DEFAULT_ca897_00003, DEFAULT_ca897_00004, DEFAULT_ca897_00005, DEFAULT_ca897_00006, DEFAULT_ca897_00007, DEFAULT_ca897_00008, DEFAULT_ca897_00009])

In [55]:

training_input = load_data()[0]
training_ouput = load_data()[1]
test_input = load_data()[2]
test_output = load_data()[3]

x = training_input
y = training_ouput

04, 5.0922e+04, 6.3416e+04],
        [7.2630e+03, 9.6109e+03, 8.5209e+03, 9.4395e+03, 1.0662e+04, 1.0899e+04,
         1.0113e+04, 1.1160e+04, 1.1399e+04, 1.4196e+04],
        [2.2404e+04, 2.9647e+04, 2.6284e+04, 2.9118e+04, 3.2890e+04, 3.3619e+04,
         3.1195e+04, 3.4426e+04, 3.5161e+04, 4.3789e+04],
        [2.3249e+03, 3.0765e+03, 2.7276e+03, 3.0216e+03, 3.4131e+03, 3.4887e+03,
         3.2371e+03, 3.5725e+03, 3.6487e+03, 4.5441e+03],
        [4.5447e+02, 6.0139e+02, 5.3318e+02, 5.9066e+02, 6.6719e+02, 6.8197e+02,
         6.3279e+02, 6.9835e+02, 7.1325e+02, 8.8827e+02],
        [4.5447e+02, 6.0139e+02, 5.3318e+02, 5.9066e+02, 6.6719e+02, 6.8197e+02,
         6.3279e+02, 6.9835e+02, 7.1325e+02, 8.8827e+02],
        [3.0365e+05, 1.6086e+05,        nan, 1.6086e+05, 4.7329e+04, 1.5680e+05,
         1.3463e+05, 1.3892e+05, 1.9200e+05, 1.7258e+05],
        [2.0392e+04, 2.9583e+05, 7.4000e+01, 5.9159e+05, 2.6952e+05, 6.4043e+05,
         1.4296e+05, 3.7954e+05, 4.9362e+05, 4.8665e+05]

In [56]:

# Transforms

# Training data
x_tensor = torch.Tensor(x.float())
x_data = (x - x.mean())/(x.max() - x.min())
test = torch.Tensor([[0,4,5]])
x_tensor_normalized = x.normal_()
y_data = (y.normal_())

transformed_x = torch.reshape(x_tensor_normalized,(1,220))
transformed_y = y_data
y = torch.reshape(y, (1,1010))


In [57]:
# Test data
transformed_test_x = torch.Tensor(test_input.float())
transformed_test_x = transformed_test_x.normal_()
transformed_test_y = torch.Tensor(test_output.float())
transformed_test_y = transformed_test_y.normal_()

In [58]:
# Define model
class NeuralNet(nn.Module):
       def __init__(self, D_in, H1, H2, H3, H4, H5, H6, D_out):
              super(NeuralNet, self).__init__()
              self.linear1 = nn.Linear(D_in, H1)
              self.linear2 = nn.Linear(H1, H2)
              self.linear3 = nn.Linear(H2, H3)
              self.linear4 = nn.Linear(H3, H4)
              self.linear5 = nn.Linear(H4, H5)
              self.linear6 = nn.Linear(H5, H6)
              self.linear7 = nn.Linear(H6, D_out)
              self.relu = nn.ReLU()
       def forward(self, x):
              y_pred = torch.tanh(self.linear1(x))
              y_pred = self.linear2(y_pred)
              y_pred = torch.tanh(self.linear3(y_pred))
              y_pred = self.linear4(y_pred)
              y_pred = torch.tanh(self.linear5(y_pred))
              y_pred = self.linear6(y_pred)
              y_pred = torch.sigmoid(self.linear7(y_pred))
              return y_pred


In [59]:


from functools import partial
import numpy as np
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import random_split
import torchvision
import torchvision.transforms as transforms
from random import randint
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler


In [60]:


def get_accuracy(model, predicted, actual, threshold_percentage):
    num_items = len(actual)
    X = predicted.view(num_items)
    Y = actual.view(num_items)
    num_correct = torch.sum(torch.abs(X-Y)<torch.abs(threshold_percentage*Y))
    accuracy = (num_correct.item()*100.0/num_items)
    return (num_correct, accuracy)
    

def random_split_training(trainset_x, trainset_y):
       # TODO:
       # Select 2 random numbers between 1 and 10
       # Use them to index into x and y.
       # That will give us the training and validation set
       # Rest of the indices are thus our training set
       index1 = randint(0,9)
       index2 = randint(0,9)
       indices=[]
       i=0
       for i in range(0,10):
              if(i not in [index1,index2]):
                 indices.append(i)    
       
       indices = torch.tensor(indices)
       val_indices = torch.tensor([index1, index2])
       t_subset_x = torch.index_select(trainset_x, 1, indices)
       t_subset_y = torch.index_select(trainset_y, 1, indices)
       v_subset_x = torch.index_select(trainset_x, 1,val_indices)
       v_subset_y = torch.index_select(trainset_y, 1, val_indices)
       return (t_subset_x, t_subset_y, v_subset_x, v_subset_y)
# 


In [61]:
# Function to train and validate data. Save the best model based on validation loss
# 
# params:
# config: hyperparameter search space
# 
def train_crop_yield(config):
       net = NeuralNet(176, config['H1'], config['H2'], config['H3'], config["H4"], config["H5"], config["H6"],8)
       criterion = torch.nn.MSELoss()
       optimizer = torch.optim.SGD(net.parameters(), lr=config["lr"], momentum=0.9)
       # train_set = torch.cat((transformed_x, transformed_y), dim=0)
       # test_set = torch.cat((test_input, test_output), dim=0)
       
       x_by_years = transformed_x.reshape(22,10)
       y_by_years = transformed_y.reshape(101,10)
       
       train_subset_x, train_subset_y, val_subset_x, val_subset_y = random_split_training(x_by_years, y_by_years)
       
       for epoch in range(500):
              running_loss = 0.0
              epoch_steps = 0
              # Zero the accumulated gradients
              optimizer.zero_grad()
              # forward + backward + optimize
              output = net(train_subset_x.float().reshape(1,176))
              loss = criterion(output, train_subset_y.float())
              loss.backward()
              optimizer.step()
              # print statistics
              running_loss += loss.item()
              epoch_steps += 1
              if epoch%5==4:
                     print("[%d] loss: %.3f"%(epoch+1, running_loss/epoch_steps))
                     running_loss=0.0
       
       # Validation loss
       net = NeuralNet(44, config['H1'], config['H2'], config['H3'], config["H4"], config["H5"], config["H6"],2)
       val_loss = 0.0
       val_steps = 0
       total = 0
       correct = 0
       with torch.no_grad():
              val_output = net(val_subset_x.float().reshape(1,44))
              total = val_subset_y.size(0)
              correct = torch.sum(torch.abs(val_output-val_subset_y)<torch.abs(0.10*val_subset_y))
              loss = criterion(val_output, val_subset_y.float())
              val_loss += loss.cpu().numpy()
              torch.save((net.state_dict(), optimizer.state_dict()), './trained_net.dat')
              tune.report(loss=(val_loss), accuracy=(correct/total))
       print("\n Finished Training \n")
              

In [62]:
# Evaluation:
def test_accuracy(net, device='cpu'):
       net.eval()
       with torch.no_grad():
              predicted = net(transformed_test_x.float().reshape(1,22))
              return get_accuracy(net, predicted.reshape(101,1), transformed_test_y.reshape(101,1), 0.50)
       
# Define hyperparameters
config = {
       "H1":tune.sample_from(lambda _: 2**np.random.randint(7,9)),
       "H2":tune.sample_from(lambda _: 2**np.random.randint(5,7)),
       "H3":tune.sample_from(lambda _: 2**np.random.randint(7,12)),
       "H4":tune.sample_from(lambda _: 2**np.random.randint(7,12)),
       "H5":tune.sample_from(lambda _: 2**np.random.randint(5,7)),
       "H6":tune.sample_from(lambda _: 2**np.random.randint(5,9)),
       "lr":tune.loguniform(1e-2, 1e-1)
}

# Scheduler. Randomly try out combination of hyperparameters
scheduler = ASHAScheduler(
       metric="loss",
       mode="min",
       max_t=10,
       grace_period=1,
       reduction_factor=2
)

reporter=CLIReporter(metric_columns=["loss","accuracy", "training_iteration"])

result = tune.run(
       partial(train_crop_yield),
       config=config,
       num_samples=10,
       scheduler=scheduler
)

best_trial = result.get_best_trial("loss", "min", "last")

print("Best trial config: {}".format(best_trial.config))
print("Best trial final validation loss: {}".format(best_trial.last_result["loss"]))
print("Best trial fnal validation accuracy: {}".format(best_trial.last_result["accuracy"]))

best_trained_model = NeuralNet(22, best_trial.config["H1"], best_trial.config["H2"], best_trial.config["H3"],best_trial.config["H4"],best_trial.config["H5"],best_trial.config["H6"],101)

device="cpu"
best_trained_model.to(device)

# best_checkpoint_dir = best_trial.checkpoint.value
# print(" \n",type(best_trial.evaluated_params))
# model_state, optimizer_state = torch.load(os.path.join(best_checkpoint_dir, "checkpoint"))
# best_trained_model.load_state_dict(model_state)

num_correct, test_acc = test_accuracy(best_trained_model, device)
print("Best trial test set accuracy: {}".format(test_acc))


2021-05-16 17:53:49,708	WARNING experiment.py:294 -- No name detected on trainable. Using DEFAULT.
2021-05-16 17:53:49,725	INFO registry.py:65 -- Detected unknown callable for trainable. Converting to class.


Trial name,status,loc,H1,H2,H3,H4,H5,H6,lr
DEFAULT_0a9bd_00000,PENDING,,128,32,1024,128,64,32,0.095975
DEFAULT_0a9bd_00001,PENDING,,256,64,512,256,32,32,0.0165373
DEFAULT_0a9bd_00002,PENDING,,256,32,128,128,64,64,0.0154145
DEFAULT_0a9bd_00003,PENDING,,128,32,128,1024,32,32,0.0793133
DEFAULT_0a9bd_00004,PENDING,,128,32,2048,2048,32,256,0.0474008
DEFAULT_0a9bd_00005,PENDING,,128,64,2048,1024,32,32,0.0229811
DEFAULT_0a9bd_00006,PENDING,,256,64,256,128,32,128,0.0648928
DEFAULT_0a9bd_00007,PENDING,,128,32,128,512,64,256,0.0108553
DEFAULT_0a9bd_00008,PENDING,,256,32,2048,128,32,32,0.013946
DEFAULT_0a9bd_00009,PENDING,,256,64,1024,512,32,64,0.0474737


Trial name,status,loc,H1,H2,H3,H4,H5,H6,lr
DEFAULT_0a9bd_00000,RUNNING,,128,32,1024,128,64,32,0.095975
DEFAULT_0a9bd_00001,RUNNING,,256,64,512,256,32,32,0.0165373
DEFAULT_0a9bd_00002,RUNNING,,256,32,128,128,64,64,0.0154145
DEFAULT_0a9bd_00003,RUNNING,,128,32,128,1024,32,32,0.0793133
DEFAULT_0a9bd_00004,PENDING,,128,32,2048,2048,32,256,0.0474008
DEFAULT_0a9bd_00005,PENDING,,128,64,2048,1024,32,32,0.0229811
DEFAULT_0a9bd_00006,PENDING,,256,64,256,128,32,128,0.0648928
DEFAULT_0a9bd_00007,PENDING,,128,32,128,512,64,256,0.0108553
DEFAULT_0a9bd_00008,PENDING,,256,32,2048,128,32,32,0.013946
DEFAULT_0a9bd_00009,PENDING,,256,64,1024,512,32,64,0.0474737


(pid=80156) /opt/anaconda3/lib/python3.7/site-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([101, 8])) that is different to the input size (torch.Size([1, 8])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
(pid=80156)   return F.mse_loss(input, target, reduction=self.reduction)
(pid=80153) /opt/anaconda3/lib/python3.7/site-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([101, 8])) that is different to the input size (torch.Size([1, 8])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
(pid=80153)   return F.mse_loss(input, target, reduction=self.reduction)
(pid=80155) /opt/anaconda3/lib/python3.7/site-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([101, 8])) that is different to the input size (torch.Size([1, 8])). This will likely lead to incorrect results due to b

Trial name,status,loc,H1,H2,H3,H4,H5,H6,lr,iter,total time (s),loss
DEFAULT_0a9bd_00000,RUNNING,,128,32,1024,128,64,32,0.095975,,,
DEFAULT_0a9bd_00001,RUNNING,,256,64,512,256,32,32,0.0165373,,,
DEFAULT_0a9bd_00002,RUNNING,10.201.135.226:80155,256,32,128,128,64,64,0.0154145,1,3.99965,1.52055
DEFAULT_0a9bd_00003,RUNNING,,128,32,128,1024,32,32,0.0793133,,,
DEFAULT_0a9bd_00004,PENDING,,128,32,2048,2048,32,256,0.0474008,,,
DEFAULT_0a9bd_00005,PENDING,,128,64,2048,1024,32,32,0.0229811,,,
DEFAULT_0a9bd_00006,PENDING,,256,64,256,128,32,128,0.0648928,,,
DEFAULT_0a9bd_00007,PENDING,,128,32,128,512,64,256,0.0108553,,,
DEFAULT_0a9bd_00008,PENDING,,256,32,2048,128,32,32,0.013946,,,
DEFAULT_0a9bd_00009,PENDING,,256,64,1024,512,32,64,0.0474737,,,


...

Trial name,status,loc,H1,H2,H3,H4,H5,H6,lr,iter,total time (s),loss
DEFAULT_0a9bd_00004,RUNNING,,128,32,2048,2048,32,256,0.0474008,,,
DEFAULT_0a9bd_00005,RUNNING,10.201.135.226:80190,128,64,2048,1024,32,32,0.0229811,1,28.369,1.49541
DEFAULT_0a9bd_00000,TERMINATED,,128,32,1024,128,64,32,0.095975,1,5.05788,1.56665
DEFAULT_0a9bd_00001,TERMINATED,,256,64,512,256,32,32,0.0165373,1,5.88889,1.54519
DEFAULT_0a9bd_00002,TERMINATED,,256,32,128,128,64,64,0.0154145,1,3.99965,1.52055
DEFAULT_0a9bd_00003,TERMINATED,,128,32,128,1024,32,32,0.0793133,1,5.31704,1.50413
DEFAULT_0a9bd_00006,TERMINATED,,256,64,256,128,32,128,0.0648928,1,4.86922,1.54662
DEFAULT_0a9bd_00007,TERMINATED,,128,32,128,512,64,256,0.0108553,1,5.28221,1.55218
DEFAULT_0a9bd_00008,TERMINATED,,256,32,2048,128,32,32,0.013946,1,7.84006,1.51051
DEFAULT_0a9bd_00009,TERMINATED,,256,64,1024,512,32,64,0.0474737,1,10.7328,1.54593



(pid=80190) 
(pid=80190)  Finished Training 
(pid=80190) 
Result for DEFAULT_0a9bd_00005:
  accuracy: tensor(0.0297)
  date: 2021-05-16_17-54-41
  done: true
  experiment_id: 9d6d8c6dd1d146008251010b5f4a84c6
  experiment_tag: 5_H1=128,H2=64,H3=2048,H4=1024,H5=32,H6=32,lr=0.022981
  hostname: tprovpk.local
  iterations_since_restore: 1
  loss: 1.4954121112823486
  node_ip: 10.201.135.226
  pid: 80190
  time_since_restore: 28.369023084640503
  time_this_iter_s: 28.369023084640503
  time_total_s: 28.369023084640503
  timestamp: 1621153481
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 0a9bd_00005
  
(pid=80188) [295] loss: 1.007
(pid=80188) [300] loss: 1.007
(pid=80188) [305] loss: 1.007
(pid=80188) [310] loss: 1.007
(pid=80188) [315] loss: 1.007
(pid=80188) [320] loss: 1.007
(pid=80188) [325] loss: 1.007
(pid=80188) [330] loss: 1.007
(pid=80188) [335] loss: 1.007
(pid=80188) [340] loss: 1.007
(pid=80188) [345] loss: 1.007
(pid=80188) [350] loss: 1.007
(pid=80188) [355]

Trial name,status,loc,H1,H2,H3,H4,H5,H6,lr,iter,total time (s),loss
DEFAULT_0a9bd_00000,TERMINATED,,128,32,1024,128,64,32,0.095975,1,5.05788,1.56665
DEFAULT_0a9bd_00001,TERMINATED,,256,64,512,256,32,32,0.0165373,1,5.88889,1.54519
DEFAULT_0a9bd_00002,TERMINATED,,256,32,128,128,64,64,0.0154145,1,3.99965,1.52055
DEFAULT_0a9bd_00003,TERMINATED,,128,32,128,1024,32,32,0.0793133,1,5.31704,1.50413
DEFAULT_0a9bd_00004,TERMINATED,,128,32,2048,2048,32,256,0.0474008,1,36.798,1.54615
DEFAULT_0a9bd_00005,TERMINATED,,128,64,2048,1024,32,32,0.0229811,1,28.369,1.49541
DEFAULT_0a9bd_00006,TERMINATED,,256,64,256,128,32,128,0.0648928,1,4.86922,1.54662
DEFAULT_0a9bd_00007,TERMINATED,,128,32,128,512,64,256,0.0108553,1,5.28221,1.55218
DEFAULT_0a9bd_00008,TERMINATED,,256,32,2048,128,32,32,0.013946,1,7.84006,1.51051
DEFAULT_0a9bd_00009,TERMINATED,,256,64,1024,512,32,64,0.0474737,1,10.7328,1.54593


Trial name,status,loc,H1,H2,H3,H4,H5,H6,lr,iter,total time (s),loss
DEFAULT_0a9bd_00000,TERMINATED,,128,32,1024,128,64,32,0.095975,1,5.05788,1.56665
DEFAULT_0a9bd_00001,TERMINATED,,256,64,512,256,32,32,0.0165373,1,5.88889,1.54519
DEFAULT_0a9bd_00002,TERMINATED,,256,32,128,128,64,64,0.0154145,1,3.99965,1.52055
DEFAULT_0a9bd_00003,TERMINATED,,128,32,128,1024,32,32,0.0793133,1,5.31704,1.50413
DEFAULT_0a9bd_00004,TERMINATED,,128,32,2048,2048,32,256,0.0474008,1,36.798,1.54615
DEFAULT_0a9bd_00005,TERMINATED,,128,64,2048,1024,32,32,0.0229811,1,28.369,1.49541
DEFAULT_0a9bd_00006,TERMINATED,,256,64,256,128,32,128,0.0648928,1,4.86922,1.54662
DEFAULT_0a9bd_00007,TERMINATED,,128,32,128,512,64,256,0.0108553,1,5.28221,1.55218
DEFAULT_0a9bd_00008,TERMINATED,,256,32,2048,128,32,32,0.013946,1,7.84006,1.51051
DEFAULT_0a9bd_00009,TERMINATED,,256,64,1024,512,32,64,0.0474737,1,10.7328,1.54593


2021-05-16 17:54:49,821	INFO tune.py:549 -- Total run time: 60.15 seconds (59.86 seconds for the tuning loop).
Best trial config: {'H1': 128, 'H2': 64, 'H3': 2048, 'H4': 1024, 'H5': 32, 'H6': 32, 'lr': 0.02298110113208653}
Best trial final validation loss: 1.4954121112823486
Best trial fnal validation accuracy: 0.029702970758080482
Best trial test set accuracy: 20.792079207920793


In [63]:


import torch
print(torch.__version__)
import pandas as pd
print(pd.__version__)
from torch.utils.data import DataLoader,ConcatDataset
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error



1.8.1
1.2.4


In [64]:

# ## Read pesticides and fertilizer products data


In [65]:
def load_data():
       pesticides_frame = pd.read_csv("Inputs_Pesticides_Use_E_All_Data_NOFLAG.csv", engine='python')
       pesticides_frame.head
  
       # Read fertilizer data in the form of products
       fertilizers_frame = pd.read_csv("Inputs_FertilizersProduct_E_All_Data_NOFLAG.csv", engine='python')
       fertilizers_frame.head
       
       # Read the crop yield data
       yield_frame = pd.read_csv("Production_Crops_E_All_Data_NOFLAG.csv",engine='python')
       yield_frame.head

       # %<br>
       # Get the fertilizer usage in terms of agriculture

       # In[4]:     
       print("Fertilizer products",fertilizers_frame.columns)
       print("\n")
       print("Input pesticides", pesticides_frame.columns)
       
       # Filter out the data for Australia
       is_Country_Australia_Products = fertilizers_frame['Area']=="Australia"
       Australia_Fertilizers_Products = fertilizers_frame[is_Country_Australia_Products]
       # In[5]:
       is_Country_Australia_Pesticides = pesticides_frame['Area']=="Australia"
       Australia_Pesticides = pesticides_frame[is_Country_Australia_Pesticides]

       # Get the fertilizer usage for agricultural use
       # In[6]:
       is_Agricultural = Australia_Fertilizers_Products['Element']=="Agricultural Use"
       Australia_Fertilizers_Products_Agricultural = Australia_Fertilizers_Products[is_Agricultural]
       print("Fertilizers \n")
       print(Australia_Fertilizers_Products_Agricultural.columns)
       print(Australia_Fertilizers_Products_Agricultural['Item'])
       print("\n")
       print("Pesticides \n")
       print(Australia_Pesticides.columns)
       print(Australia_Pesticides['Item'])
       
       # %<br>
       # 7 years for training - 2007 to 2013<br>
       # 3 years fro validation - 2014 to 2016<br>
       # 2017 for testing
       # In[7]:
       X_ten_years_fertlizers = Australia_Fertilizers_Products_Agricultural.drop(['Area Code', 'Area', 'Item Code', 'Element Code','Element',
              'Unit','Y2002', 'Y2003', 'Y2004', 'Y2005', 'Y2006', 'Y2017'], axis=1)
       print(X_ten_years_fertlizers)
       print(X_ten_years_fertlizers.columns)
       fertilizer_2017 = Australia_Fertilizers_Products_Agricultural['Y2017']
       print("2017 \n",fertilizer_2017)

       # In[8]:
       X_ten_years_pesticides = Australia_Pesticides.drop(['Area Code', 'Area', 'Item Code', 'Element Code', 'Element',
              'Unit', 'Y1990', 'Y1991', 'Y1992', 'Y1993', 'Y1994', 'Y1995', 'Y1996', 'Y1997', 'Y1998', 'Y1999', 'Y2000', 'Y2001', 'Y2002', 'Y2003', 'Y2004', 'Y2005', 'Y2006', 'Y2017'], axis=1)
       print(X_ten_years_pesticides)
       print(X_ten_years_pesticides.columns)
       pesticides_2017 = Australia_Pesticides['Y2017']
       print("2017 \n",pesticides_2017)

       # %<br>
       # Drop Ammonia anyhydrous and Ammonia nitrate from 2007 to 2013. It has not been used in these years.
       # In[9]:
       X_ten_years_fertlizers = X_ten_years_fertlizers.drop([602, 607], axis=0)
       print(X_ten_years_fertlizers)
       print(X_ten_years_fertlizers.columns)
       fertilizer_2017 = fertilizer_2017.drop([602, 607], axis=0)
       print(fertilizer_2017)
       
       # %<br>
       # 2009 to 2013
       # ## Handle missing values from 2009 to 2013
       # In[10]:
       interpolate_2009_to_2016 = X_ten_years_fertlizers.interpolate()
       interpolate_2009_to_2016
       interpolate_2017 = fertilizer_2017.interpolate()
       print("interpolated 2017 fertilizers \n",interpolate_2017)
       # TODO: Don't just interpolate. Use the means of the previous 2  years
       # interpolate_pesticides_2017 = pesticides_2017.interpolate()
       # print("interpolated 2017 pesticides \n",interpolate_pesticides_2017)

       # %<br>
       #  Interpolate missing fertlizers data<br>
       #  Replace na with mean values of 2004,2005,2009,2010
       # ##  Replace 2007 nan values with the mean of 2004 and 2005

       # In[11]:

       interpolate_2004_2005 = Australia_Fertilizers_Products_Agricultural[['Y2004','Y2005']].copy().interpolate()
       index_names = interpolate_2004_2005.index
       index_names
       interpolate_2004_2005=interpolate_2004_2005.drop([602,607], axis=0)
       interpolate_2004_2005['mean'] = interpolate_2004_2005.mean(axis=1)
       print("Mean of 2004 and 2005 ==== \n")
       print(interpolate_2004_2005)
       pesticides_2015_2016 = Australia_Pesticides[['Y2015','Y2016']]
       pesticides_2015_2016['mean'] = pesticides_2015_2016.mean(axis=1)
       print("Mean of 2015 and 2016 === \n")
       print(pesticides_2015_2016)

       # Use this for 2007

       # In[ ]:
       # ##  Replace 2008 nan values with the mean of 2009 and 2010

       # In[12]:
       interpolate_2009_2010 = Australia_Fertilizers_Products_Agricultural[['Y2009','Y2010']].copy().interpolate()
       index_names = interpolate_2009_2010.index
       index_names

       # In[13]:
       interpolate_2009_2010['mean'] = interpolate_2009_2010.mean(axis=1)
       print("Mean of 2009 and 2010 ==== \n")
       # Use this for 2008
       interpolate_2009_2010=interpolate_2009_2010.drop([602,607], axis=0)
       print(interpolate_2009_2010)


       # ## Populate 2007 nan values with computed mean
       # In[14]:
       # Fertilizers
       i=0
       for i in range(1,16):
              interpolate_2009_to_2016.iloc[0:i,1:2] = interpolate_2004_2005['mean'].iloc[0:i]

       #Pesticides
       j=0
       for j in range(1,7):
              pesticides_2017.iloc[0:j,] = pesticides_2015_2016['mean'].iloc[0:j]

       # ## Populate 2008 nan values with computed mean
       # In[15]:
       j=0
       for j in range(1,16):
              interpolate_2009_to_2016.iloc[0:j,2:3] = interpolate_2009_2010['mean'].iloc[0:j]
                
       interpolate_2009_to_2016.iloc[15, 1] = 1437658.50
       print(interpolate_2009_to_2016.iloc[15, 1])
       interpolate_2009_to_2016.iloc[15, 2] = 171351.00

       # In[16]:
       X_ten_years_fertilizers_interpolated = interpolate_2009_to_2016
       X_ten_years_fertilizers_interpolated
       fertilizer_2017_interpolated = interpolate_2017
       print(fertilizer_2017_interpolated)

       # #### TODO: Combine pesticdes and fertilizers data
       # In[17]:
       print(X_ten_years_pesticides)
       print(X_ten_years_fertilizers_interpolated)

       pesticides_without_Item = X_ten_years_pesticides.copy().drop(['Item'], axis=1)
       pesticides_2017_wo_item = pesticides_2017
       fertilizers_without_Item = X_ten_years_fertilizers_interpolated.copy().drop(['Item'], axis=1)
       fertilizer_2017_wo_item = fertilizer_2017_interpolated
       # torch.tensor(X_ten_years_pesticides.values.astype(np.float64))
       print(X_ten_years_pesticides.values)
       print(X_ten_years_fertilizers_interpolated.values)
       print(pesticides_2017_wo_item.values)
       print(fertilizer_2017_wo_item.values)

       print(pesticides_without_Item)
       pesticides_tensor = torch.from_numpy(pesticides_without_Item.values)
       print("\t Pesticides tensor", pesticides_tensor)
       pesticides_2017_tensor = torch.from_numpy(pesticides_2017_wo_item.values)

       print(fertilizers_without_Item)
       fertilizers_tensor = torch.from_numpy(fertilizers_without_Item.values)
       fertilizer_2017_tensor = torch.from_numpy(fertilizer_2017_wo_item.values)
       print("\t Fertilizers tensor", fertilizers_tensor)

       # In[18]:
       print("Train : \n",pesticides_tensor.shape)
       print("Train: \n",fertilizers_tensor.shape)
       print("Test: \n",pesticides_2017_tensor.shape)
       print("Test: \n",fertilizer_2017_tensor.shape)

       # TODO: Normalize inputs before concatenation
       pesticides_and_fertilizers = torch.cat((pesticides_tensor, fertilizers_tensor), dim=0)
       print(pesticides_and_fertilizers.shape)
       print("\n")
       print("Training X data for 10 years: ",pesticides_and_fertilizers)
       print("\n")
       pesticides_and_fertilizers_2017 = torch.cat((pesticides_2017_tensor, fertilizer_2017_tensor), dim=0)
       print(pesticides_and_fertilizers_2017.shape)
       print("\n")
       print("Testing X data for 1 year: ",pesticides_and_fertilizers_2017)
       print("\n")

       # Training + Validation data : Input
       X_train_ten = pesticides_and_fertilizers;
       X_test = pesticides_and_fertilizers_2017

       # #### TODO: Create output vector

       # In[19]:
       print(yield_frame.columns)
       #%%
       yield_frame.head
       #%%
       Australia = yield_frame['Area']=="Australia"
       Production = yield_frame['Element']=="Production"
       Australia_yield = yield_frame[Australia]
       Australia_yield_production = Australia_yield[Production]
       print(yield_frame[Australia])
       #%%
       # Get the yield data for the year 2017 only
       Australia_yield_2017 = Australia_yield_production.drop(['Area Code', 'Area', 'Item Code', 'Element Code','Element',
              'Unit','Y1961', 'Y1962', 'Y1963', 'Y1964', 'Y1965', 'Y1966', 'Y1967', 'Y1968', 'Y1969', 'Y1970', 'Y1971', 'Y1972', 'Y1973', 
              'Y1974', 'Y1975', 'Y1976', 'Y1977', 'Y1978', 'Y1979', 'Y1980', 'Y1981', 'Y1982', 'Y1983', 'Y1984', 'Y1985', 'Y1986', 'Y1987',
              'Y1988', 'Y1989', 'Y1990', 'Y1991', 'Y1992', 'Y1993', 'Y1994', 'Y1995', 'Y1996', 'Y1997', 'Y1998', 'Y1999', 'Y2000', 'Y2001', 
              'Y2002', 'Y2003', 'Y2004', 'Y2005', 'Y2006', 'Y2007', 'Y2008', 'Y2009', 'Y2010', 'Y2011', 'Y2012', 'Y2013', 'Y2014', 'Y2015', 
              'Y2016', 'Y2018'], axis=1)
       print("\n")
       print("Test Y data: ",Australia_yield_2017)
       print("\n")
       print(Australia_yield_2017.columns)
       print(Australia_yield_2017['Item'].unique())
       print("Number of unique crops in 2017: ",len(Australia_yield_2017['Item'].unique()))

       Australia_yield_2007_to_2016 = Australia_yield_production.drop(['Area Code', 'Area', 'Item Code', 'Element Code','Element',
              'Unit','Y1961', 'Y1962', 'Y1963', 'Y1964', 'Y1965', 'Y1966', 'Y1967', 'Y1968', 'Y1969', 'Y1970', 'Y1971', 'Y1972', 'Y1973', 
              'Y1974', 'Y1975', 'Y1976', 'Y1977', 'Y1978', 'Y1979', 'Y1980', 'Y1981', 'Y1982', 'Y1983', 'Y1984', 'Y1985', 'Y1986', 'Y1987',
              'Y1988', 'Y1989', 'Y1990', 'Y1991', 'Y1992', 'Y1993', 'Y1994', 'Y1995', 'Y1996', 'Y1997', 'Y1998', 'Y1999', 'Y2000', 'Y2001', 
              'Y2002', 'Y2003', 'Y2004', 'Y2005', 'Y2006', 'Y2017','Y2018'], axis=1)
       print(Australia_yield_2007_to_2016.columns)
       print(Australia_yield_2007_to_2016['Item'].unique())
       print("\n")
       print("Train Y data: ",Australia_yield_2007_to_2016)
       print("\n")
       print("Number of unique crops from 2007 to 2016",len(Australia_yield_2007_to_2016['Item'].unique()))

       # In[20]:
       yield_without_Item = Australia_yield_2007_to_2016.copy().drop(['Item'],axis=1)
       yield_without_Item_2017 = Australia_yield_2017.copy().drop(['Item'],axis=1)

       print(yield_without_Item)
       yield_tensor = torch.from_numpy(yield_without_Item.values)
       print("Shape: ", yield_tensor.shape)
       print("\t Yield tensor", yield_tensor)
       print(yield_without_Item_2017)
       yield_tensor_2017 = torch.from_numpy(yield_without_Item_2017.values)
       print("Shape: ", yield_tensor_2017.shape)
       print("\t Yield tensor", yield_tensor_2017)

       # Training + Validation: Output
       Y_train_ten = yield_tensor
       # Test Y
       Y_test = yield_tensor_2017

       print("Train X: \n",X_train_ten.shape)
       print("Train Y: \n",Y_train_ten.shape)
       print("Test X: \n",X_test.shape)
       print("Test Y: \n",Y_test.shape)
       
       return [X_train_ten, Y_train_ten, X_test, Y_test]

In [66]:

training_input = load_data()[0]
training_ouput = load_data()[1]
test_input = load_data()[2]
test_output = load_data()[3]

x = training_input
y = training_ouput

   [2.3410e+05, 2.0284e+04, 5.7860e+03, 3.4782e+04, 1.2869e+04, 3.3525e+04,
         3.1054e+05, 4.0976e+04, 5.9000e+04, 4.8019e+04],
        [2.5426e+04, 9.8828e+04, 2.1154e+04, 1.7650e+05, 5.0525e+04, 1.2845e+05,
         4.1147e+05, 3.3659e+05, 4.0550e+05, 3.9277e+05],
        [2.8115e+05, 5.2029e+05, 7.8618e+04, 9.6196e+05, 8.9395e+04, 8.9976e+05,
         5.1240e+05, 6.3220e+05, 7.5200e+05, 7.3752e+05],
        [1.1858e+06, 7.0965e+05, 4.0956e+05, 1.0097e+06, 7.8751e+05, 1.0286e+06,
         1.2400e+05, 1.6100e+05, 1.9800e+05, 1.4450e+05],
        [1.4377e+06, 8.9901e+05, 7.4050e+05, 1.0575e+06, 1.4856e+06, 1.1575e+06,
         1.0925e+06, 1.7532e+06, 1.7720e+06, 1.5482e+06],
        [1.4377e+06, 1.7135e+05, 1.1072e+05, 2.3198e+05, 1.4856e+06, 2.7388e+05,
         1.4371e+05, 1.2240e+05, 1.0940e+05, 1.4470e+05]], dtype=torch.float64)
Train : 
 torch.Size([6, 10])
Train: 
 torch.Size([16, 10])
Test: 
 torch.Size([6])
Test: 
 torch.Size([16])
torch.Size([22, 10])


Training X data f

In [67]:

# Transforms

# Training data
x_tensor = torch.Tensor(x.float())
x_data = (x - x.mean())/(x.max() - x.min())
test = torch.Tensor([[0,4,5]])
x_tensor_normalized = x.normal_()
y_data = (y.normal_())

transformed_x = torch.reshape(x_tensor_normalized,(1,220))
transformed_y = y_data
y = torch.reshape(y, (1,1010))


In [68]:
# Test data
transformed_test_x = torch.Tensor(test_input.float())
transformed_test_x = transformed_test_x.normal_()
transformed_test_y = torch.Tensor(test_output.float())
transformed_test_y = transformed_test_y.normal_()

In [69]:
# Define model
class NeuralNet(nn.Module):
       def __init__(self, D_in, H1, H2, H3, H4, H5, H6, D_out):
              super(NeuralNet, self).__init__()
              self.linear1 = nn.Linear(D_in, H1)
              self.linear2 = nn.Linear(H1, H2)
              self.linear3 = nn.Linear(H2, H3)
              self.linear4 = nn.Linear(H3, H4)
              self.linear5 = nn.Linear(H4, H5)
              self.linear6 = nn.Linear(H5, H6)
              self.linear7 = nn.Linear(H6, D_out)
              self.relu = nn.ReLU()
       def forward(self, x):
              y_pred = torch.tanh(self.linear1(x))
              y_pred = self.linear2(y_pred)
              y_pred = torch.tanh(self.linear3(y_pred))
              y_pred = self.linear4(y_pred)
              y_pred = torch.tanh(self.linear5(y_pred))
              y_pred = self.linear6(y_pred)
              y_pred = torch.sigmoid(self.linear7(y_pred))
              return y_pred


In [70]:


from functools import partial
import numpy as np
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import random_split
import torchvision
import torchvision.transforms as transforms
from random import randint
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler


In [71]:


def get_accuracy(model, predicted, actual, threshold_percentage):
    num_items = len(actual)
    X = predicted.view(num_items)
    Y = actual.view(num_items)
    num_correct = torch.sum(torch.abs(X-Y)<torch.abs(threshold_percentage*Y))
    accuracy = (num_correct.item()*100.0/num_items)
    return (num_correct, accuracy)
    

def random_split_training(trainset_x, trainset_y):
       # TODO:
       # Select 2 random numbers between 1 and 10
       # Use them to index into x and y.
       # That will give us the training and validation set
       # Rest of the indices are thus our training set
       index1 = randint(0,9)
       index2 = randint(0,9)
       indices=[]
       i=0
       for i in range(0,10):
              if(i not in [index1,index2]):
                 indices.append(i)    
       
       indices = torch.tensor(indices)
       val_indices = torch.tensor([index1, index2])
       t_subset_x = torch.index_select(trainset_x, 1, indices)
       t_subset_y = torch.index_select(trainset_y, 1, indices)
       v_subset_x = torch.index_select(trainset_x, 1,val_indices)
       v_subset_y = torch.index_select(trainset_y, 1, val_indices)
       return (t_subset_x, t_subset_y, v_subset_x, v_subset_y)
# 


In [72]:
# Function to train and validate data. Save the best model based on validation loss
# 
# params:
# config: hyperparameter search space
# 
def train_crop_yield(config):
       net = NeuralNet(176, config['H1'], config['H2'], config['H3'], config["H4"], config["H5"], config["H6"],8)
       criterion = torch.nn.MSELoss()
       optimizer = torch.optim.SGD(net.parameters(), lr=config["lr"], momentum=0.9)
       # train_set = torch.cat((transformed_x, transformed_y), dim=0)
       # test_set = torch.cat((test_input, test_output), dim=0)
       
       x_by_years = transformed_x.reshape(22,10)
       y_by_years = transformed_y.reshape(101,10)
       
       train_subset_x, train_subset_y, val_subset_x, val_subset_y = random_split_training(x_by_years, y_by_years)
       
       for epoch in range(50000):
              running_loss = 0.0
              epoch_steps = 0
              # Zero the accumulated gradients
              optimizer.zero_grad()
              # forward + backward + optimize
              output = net(train_subset_x.float().reshape(1,176))
              loss = criterion(output, train_subset_y.float())
              loss.backward()
              optimizer.step()
              # print statistics
              running_loss += loss.item()
              epoch_steps += 1
              if epoch%5==4:
                     print("[%d] loss: %.3f"%(epoch+1, running_loss/epoch_steps))
                     running_loss=0.0
       
       # Validation loss
       net = NeuralNet(44, config['H1'], config['H2'], config['H3'], config["H4"], config["H5"], config["H6"],2)
       val_loss = 0.0
       val_steps = 0
       total = 0
       correct = 0
       with torch.no_grad():
              val_output = net(val_subset_x.float().reshape(1,44))
              total = val_subset_y.size(0)
              correct = torch.sum(torch.abs(val_output-val_subset_y)<torch.abs(0.10*val_subset_y))
              loss = criterion(val_output, val_subset_y.float())
              val_loss += loss.cpu().numpy()
              torch.save((net.state_dict(), optimizer.state_dict()), './trained_net.dat')
              tune.report(loss=(val_loss), accuracy=(correct/total))
       print("\n Finished Training \n")
              

In [73]:
# Evaluation:
def test_accuracy(net, device='cpu'):
       net.eval()
       with torch.no_grad():
              predicted = net(transformed_test_x.float().reshape(1,22))
              return get_accuracy(net, predicted.reshape(101,1), transformed_test_y.reshape(101,1), 0.50)
       
# Define hyperparameters
config = {
       "H1":tune.sample_from(lambda _: 2**np.random.randint(7,9)),
       "H2":tune.sample_from(lambda _: 2**np.random.randint(5,7)),
       "H3":tune.sample_from(lambda _: 2**np.random.randint(7,12)),
       "H4":tune.sample_from(lambda _: 2**np.random.randint(7,12)),
       "H5":tune.sample_from(lambda _: 2**np.random.randint(5,7)),
       "H6":tune.sample_from(lambda _: 2**np.random.randint(7,9)),
       "lr":tune.loguniform(1e-2, 1e-1)
}

# Scheduler. Randomly try out combination of hyperparameters
scheduler = ASHAScheduler(
       metric="loss",
       mode="min",
       max_t=10,
       grace_period=1,
       reduction_factor=2
)

reporter=CLIReporter(metric_columns=["loss","accuracy", "training_iteration"])

result = tune.run(
       partial(train_crop_yield),
       config=config,
       num_samples=10,
       scheduler=scheduler
)

best_trial = result.get_best_trial("loss", "min", "last")

print("Best trial config: {}".format(best_trial.config))
print("Best trial final validation loss: {}".format(best_trial.last_result["loss"]))
print("Best trial fnal validation accuracy: {}".format(best_trial.last_result["accuracy"]))

best_trained_model = NeuralNet(22, best_trial.config["H1"], best_trial.config["H2"], best_trial.config["H3"],best_trial.config["H4"],best_trial.config["H5"],best_trial.config["H6"],101)

device="cpu"
best_trained_model.to(device)

# best_checkpoint_dir = best_trial.checkpoint.value
# print(" \n",type(best_trial.evaluated_params))
# model_state, optimizer_state = torch.load(os.path.join(best_checkpoint_dir, "checkpoint"))
# best_trained_model.load_state_dict(model_state)

num_correct, test_acc = test_accuracy(best_trained_model, device)
print("Best trial test set accuracy: {}".format(test_acc))


2021-05-16 17:56:41,589	WARNING experiment.py:294 -- No name detected on trainable. Using DEFAULT.
2021-05-16 17:56:41,595	INFO registry.py:65 -- Detected unknown callable for trainable. Converting to class.


Trial name,status,loc,H1,H2,H3,H4,H5,H6,lr
DEFAULT_7104f_00000,PENDING,,256,32,256,1024,64,256,0.0750056
DEFAULT_7104f_00001,PENDING,,256,64,1024,256,64,256,0.0520169
DEFAULT_7104f_00002,PENDING,,128,64,512,512,64,128,0.0166839
DEFAULT_7104f_00003,PENDING,,128,64,1024,2048,32,256,0.0202657
DEFAULT_7104f_00004,PENDING,,128,64,2048,128,32,256,0.011306
DEFAULT_7104f_00005,PENDING,,256,64,128,256,64,128,0.0936394
DEFAULT_7104f_00006,PENDING,,256,64,128,1024,64,256,0.0147075
DEFAULT_7104f_00007,PENDING,,256,64,128,512,32,128,0.0403951
DEFAULT_7104f_00008,PENDING,,256,32,128,256,64,256,0.0643413
DEFAULT_7104f_00009,PENDING,,128,64,128,512,32,256,0.0780191


Trial name,status,loc,H1,H2,H3,H4,H5,H6,lr
DEFAULT_7104f_00000,RUNNING,,256,32,256,1024,64,256,0.0750056
DEFAULT_7104f_00001,RUNNING,,256,64,1024,256,64,256,0.0520169
DEFAULT_7104f_00002,RUNNING,,128,64,512,512,64,128,0.0166839
DEFAULT_7104f_00003,RUNNING,,128,64,1024,2048,32,256,0.0202657
DEFAULT_7104f_00004,PENDING,,128,64,2048,128,32,256,0.011306
DEFAULT_7104f_00005,PENDING,,256,64,128,256,64,128,0.0936394
DEFAULT_7104f_00006,PENDING,,256,64,128,1024,64,256,0.0147075
DEFAULT_7104f_00007,PENDING,,256,64,128,512,32,128,0.0403951
DEFAULT_7104f_00008,PENDING,,256,32,128,256,64,256,0.0643413
DEFAULT_7104f_00009,PENDING,,128,64,128,512,32,256,0.0780191


 0.994
(pid=80250) [49345] loss: 0.994
(pid=80249) [47425] loss: 0.994
(pid=80251) [46895] loss: 0.994
(pid=80250) [49350] loss: 0.994
(pid=80249) [47430] loss: 0.994
(pid=80251) [46900] loss: 0.994
(pid=80252) [12975] loss: 0.994
(pid=80250) [49355] loss: 0.994
(pid=80250) [49360] loss: 0.994
(pid=80249) [47435] loss: 0.994
(pid=80249) [47440] loss: 0.994
(pid=80251) [46905] loss: 0.994
(pid=80249) [47445] loss: 0.994
(pid=80251) [46910] loss: 0.994
(pid=80250) [49365] loss: 0.994
(pid=80251) [46915] loss: 0.994
(pid=80252) [12980] loss: 0.994
(pid=80250) [49370] loss: 0.994
(pid=80249) [47450] loss: 0.994
(pid=80251) [46920] loss: 0.994
(pid=80250) [49375] loss: 0.994
(pid=80250) [49380] loss: 0.994
(pid=80249) [47455] loss: 0.994
(pid=80251) [46925] loss: 0.994
(pid=80249) [47460] loss: 0.994
(pid=80250) [49385] loss: 0.994
(pid=80249) [47465] loss: 0.994
(pid=80251) [46930] loss: 0.994
(pid=80252) [12985] loss: 0.994
(pid=80251) [46935] loss: 0.994
(pid=80250) [49390] loss: 0.994
(

Trial name,status,loc,H1,H2,H3,H4,H5,H6,lr,iter,total time (s),loss
DEFAULT_7104f_00000,RUNNING,,256,32,256,1024,64,256,0.0750056,,,
DEFAULT_7104f_00001,RUNNING,,256,64,1024,256,64,256,0.0520169,,,
DEFAULT_7104f_00002,RUNNING,10.201.135.226:80250,128,64,512,512,64,128,0.0166839,1,705.857,1.36219
DEFAULT_7104f_00003,RUNNING,,128,64,1024,2048,32,256,0.0202657,,,
DEFAULT_7104f_00004,PENDING,,128,64,2048,128,32,256,0.011306,,,
DEFAULT_7104f_00005,PENDING,,256,64,128,256,64,128,0.0936394,,,
DEFAULT_7104f_00006,PENDING,,256,64,128,1024,64,256,0.0147075,,,
DEFAULT_7104f_00007,PENDING,,256,64,128,512,32,128,0.0403951,,,
DEFAULT_7104f_00008,PENDING,,256,32,128,256,64,256,0.0643413,,,
DEFAULT_7104f_00009,PENDING,,128,64,128,512,32,256,0.0780191,,,


...

[47890] loss: 0.994
(pid=80252) [47895] loss: 0.994
(pid=80252) [47900] loss: 0.994
(pid=80252) [47905] loss: 0.994
(pid=80252) [47910] loss: 0.994
(pid=80252) [47915] loss: 0.994
(pid=80252) [47920] loss: 0.994
(pid=80252) [47925] loss: 0.994
(pid=80252) [47930] loss: 0.994
(pid=80252) [47935] loss: 0.994
(pid=80252) [47940] loss: 0.994
(pid=80252) [47945] loss: 0.994
(pid=80252) [47950] loss: 0.994
(pid=80252) [47955] loss: 0.994
(pid=80252) [47960] loss: 0.994
(pid=80252) [47965] loss: 0.994
(pid=80252) [47970] loss: 0.994
(pid=80252) [47975] loss: 0.994
(pid=80252) [47980] loss: 0.994
(pid=80252) [47985] loss: 0.994
(pid=80252) [47990] loss: 0.994
(pid=80252) [47995] loss: 0.994
(pid=80252) [48000] loss: 0.994
(pid=80252) [48005] loss: 0.994
(pid=80252) [48010] loss: 0.994
(pid=80252) [48015] loss: 0.994
(pid=80252) [48020] loss: 0.994
(pid=80252) [48025] loss: 0.994
(pid=80252) [48030] loss: 0.994
(pid=80252) [48035] loss: 0.994
(pid=80252) [48040] loss: 0.994
(pid=80252) [48045] 

Trial name,status,loc,H1,H2,H3,H4,H5,H6,lr,iter,total time (s),loss
DEFAULT_7104f_00003,RUNNING,10.201.135.226:80252,128,64,1024,2048,32,256,0.0202657,1,2122.61,1.34991
DEFAULT_7104f_00000,TERMINATED,,256,32,256,1024,64,256,0.0750056,1,742.255,1.36076
DEFAULT_7104f_00001,TERMINATED,,256,64,1024,256,64,256,0.0520169,1,733.908,1.34616
DEFAULT_7104f_00002,TERMINATED,,128,64,512,512,64,128,0.0166839,1,705.857,1.36219
DEFAULT_7104f_00004,TERMINATED,,128,64,2048,128,32,256,0.011306,1,722.911,1.37144
DEFAULT_7104f_00005,TERMINATED,,256,64,128,256,64,128,0.0936394,1,423.285,1.33343
DEFAULT_7104f_00006,TERMINATED,,256,64,128,1024,64,256,0.0147075,1,586.057,1.36042
DEFAULT_7104f_00007,TERMINATED,,256,64,128,512,32,128,0.0403951,1,438.153,1.38378
DEFAULT_7104f_00008,TERMINATED,,256,32,128,256,64,256,0.0643413,1,367.952,1.34567
DEFAULT_7104f_00009,TERMINATED,,128,64,128,512,32,256,0.0780191,1,344.525,1.36894


Result for DEFAULT_7104f_00003:
  accuracy: tensor(0.0396)
  date: 2021-05-16_18-32-14
  done: true
  experiment_id: 42821d24b84a47c49b21324748b41f56
  experiment_tag: 3_H1=128,H2=64,H3=1024,H4=2048,H5=32,H6=256,lr=0.020266
  hostname: tprovpk.local
  iterations_since_restore: 1
  loss: 1.3499078750610352
  node_ip: 10.201.135.226
  pid: 80252
  time_since_restore: 2122.6106979846954
  time_this_iter_s: 2122.6106979846954
  time_total_s: 2122.6106979846954
  timestamp: 1621155734
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 7104f_00003
  


Trial name,status,loc,H1,H2,H3,H4,H5,H6,lr,iter,total time (s),loss
DEFAULT_7104f_00000,TERMINATED,,256,32,256,1024,64,256,0.0750056,1,742.255,1.36076
DEFAULT_7104f_00001,TERMINATED,,256,64,1024,256,64,256,0.0520169,1,733.908,1.34616
DEFAULT_7104f_00002,TERMINATED,,128,64,512,512,64,128,0.0166839,1,705.857,1.36219
DEFAULT_7104f_00003,TERMINATED,,128,64,1024,2048,32,256,0.0202657,1,2122.61,1.34991
DEFAULT_7104f_00004,TERMINATED,,128,64,2048,128,32,256,0.011306,1,722.911,1.37144
DEFAULT_7104f_00005,TERMINATED,,256,64,128,256,64,128,0.0936394,1,423.285,1.33343
DEFAULT_7104f_00006,TERMINATED,,256,64,128,1024,64,256,0.0147075,1,586.057,1.36042
DEFAULT_7104f_00007,TERMINATED,,256,64,128,512,32,128,0.0403951,1,438.153,1.38378
DEFAULT_7104f_00008,TERMINATED,,256,32,128,256,64,256,0.0643413,1,367.952,1.34567
DEFAULT_7104f_00009,TERMINATED,,128,64,128,512,32,256,0.0780191,1,344.525,1.36894


2021-05-16 18:32:14,361	INFO tune.py:549 -- Total run time: 2132.77 seconds (2132.54 seconds for the tuning loop).
Best trial config: {'H1': 256, 'H2': 64, 'H3': 128, 'H4': 256, 'H5': 64, 'H6': 128, 'lr': 0.09363940153806419}
Best trial final validation loss: 1.3334254026412964
Best trial fnal validation accuracy: 0.039603959769010544
Best trial test set accuracy: 24.752475247524753
